In [1]:
import numpy as np

#storage
import shelve
import zlib

# filesistem
import tempfile
import os

# pretty printing
from pprint import pformat
from IPython.display import Markdown

# multiprocessing
from multiprocessing import Pool, cpu_count

# Esegue un file .wip

In [2]:
file_name = "./simulation_data/square.wip"
workers = cpu_count()

Otteniamo quanto abbiamo ancora da fare, estraendo l'index e eliminando le simulazioni già fatte

In [3]:
with shelve.open(file_name, "r") as wip_file:
    simulations_index = wip_file[".index"]
total_sims = len(simulations_index)
# filtering out already done simulations
simulations_index={simulation:simulations_index[simulation] for simulation in simulations_index if simulations_index[simulation]['status'] != "done"}
print(f"Wip file is {100 * (1-len(simulations_index)/total_sims)}% done")

Wip file is 0.0% done


Per eseguire una simulazione devo estrarre il simulatore e lanciarlo con i parametri dati, catturando lo stdout

In [4]:
with shelve.open(file_name, "r") as wip_file:
    invocation_string = wip_file[".invocation_string"]

def run_simulation(simulator_file_name, setup):
    # running the simulator
    result = !{simulator_file_name} {invocation_string.format(**setup)}
    # splitting results and converting to float
    result = np.array([
        [float(val) for val in line.split(" ")] 
        for line in result
    ])
    result = {
        'acceptance': np.mean(result[:,0]),
        'measures': result[:,1:]
    }
    return result

Eseguendo ogni simulazione ancora da fare

In [5]:
with tempfile.TemporaryDirectory() as run_dir:
    with Pool(workers) as pool:
        future_results = {}
        # loading calls into the pool
        for simulation in simulations_index:
            simulations_index[simulation]['status'] = 'running'
            with shelve.open(file_name, "w") as wip_file:
                # marking the simulation as running
                wip_file[".index"] = simulations_index
                # preparing simulator file
                simulator_file_name = os.path.join(run_dir, simulations_index[simulation]['simulator_name'])
                if not os.path.exists(simulator_file_name):
                    with open(simulator_file_name, "wb") as simulator_file:
                        simulator_file.write(zlib.decompress(wip_file[simulations_index[simulation]['simulator_name']]))
                !chmod +x {simulator_file_name}
            print(f"Running '{simulation}'")
            # running the simulation
            future_results[simulation] = pool.apply_async(run_simulation, (
                simulator_file_name, 
                simulations_index[simulation]['setup']
            ))
        # reading results back
        for simulation in simulations_index:
            result = future_results[simulation].get()
            print(f"'{simulation}' is done.")
            # shelving result
            with shelve.open(file_name, "w") as wip_file:
                wip_file[simulation] = result
                simulations_index[simulation]['status'] = 'done'
                wip_file[".index"] = simulations_index
            del future_results[simulation]

Running 'V334-FiAa-OCnaHmXYnYnAI4XCdus01cdmXODg==.subsim'
Running 'KZIKr-eRoAxMYI9vVmXrcGR3DVs2q465c0Zs7g==.subsim'
Running 'jPMGJ7VoFyp41-D8z7562JkqFwzAxr35KnG5FQ==.subsim'
Running 'Y0zMx8aQspmOQ1nRE20a4ZwIkUnKO-MowrQ2AQ==.subsim'
Running 'QBIscaV5XuE93mbzqbxKmgCE_3HfisIB1fM56A==.subsim'
Running 'Qh7Eh5VHY8GsW0r5nkRCqyUNCQEkcsoGp6wk2A==.subsim'
Running 'YY_VYlCjiHRhwyGob0k5BN_D9kpvxxRuh0BuSw==.subsim'
Running '1VvXWf5EqjZWDOROsRdDs1VgxMSqqQxac09VnQ==.subsim'
Running 'mXg5Fb3IKR-adzZwMsP4xFrG0e4yMhR0tjcdqg==.subsim'
Running 'nMDudP5twtBqL6zWsH8KkW7K03_oYGs_yAMxsA==.subsim'
Running '-ltzAh3tG74yZFezyLNOHEXv9VrWDX2T1jLVNw==.subsim'
Running '5y5OEYcdqJlIED2tALLyk0YoIsbI4iNo9ifaOg==.subsim'
Running '8hlZSnCtW3gt6X6uW1F2e4BvH5rdtpChmDu2cQ==.subsim'
Running 'G9xHH7Jk58cGd76L-ui9vL97EhSre5LV_aFdlA==.subsim'
Running 'jDYQUct7_lbmHxrrJrRgRIx639DBSWC0tHjZwA==.subsim'
Running 'wuIxFYiwx9rWLPjp4AG1rHfGmi1ya2kaJxfJLQ==.subsim'
Running 'CFMyrSucNxzxRydEJ9K_1KLc9SDDFF44Ba7Cqg==.subsim'
Running '22cUL

In [6]:
report = "### Content of the wip file:\n"
with shelve.open(file_name, "r") as wip_file:
    for item in wip_file:
        report += f"#### {item}:\n    "
        if item.endswith(".simulator"):
            report += "<binary data>\n"
        else:
            report += pformat(wip_file[item], depth=2).replace("\n", "\n    ") + "\n"
Markdown(report)

### Content of the wip file:
#### 1vcjQwf8A4w2sYhKbxvBoWkCM1w3fZSRyniRdg==.subsim:
    {'acceptance': 0.2757914,
     'correlations': array([ 4.19071730e-01,  4.06804521e-01,  3.98847722e-01,  3.95540377e-01,
            3.89320073e-01,  3.82297889e-01,  3.78543525e-01,  3.73546365e-01,
            3.68423490e-01,  3.58628924e-01,  3.53394499e-01,  3.48218383e-01,
            3.38566825e-01,  3.34296084e-01,  3.29399625e-01,  3.32100245e-01,
            3.26815833e-01,  3.21607428e-01,  3.14795485e-01,  3.14814677e-01,
            3.08916855e-01,  3.06373211e-01,  3.00781707e-01,  2.97461292e-01,
            2.94534954e-01,  2.89549488e-01,  2.93002368e-01,  2.91685949e-01,
            2.86285091e-01,  2.83952318e-01,  2.76974667e-01,  2.75820143e-01,
            2.74740767e-01,  2.67209671e-01,  2.66421025e-01,  2.62268597e-01,
            2.60653457e-01,  2.55772580e-01,  2.52609009e-01,  2.47668034e-01,
            2.45775300e-01,  2.43349988e-01,  2.37562247e-01,  2.30008812e-01,
            2.28943983e-01,  2.26854102e-01,  2.29123221e-01,  2.26551545e-01,
            2.21180353e-01,  2.22371510e-01,  2.19609146e-01,  2.12354217e-01,
            2.13649820e-01,  2.09552600e-01,  2.02005592e-01,  2.03765056e-01,
            2.05901831e-01,  1.96269431e-01,  1.92557252e-01,  1.89635282e-01,
            1.81912166e-01,  1.81033907e-01,  1.77952904e-01,  1.75067995e-01,
            1.69575490e-01,  1.70637272e-01,  1.67857465e-01,  1.66927830e-01,
            1.64718294e-01,  1.62641702e-01,  1.63071519e-01,  1.58570717e-01,
            1.62066836e-01,  1.60367166e-01,  1.60810949e-01,  1.63451355e-01,
            1.62627453e-01,  1.57007453e-01,  1.55752830e-01,  1.56800602e-01,
            1.56726608e-01,  1.51554333e-01,  1.54860890e-01,  1.52659677e-01,
            1.44955691e-01,  1.45806883e-01,  1.39511587e-01,  1.38398871e-01,
            1.35588614e-01,  1.36022144e-01,  1.38876060e-01,  1.38396647e-01,
            1.32417186e-01,  1.35671606e-01,  1.31995738e-01,  1.34381120e-01,
            1.32395145e-01,  1.30986156e-01,  1.32629088e-01,  1.26920348e-01,
            1.27403561e-01,  1.25459749e-01,  1.16321011e-01,  1.15462161e-01,
            1.12595602e-01,  1.15908963e-01,  1.15395787e-01,  1.14688714e-01,
            1.12962093e-01,  1.09335775e-01,  1.11800816e-01,  1.13570438e-01,
            1.10853812e-01,  1.15963068e-01,  1.17109641e-01,  1.12985727e-01,
            1.10768629e-01,  1.07686767e-01,  1.04077968e-01,  1.05203610e-01,
            1.03887913e-01,  1.03739609e-01,  1.03146995e-01,  1.04934960e-01,
            1.05581145e-01,  1.02935062e-01,  9.90750754e-02,  9.55489866e-02,
            9.67193605e-02,  9.63538205e-02,  9.74064981e-02,  1.01995026e-01,
            1.03414775e-01,  1.01398837e-01,  9.93219495e-02,  9.63921770e-02,
            9.25893429e-02,  8.96915917e-02,  8.27211735e-02,  8.06545456e-02,
            7.95236413e-02,  7.93688034e-02,  8.02020244e-02,  7.86886271e-02,
            8.13662417e-02,  7.34282974e-02,  7.37661611e-02,  7.52091643e-02,
            7.47545178e-02,  8.03809802e-02,  7.70161717e-02,  7.39280671e-02,
            7.55596504e-02,  7.83462186e-02,  7.90077015e-02,  7.18019361e-02,
            7.11227379e-02,  7.39023542e-02,  7.24526115e-02,  7.06298358e-02,
            7.25375074e-02,  7.17242862e-02,  7.14898920e-02,  7.05003493e-02,
            6.68392408e-02,  6.57268919e-02,  6.82491058e-02,  6.90419333e-02,
            6.66224172e-02,  6.68461231e-02,  6.12159202e-02,  5.90960154e-02,
            6.07038888e-02,  5.98443233e-02,  6.11732612e-02,  6.30058101e-02,
            6.71985743e-02,  6.56572285e-02,  6.88493808e-02,  6.36531025e-02,
            6.78324855e-02,  6.61209863e-02,  6.62888052e-02,  6.64742106e-02,
            6.12379991e-02,  6.02396187e-02,  6.07400069e-02,  5.91049524e-02,
            5.72183061e-02,  5.54179676e-02,  5.36792152e-02,  4.95811032e-02,
            4.79006111e-02,  4.58460264e-02,  4.72749435e-02,  4.53822800e-02,
            4.47113248e-02,  4.54348959e-02,  4.79003026e-02,  4.92709341e-02,
            5.13316040e-02,  5.17480113e-02,  5.08463380e-02,  4.96490590e-02,
            4.84755089e-02,  4.75210141e-02,  5.00119330e-02,  5.50673517e-02,
            5.71509013e-02,  5.19611888e-02,  4.90601515e-02,  4.69457857e-02,
            4.36215773e-02,  4.00756897e-02,  4.03092911e-02,  4.01514775e-02,
            4.16746832e-02,  4.27730907e-02,  3.96749720e-02,  3.86048706e-02,
            3.80573866e-02,  3.48719216e-02,  3.80880077e-02,  3.59310331e-02,
            3.00772531e-02,  2.92308300e-02,  3.09217633e-02,  3.38116776e-02,
            3.37466549e-02,  3.20637631e-02,  2.80069219e-02,  2.72605477e-02,
            2.37326785e-02,  2.41509392e-02,  2.27140706e-02,  2.48783555e-02,
            2.40054075e-02,  2.53028141e-02,  2.95279582e-02,  2.87101347e-02,
            2.64692464e-02,  2.78503742e-02,  3.03641446e-02,  2.82634448e-02,
            2.70545905e-02,  2.94424130e-02,  2.68209458e-02,  2.62655935e-02,
            3.16667633e-02,  2.76947035e-02,  2.93725790e-02,  2.89701804e-02,
            3.11667776e-02,  3.04873245e-02,  3.33623984e-02,  3.08376882e-02,
            2.87835212e-02,  2.79919671e-02,  2.85064492e-02,  2.66588220e-02,
            2.97391246e-02,  3.12020342e-02,  2.59834012e-02,  2.83452236e-02,
            2.89935873e-02,  2.56901916e-02,  2.32728944e-02,  2.37193688e-02,
            2.13385820e-02,  2.06079559e-02,  2.06315001e-02,  2.13609045e-02,
            1.80692481e-02,  1.42251708e-02,  1.45996024e-02,  1.48634021e-02,
            1.77638650e-02,  2.02310286e-02,  1.60854790e-02,  1.31096229e-02,
            5.15965874e-03,  3.08886479e-03,  3.87687714e-03,  1.52324595e-03,
            7.40635480e-04,  5.16900539e-03,  5.87314794e-03,  4.48782104e-03,
           -1.72836518e-03,  3.41268253e-03,  8.67286585e-03,  6.18440977e-03,
            1.17511429e-02,  5.24497593e-03, -7.06525314e-04, -3.93349996e-04,
            1.50954663e-03,  1.23484020e-03, -2.47210327e-03, -9.53207151e-04,
           -6.00814322e-03, -4.62654376e-03, -2.41784660e-03, -2.79701204e-03,
            1.68564804e-04,  4.28165730e-03,  4.95462680e-03,  6.25728255e-03,
            7.24767516e-03,  2.23744280e-03, -3.83953667e-03, -4.73303700e-05,
            3.07799074e-03, -4.26762538e-04,  6.35814726e-04,  4.46054484e-03,
           -5.27138807e-04, -4.97605402e-03,  1.17364741e-03,  4.31170404e-03,
            3.59141324e-03,  3.89453648e-03,  4.39750416e-03,  4.34430770e-03,
            4.81062122e-03,  3.01076726e-03,  1.83548767e-03,  5.40077987e-03,
            1.52984697e-03,  1.16696056e-03,  2.33492192e-03,  2.34169704e-04,
           -1.91036580e-03,  4.82211071e-04,  2.40786897e-04, -3.49092832e-03,
           -5.93616450e-04, -1.07406622e-03, -2.84077346e-03, -1.18978074e-03,
            2.82387746e-03,  5.65960307e-04,  1.25942003e-03,  1.32177255e-03,
           -2.32099010e-03, -4.21280063e-03, -6.91332714e-03, -3.70544464e-03,
           -4.03311079e-03, -5.49113895e-03, -6.08771098e-03, -3.45073261e-03,
            7.35486536e-04,  5.08206626e-04, -1.42551827e-03, -3.76132916e-03,
            1.08072492e-03, -1.23556209e-04,  2.31969082e-04,  6.41702785e-04,
            1.74055736e-03,  2.34292537e-03,  4.93945233e-03,  1.32585825e-03,
           -1.68333885e-03, -9.10740999e-04, -5.96540399e-05,  1.07457723e-03,
           -1.40542900e-03, -2.67096794e-03, -6.81557039e-03, -9.18083429e-03,
           -7.87190030e-03, -9.08257901e-03, -1.00033331e-02, -1.24714299e-02,
           -1.07434448e-02, -8.26072513e-03, -8.95727089e-03, -6.86269466e-03,
           -5.95471367e-03, -4.02134805e-03, -9.83826227e-04, -6.12340784e-03,
           -7.08785321e-03, -6.81266305e-03, -9.01613112e-03, -1.00278544e-02,
           -6.62312531e-03, -7.09583372e-03, -6.53764848e-03, -9.60196788e-03,
           -7.51997718e-03, -3.70348669e-03, -6.70490149e-03, -4.14412606e-03,
           -4.22038977e-03, -1.97449165e-03,  4.38937157e-04,  5.55464506e-04,
           -5.22592272e-03, -1.17765151e-02, -8.16803972e-03, -9.36823923e-03,
           -3.27128816e-03, -7.57945216e-03, -7.34390304e-03, -1.12375317e-02,
           -1.39709283e-02, -1.77743629e-02, -1.80379556e-02, -1.40526974e-02,
           -1.78973984e-02, -1.68355823e-02, -1.28610450e-02, -9.04587177e-03,
           -1.05424964e-02, -8.35651835e-03, -8.71042224e-03, -1.21247900e-02,
           -9.97819633e-03, -5.13436318e-03, -6.82770960e-03, -5.26162355e-03,
           -8.58289119e-04, -6.08334677e-03, -7.74211131e-03, -7.02629120e-03,
           -6.76676463e-03, -7.21456378e-03, -1.17453086e-02, -1.65616995e-02,
           -1.17587791e-02, -1.60004517e-02, -1.92992260e-02, -1.84189241e-02,
           -1.68957802e-02, -1.64290030e-02, -1.64705255e-02, -8.17192302e-03,
           -5.53796882e-03, -9.32841113e-03, -7.59526327e-03, -8.20571571e-03,
           -1.10272418e-02, -1.15929903e-02, -6.52806540e-03, -1.10420564e-02,
           -1.21966762e-02, -1.17127917e-02, -1.05271843e-02, -1.32456439e-02,
           -1.19025319e-02, -9.17813680e-03, -1.37606964e-02, -1.95387085e-02,
           -2.48271945e-02, -2.25586649e-02, -2.48029752e-02, -2.47222413e-02,
           -2.65382239e-02, -2.67709884e-02, -2.43413734e-02, -2.50906337e-02,
           -2.47395961e-02, -3.33694657e-02, -3.67517946e-02, -3.85000817e-02,
           -3.21058623e-02, -2.76289572e-02, -3.18577202e-02, -2.45892431e-02,
           -2.82690917e-02, -3.27603105e-02, -3.76549068e-02, -3.42549044e-02,
           -3.11177382e-02, -3.19647212e-02, -3.17330070e-02, -3.34756699e-02,
           -3.08008359e-02, -3.33174326e-02, -3.33673509e-02, -3.42159378e-02,
           -3.46316499e-02, -3.23804147e-02, -3.18625583e-02, -3.21264455e-02,
           -3.67468125e-02, -3.42043852e-02, -4.04688232e-02, -4.36921091e-02,
           -4.53553800e-02, -4.49576005e-02, -3.81299363e-02, -3.82594877e-02,
           -3.81318768e-02, -3.53918142e-02, -4.20096486e-02, -4.35382496e-02]),
     'measures': array([[-0.197622, -0.329513, -0.251783, ..., -0.073938, -0.17963 ,
            -0.305159],
           [-0.521247, -0.395565, -0.497727, ..., -0.236464, -0.194783,
            -0.468623],
           [-0.036668, -0.15113 , -0.113537, ..., -0.121833, -0.242171,
            -0.216091],
           ...,
           [-0.323315, -0.250193, -0.260425, ...,  0.05077 , -0.153066,
            -0.246047],
           [-0.080493, -0.247192,  0.009307, ..., -0.26179 , -0.136646,
            -0.283364],
           [-0.269701, -0.397905, -0.371303, ..., -0.374445, -0.340133,
            -0.380577]]),
     'sigma_sqr': 0.3401164904588344}
#### Qh7Eh5VHY8GsW0r5nkRCqyUNCQEkcsoGp6wk2A==.subsim:
    {'acceptance': 0.21404536500000002,
     'measures': array([[-0.172091, -0.317658, -0.232609, ..., -0.307997, -0.175984,
            -0.228412],
           [-0.370203, -0.264615, -0.250413, ..., -0.557085, -0.496403,
            -0.399497],
           [-0.028535, -0.026636,  0.113438, ..., -0.042074, -0.147071,
            -0.123853],
           ...,
           [ 0.748533,  0.672582,  0.672114, ...,  0.82406 ,  0.862682,
             0.656088],
           [ 0.877259,  0.892601,  1.075305, ...,  0.917022,  0.785003,
             0.790407],
           [ 0.873893,  0.811545,  0.751384, ...,  0.838183,  0.810478,
             0.731328]])}
#### rw_QDA8820uuNNK1PhCjEVw4DDj2pQsE5II0XA==.subsim:
    {'acceptance': 0.313179132,
     'correlations': array([ 6.32440635e-01,  6.26989429e-01,  6.14286511e-01,  6.00799859e-01,
            6.01494857e-01,  5.97079169e-01,  5.82919085e-01,  5.67895750e-01,
            5.61169877e-01,  5.50902733e-01,  5.43296321e-01,  5.29120049e-01,
            5.24105377e-01,  5.12886009e-01,  5.09682106e-01,  5.06948081e-01,
            4.94112002e-01,  4.90613077e-01,  4.78482810e-01,  4.72091490e-01,
            4.65264003e-01,  4.54943239e-01,  4.45258657e-01,  4.35872366e-01,
            4.31579596e-01,  4.23898672e-01,  4.10447840e-01,  4.02266926e-01,
            3.97417049e-01,  3.85067576e-01,  3.80780733e-01,  3.75241847e-01,
            3.64481192e-01,  3.63956326e-01,  3.56934308e-01,  3.54538962e-01,
            3.44997212e-01,  3.41705826e-01,  3.30819605e-01,  3.28039123e-01,
            3.24237981e-01,  3.15414932e-01,  3.12761787e-01,  3.06884486e-01,
            2.89320034e-01,  2.88880031e-01,  2.79888920e-01,  2.72621230e-01,
            2.66766229e-01,  2.59836664e-01,  2.51099694e-01,  2.46665738e-01,
            2.40947539e-01,  2.35357016e-01,  2.31143939e-01,  2.28392138e-01,
            2.25268950e-01,  2.23134625e-01,  2.33957775e-01,  2.32012742e-01,
            2.26053741e-01,  2.26073831e-01,  2.20383763e-01,  2.19936780e-01,
            2.15246999e-01,  2.10259813e-01,  2.07870197e-01,  2.01732254e-01,
            2.06143221e-01,  2.02035129e-01,  2.00375014e-01,  1.99683538e-01,
            2.04580206e-01,  1.92900050e-01,  1.86485031e-01,  1.84583184e-01,
            1.71646127e-01,  1.66891328e-01,  1.68438362e-01,  1.66718918e-01,
            1.69229701e-01,  1.64470175e-01,  1.67579751e-01,  1.67343266e-01,
            1.62299260e-01,  1.59842446e-01,  1.51937719e-01,  1.45333322e-01,
            1.37143791e-01,  1.36770392e-01,  1.34217934e-01,  1.25652500e-01,
            1.21137544e-01,  1.17579184e-01,  1.17674465e-01,  1.14557933e-01,
            1.04751779e-01,  1.00357158e-01,  1.00808568e-01,  9.90435840e-02,
            9.77883413e-02,  9.72250079e-02,  8.64751829e-02,  8.64849540e-02,
            8.44854155e-02,  8.76017669e-02,  8.49695176e-02,  8.90051118e-02,
            8.17977749e-02,  7.86557642e-02,  7.98137713e-02,  7.36954942e-02,
            6.70397754e-02,  6.46228377e-02,  6.34314467e-02,  6.10115483e-02,
            6.83086300e-02,  6.89108718e-02,  7.48696153e-02,  7.23821459e-02,
            6.35954896e-02,  6.44730719e-02,  6.37129155e-02,  6.11230570e-02,
            5.53813152e-02,  4.92405671e-02,  4.77575295e-02,  5.23006615e-02,
            4.99163677e-02,  4.61902449e-02,  4.59841195e-02,  3.91854852e-02,
            3.28168426e-02,  3.42282139e-02,  3.20792272e-02,  2.83014109e-02,
            2.67869320e-02,  2.98128007e-02,  3.30164766e-02,  3.36172812e-02,
            3.52201016e-02,  2.75574730e-02,  2.99837774e-02,  2.67798766e-02,
            2.45394661e-02,  2.39581943e-02,  2.41407535e-02,  2.95007140e-02,
            2.30700978e-02,  1.93594766e-02,  2.37979889e-02,  2.93312843e-02,
            2.09167523e-02,  2.02559863e-02,  1.91055470e-02,  1.55317518e-02,
            1.76003385e-02,  2.19927457e-02,  2.68905021e-02,  2.52573024e-02,
            2.41144255e-02,  2.54865082e-02,  2.02627929e-02,  1.94795093e-02,
            2.05431387e-02,  2.02236380e-02,  1.30589347e-02,  1.15618811e-02,
            1.37533912e-02,  1.98936787e-02,  1.93670915e-02,  1.66971252e-02,
            1.29612225e-02,  1.01045335e-02,  5.67119165e-03,  6.36491426e-03,
            3.77096605e-03,  4.42387629e-03,  3.99058748e-03,  1.87281014e-03,
            3.13819088e-03, -3.61591292e-03, -7.16974687e-03, -9.16490580e-03,
           -1.29632959e-02, -1.19314547e-02, -8.85643895e-03, -1.13757821e-02,
           -6.43355783e-03, -7.49153304e-03, -1.63317301e-02, -8.59470207e-03,
           -1.63860430e-02, -1.16387425e-02, -8.68490781e-03, -5.60081392e-03,
           -6.89501008e-03, -5.24058205e-03, -1.50577226e-03, -4.58157384e-03,
            7.49517302e-04,  9.35222081e-03,  8.13996847e-03,  3.65868491e-03,
            8.68745803e-03,  1.32695188e-02,  2.12801470e-02,  1.86244622e-02,
            1.94967944e-02,  1.89516675e-02,  1.98835655e-02,  1.82609931e-02,
            1.98653962e-02,  2.19487271e-02,  1.97321440e-02,  2.49992179e-02,
            2.40766998e-02,  1.83975157e-02,  1.72708315e-02,  2.01693569e-02,
            1.61659313e-02,  1.23421478e-02,  1.00838208e-02,  1.10967058e-02,
            9.99710378e-03,  2.12705727e-02,  1.85752617e-02,  1.83349467e-02,
            1.17652662e-02,  1.61994219e-02,  1.46271414e-02,  9.14742486e-03,
            5.74161867e-03, -4.91616545e-03, -6.85057372e-03, -1.19964806e-02,
           -1.31117925e-02, -1.70483472e-02, -1.25137167e-02, -1.26005255e-02,
           -1.19230788e-02, -1.99591317e-02, -1.87999441e-02, -2.37455515e-02,
           -3.22160039e-02, -3.19056296e-02, -2.82629659e-02, -2.92289627e-02,
           -2.87543505e-02, -2.06847894e-02, -2.64417458e-02, -2.37916552e-02,
           -2.31478110e-02, -2.21568517e-02, -1.55661580e-02, -9.62821869e-03,
           -8.95968979e-03, -5.45430487e-03, -1.32905652e-03,  5.93968755e-03,
            3.52863665e-03,  1.30238173e-03,  8.30174146e-04,  3.99064203e-03,
            7.53058305e-03,  5.63241496e-03,  1.00824827e-02,  1.39020572e-02,
            1.68903011e-02,  1.75110413e-02,  1.03118231e-02,  1.22314554e-02,
            1.19062625e-02,  1.02448647e-02,  1.54469561e-02,  1.54859490e-02,
            1.73827735e-02,  1.38936178e-02,  1.63235035e-02,  1.79536812e-02,
            2.13436556e-02,  2.26329707e-02,  1.97598459e-02,  1.37990647e-02,
            1.43176529e-02,  1.53416828e-02,  1.89566479e-02,  1.30124206e-02,
            1.18922342e-02,  7.30135181e-03,  1.57892868e-02,  1.86522258e-02,
            1.96403111e-02,  1.89137315e-02,  1.83364916e-02,  2.33788398e-02,
            2.35242594e-02,  1.47976785e-02,  1.48493372e-02,  1.73742219e-02,
            1.28299263e-02,  1.69230362e-02,  1.95544250e-02,  1.23506832e-02,
            1.42420943e-02,  5.63328113e-03,  4.41250432e-03,  3.64837683e-03,
            8.20683803e-05,  8.21843466e-03,  9.09650479e-03,  8.11070774e-03,
            8.23361558e-03,  1.00215881e-02,  1.16156555e-02,  3.64163720e-03,
           -4.29160278e-03, -5.30734967e-03, -8.51933983e-03,  1.08119670e-03,
            1.64092208e-03, -1.04882828e-03, -1.72497109e-03, -2.45292069e-03,
           -3.92094514e-03, -5.82657216e-03, -9.44159978e-03, -8.73418777e-03,
           -3.49478269e-03, -4.08163498e-03, -3.32140732e-03, -3.00236586e-03,
           -1.30884416e-02, -1.52422374e-02, -1.40427088e-02, -1.64483000e-02,
           -2.46669840e-02, -2.87731600e-02, -1.54594777e-02, -1.12910837e-02,
           -1.26500970e-02, -1.21289118e-02, -1.14168655e-02, -1.33947971e-02,
           -1.49165349e-02, -1.31679705e-02, -1.29044823e-02, -1.81960692e-02,
           -1.92769349e-02, -1.88958310e-02, -9.39631919e-03, -1.04926920e-02,
           -7.59361664e-03, -3.11731712e-04, -5.88063244e-03, -1.95809870e-04,
            4.90534477e-03,  2.53935269e-03, -2.67647837e-03, -2.54603516e-03,
            1.97766814e-03,  7.87283263e-03, -1.16842532e-03,  1.83527732e-03,
           -6.86084619e-03, -5.20948214e-03, -6.25491174e-03, -7.74197483e-03,
           -5.89990699e-03, -1.40150635e-02, -1.36083819e-02, -1.53285519e-02,
           -1.88762548e-02, -1.85889530e-02, -1.76000053e-02, -2.29970880e-02,
           -3.08589351e-02, -2.97708055e-02, -3.20157810e-02, -3.23647450e-02,
           -2.91008172e-02, -3.29582392e-02, -2.42248560e-02, -2.78067677e-02,
           -2.76607268e-02, -2.34676980e-02, -2.01777518e-02, -1.96055313e-02,
           -2.31953105e-02, -2.51218644e-02, -2.26845178e-02, -1.77815209e-02,
           -2.05214513e-02, -1.98385981e-02, -1.86400650e-02, -2.00649795e-02,
           -1.64121237e-02, -1.16798098e-02, -1.07401129e-02, -1.67815625e-02,
           -1.31299740e-02, -1.03314804e-02, -1.22699431e-02, -7.08199298e-03,
           -1.05686502e-02, -1.59688075e-02, -1.81112659e-02, -1.56304498e-02,
           -1.13757028e-02, -1.04208474e-02, -7.56945403e-03, -1.12810436e-02,
           -1.23414432e-02, -7.25110426e-03, -6.33904226e-03, -3.72427434e-03,
           -7.29981771e-03, -9.98913734e-03, -9.20573944e-03, -2.88461392e-03,
           -3.80420487e-03,  7.45461875e-03,  1.19018516e-02,  7.92617176e-03,
            1.29198302e-02,  1.46203755e-02,  1.62988533e-02,  2.19051986e-02,
            2.71014088e-02,  1.95375843e-02,  1.75881094e-02,  9.95085712e-03,
            1.09789850e-02,  7.53355709e-03,  1.02343803e-02,  6.04052056e-03,
            2.73206451e-03,  6.10431346e-03,  7.01251333e-03,  6.43554858e-03,
            4.86977776e-03,  2.19555376e-03, -9.75575896e-04, -3.74977926e-03,
            6.07195727e-04,  3.72401735e-03, -6.07855289e-03, -1.44746095e-03,
            3.33562597e-03, -2.98325145e-03, -1.01246801e-02, -6.48026818e-03,
           -2.42641218e-03, -3.21723211e-03, -3.71116592e-03,  1.06618774e-03,
           -8.60238365e-03, -7.05929644e-04, -1.82906958e-04, -4.86092829e-03,
           -1.24217208e-02, -1.46705005e-02, -1.43690807e-02, -1.21144067e-02,
           -1.60097427e-02, -1.39531977e-02, -1.75047423e-02, -2.10019510e-02,
           -1.59743528e-02, -1.85726663e-02, -2.14598220e-02, -2.45071952e-02,
           -2.66849955e-02, -2.98453845e-02, -2.95988167e-02, -2.72847967e-02,
           -2.54652249e-02, -2.54689352e-02, -2.03576757e-02, -1.61055708e-02,
           -1.93182582e-02, -2.70342175e-02, -2.82724290e-02, -2.63304199e-02,
           -2.61878236e-02, -2.25984195e-02, -2.44632464e-02, -1.90334071e-02,
           -1.72887032e-02, -1.59278030e-02, -1.79403121e-02, -6.99238279e-03,
           -1.00612851e-02, -1.10046408e-02, -1.08478719e-02, -1.68146540e-02,
           -1.81554728e-02, -1.58942563e-02, -1.53248763e-02, -1.31272954e-02]),
     'measures': array([[-0.216431, -0.438687, -0.320888, ..., -0.294226, -0.179955,
            -0.463425],
           [-0.019482, -0.177332, -0.290608, ...,  0.290373, -0.032299,
            -0.164518],
           [ 1.17084 ,  1.235577,  1.40515 , ...,  0.549542,  0.996258,
             0.984493],
           ...,
           [-0.270694, -0.372228, -0.319031, ..., -0.21074 , -0.338183,
            -0.299902],
           [ 0.878542,  0.910337,  0.931802, ...,  0.579768,  0.691238,
             0.892418],
           [ 0.923597,  1.070362,  0.633506, ...,  0.815443,  0.763254,
             0.87837 ]]),
     'sigma_sqr': 0.5475641169588198}
#### FWu3AXxD1Cl6dNVuBqhEIPXoNF22KL-XVxN4ng==.subsim:
    {'DeltaE': (0.23807924593583216, 0.002880063428161469),
     'acceptance': 0.31323443100000004,
     'correlations': array([ 6.01880603e-01,  5.87257230e-01,  5.80140792e-01,  5.69982571e-01,
            5.55608971e-01,  5.46730653e-01,  5.35223987e-01,  5.24572393e-01,
            5.19320057e-01,  5.08089918e-01,  5.00753651e-01,  4.92693662e-01,
            4.82624463e-01,  4.69935525e-01,  4.59743393e-01,  4.47791515e-01,
            4.33316837e-01,  4.23150496e-01,  4.14725826e-01,  4.06590413e-01,
            3.97500236e-01,  3.86723942e-01,  3.76562187e-01,  3.66315524e-01,
            3.59130579e-01,  3.52562085e-01,  3.41298221e-01,  3.29643112e-01,
            3.20444890e-01,  3.13825414e-01,  3.04205322e-01,  2.99668985e-01,
            2.93625255e-01,  2.88152468e-01,  2.82269260e-01,  2.76616321e-01,
            2.72609370e-01,  2.71885778e-01,  2.63435938e-01,  2.62609631e-01,
            2.59378747e-01,  2.54005597e-01,  2.45736098e-01,  2.37321512e-01,
            2.33497821e-01,  2.26762145e-01,  2.27502458e-01,  2.23904162e-01,
            2.20597992e-01,  2.16412410e-01,  2.06674070e-01,  2.00629222e-01,
            2.00151864e-01,  1.94563420e-01,  1.94958075e-01,  1.89751015e-01,
            1.85966531e-01,  1.84196809e-01,  1.80377593e-01,  1.75036417e-01,
            1.72185623e-01,  1.68598306e-01,  1.65956164e-01,  1.62811754e-01,
            1.61223503e-01,  1.57536757e-01,  1.53982342e-01,  1.55323496e-01,
            1.49452273e-01,  1.45566445e-01,  1.38847871e-01,  1.35380037e-01,
            1.39046892e-01,  1.33049679e-01,  1.23691452e-01,  1.17083163e-01,
            1.07781428e-01,  1.01408830e-01,  9.83760689e-02,  1.01012363e-01,
            9.61474697e-02,  8.64794105e-02,  8.81346930e-02,  8.26238398e-02,
            8.59805538e-02,  8.38983591e-02,  8.59335260e-02,  7.79482039e-02,
            7.31282685e-02,  7.07855920e-02,  7.27899694e-02,  7.25680740e-02,
            7.07107696e-02,  6.64574432e-02,  5.93553372e-02,  5.66334102e-02,
            5.53282469e-02,  5.16674233e-02,  4.95491882e-02,  5.03596172e-02,
            4.50592632e-02,  4.43995315e-02,  4.07219114e-02,  3.67575463e-02,
            3.31793983e-02,  3.63564977e-02,  4.31198364e-02,  4.86462785e-02,
            4.39378173e-02,  3.97115783e-02,  3.62455734e-02,  3.23178060e-02,
            3.27313950e-02,  2.73274823e-02,  2.94341906e-02,  3.30077705e-02,
            2.53900354e-02,  2.48430121e-02,  2.27146930e-02,  2.12841321e-02,
            1.74557865e-02,  1.76234091e-02,  1.45987081e-02,  1.78655453e-02,
            1.51544092e-02,  1.18837416e-02,  7.50486585e-03,  5.66074436e-03,
            4.43687804e-03,  2.11505861e-03, -1.16589051e-04, -2.96126594e-03,
           -2.93015065e-03, -4.39265597e-03, -8.60729751e-03, -8.58144523e-03,
           -6.18467109e-03, -8.40932273e-03, -4.74997214e-03, -3.32986628e-03,
           -1.41873410e-03, -9.02654550e-03, -1.03259870e-02, -1.22300707e-02,
           -1.60826422e-02, -1.91104578e-02, -2.15383899e-02, -2.29075269e-02,
           -2.14204347e-02, -1.92727557e-02]),
     'measures': array([[-0.374655, -0.460602, -0.521731, ...,  0.230068, -0.183346,
            -0.380221],
           [-0.32217 , -0.292955, -0.286522, ..., -0.05078 , -0.340881,
            -0.431458],
           [-0.666354, -0.805076, -0.673709, ..., -0.223081, -0.222472,
            -0.504752],
           ...,
           [-1.198893, -1.183569, -1.245452, ..., -1.079331, -0.991076,
            -1.045196],
           [-0.036675, -0.154373, -0.276699, ..., -0.066298,  0.172738,
            -0.133697],
           [ 0.913151,  1.061048,  1.00036 , ...,  0.942585,  0.803944,
             0.648995]]),
     'meff': (0.7056947957191045, 0.008536845639801222),
     'sigma_sqr': 0.41046177798375233}
#### Y0zMx8aQspmOQ1nRE20a4ZwIkUnKO-MowrQ2AQ==.subsim:
    {'acceptance': 0.242475137,
     'measures': array([[-0.056127, -0.114942, -0.053732, ..., -0.392939, -0.211612,
            -0.116011],
           [-0.738253, -0.844303, -0.602389, ..., -0.75182 , -0.861605,
            -0.760103],
           [-0.520789, -0.570925, -0.584112, ..., -0.931657, -0.608179,
            -0.550723],
           ...,
           [-1.413448, -1.457225, -1.387766, ..., -1.175865, -1.132616,
            -1.469028],
           [-2.074622, -1.951236, -1.794948, ..., -2.094043, -2.203116,
            -2.117203],
           [-1.836018, -1.897893, -1.910849, ..., -1.737705, -1.857834,
            -1.967421]])}
#### yARqE2yrPU6UneAaFj8NpcLBFFtQC4hnRTSeGQ==.subsim:
    {'acceptance': 0.30006655,
     'correlations': array([ 5.07457759e-01,  4.97761989e-01,  4.88904224e-01,  4.76432406e-01,
            4.62606430e-01,  4.52332969e-01,  4.40608655e-01,  4.36705698e-01,
            4.27242865e-01,  4.26245467e-01,  4.19127911e-01,  4.12330722e-01,
            4.01108818e-01,  3.99009521e-01,  3.88799954e-01,  3.79543583e-01,
            3.70753827e-01,  3.61970016e-01,  3.54071731e-01,  3.45657115e-01,
            3.39704418e-01,  3.35332592e-01,  3.28470621e-01,  3.27598936e-01,
            3.23945342e-01,  3.16908102e-01,  3.16499919e-01,  3.10930465e-01,
            3.11659219e-01,  3.08975468e-01,  3.00571151e-01,  2.93943357e-01,
            2.91105489e-01,  2.85586013e-01,  2.79080007e-01,  2.70856255e-01,
            2.66231563e-01,  2.63448976e-01,  2.65700067e-01,  2.57588518e-01,
            2.50898371e-01,  2.45092049e-01,  2.31838229e-01,  2.27470096e-01,
            2.25442745e-01,  2.19213737e-01,  2.15608694e-01,  2.08461490e-01,
            2.08357214e-01,  2.06464710e-01,  2.12138319e-01,  2.03896203e-01,
            1.98393470e-01,  1.93162650e-01,  1.90705848e-01,  1.90434934e-01,
            1.86584631e-01,  1.84208830e-01,  1.77979424e-01,  1.74095014e-01,
            1.64414922e-01,  1.59861294e-01,  1.57453111e-01,  1.56515370e-01,
            1.50881651e-01,  1.53028752e-01,  1.51168794e-01,  1.48221567e-01,
            1.47225036e-01,  1.42094558e-01,  1.43328309e-01,  1.41711289e-01,
            1.40808770e-01,  1.34496796e-01,  1.34532975e-01,  1.33182716e-01,
            1.28652943e-01,  1.23083007e-01,  1.18409041e-01,  1.18345572e-01,
            1.13602940e-01,  1.15665437e-01,  1.16592091e-01,  1.15411582e-01,
            1.05450425e-01,  1.09824756e-01,  1.06511995e-01,  1.01418598e-01,
            1.06170502e-01,  1.09395304e-01,  1.08836300e-01,  1.05037308e-01,
            1.04564719e-01,  1.07844719e-01,  1.07618346e-01,  1.06606212e-01,
            1.00309079e-01,  9.09789940e-02,  8.95393940e-02,  9.09381426e-02,
            9.63548195e-02,  1.02872875e-01,  1.01639229e-01,  1.01234212e-01,
            1.02895584e-01,  1.02011579e-01,  9.42956275e-02,  9.31470012e-02,
            9.34736180e-02,  8.93310034e-02,  8.16618708e-02,  7.68601126e-02,
            7.90771415e-02,  7.67234207e-02,  7.30061565e-02,  7.29930641e-02,
            6.95186340e-02,  6.83350493e-02,  6.01040126e-02,  6.54924206e-02,
            6.55684240e-02,  6.59452933e-02,  7.36681654e-02,  7.43134763e-02,
            7.23159509e-02,  7.41504248e-02,  7.89847359e-02,  8.09483606e-02,
            7.59122805e-02,  7.76805732e-02,  7.93122855e-02,  7.29441436e-02,
            7.48808173e-02,  7.35611543e-02,  6.91199995e-02,  6.81481994e-02,
            6.47459544e-02,  6.69253714e-02,  6.73359508e-02,  6.19133890e-02,
            5.29971069e-02,  5.17104285e-02,  4.43031488e-02,  4.26963917e-02,
            4.20939184e-02,  4.36417889e-02,  4.04353558e-02,  4.10482884e-02,
            4.12039972e-02,  4.20289252e-02,  4.60013522e-02,  4.48167572e-02,
            4.44832026e-02,  4.61425333e-02,  4.65664835e-02,  4.80559967e-02,
            5.24679485e-02,  5.49933205e-02,  5.47682300e-02,  4.74301984e-02,
            4.00181205e-02,  3.77530516e-02,  3.89513695e-02,  3.51775008e-02,
            3.90347838e-02,  3.46395740e-02,  3.46539127e-02,  2.77400057e-02,
            2.28958847e-02,  2.46031085e-02,  3.08202787e-02,  3.68400437e-02,
            3.31440040e-02,  3.20207564e-02,  3.15968711e-02,  3.09070619e-02,
            3.26191966e-02,  3.07270069e-02,  2.26592714e-02,  2.22609672e-02,
            2.86757329e-02,  2.85435412e-02,  2.71518849e-02,  2.75723754e-02,
            2.61434210e-02,  3.14423602e-02,  2.77916301e-02,  3.00145662e-02,
            2.36148105e-02,  2.08336949e-02,  1.91888695e-02,  1.66419037e-02,
            1.73563274e-02,  2.14502508e-02,  2.72673853e-02,  2.72397692e-02,
            2.89331239e-02,  3.16252334e-02,  3.13169812e-02,  3.16734379e-02,
            2.77764937e-02,  2.49945897e-02,  2.06646331e-02,  1.82850542e-02,
            1.97827293e-02,  2.18469988e-02,  2.39619599e-02,  1.97113440e-02,
            1.60187794e-02,  1.43665512e-02,  1.39127621e-02,  1.38775425e-02,
            9.98990855e-03,  5.42136547e-03,  5.40028380e-03,  4.95374915e-03,
            9.53594798e-03,  4.67889434e-03, -1.63557110e-03, -6.65731146e-03,
           -6.40086561e-03, -8.57629586e-03, -1.55912648e-02, -1.26575922e-02,
           -1.56325705e-02, -1.40628637e-02, -1.26894078e-02, -1.55830615e-02,
           -1.14742683e-02, -1.47426068e-02, -2.01354464e-02, -2.23354281e-02,
           -2.62127694e-02, -2.72167037e-02, -2.82018999e-02, -2.60472351e-02,
           -2.65815006e-02, -2.72214233e-02, -2.27896703e-02, -2.18449492e-02,
           -2.02959168e-02, -1.22672848e-02, -1.05478544e-02, -6.28973587e-03,
           -1.02431431e-02, -1.31082636e-02, -9.62214420e-03, -6.36615549e-03,
           -8.40578925e-03, -6.63590086e-03, -3.52548191e-03, -8.67597700e-03,
           -1.49225436e-03, -2.68787926e-03,  1.07314487e-03, -1.37547926e-03,
            3.20695086e-04,  1.11958216e-03,  1.58365651e-03,  3.78947075e-03,
            2.22581307e-03, -1.93064344e-03, -3.54749918e-03, -3.41951484e-03,
           -2.55322292e-03,  2.07312099e-03, -2.72017876e-03, -1.76400154e-03,
           -6.34287393e-06,  2.66813976e-03,  2.58868107e-03,  7.45452333e-03,
            6.78040101e-03,  1.42662603e-02,  8.32703285e-03,  6.11854059e-03,
            8.77402081e-03,  8.41093934e-03,  6.59234150e-03,  1.24032311e-02,
            1.33661224e-02,  1.52455882e-02,  1.80522318e-02,  1.99698652e-02,
            2.56219190e-02,  2.38602704e-02,  2.33178857e-02,  3.27294723e-02,
            2.50053141e-02,  2.32004711e-02,  2.51300731e-02,  2.57108950e-02,
            1.78176579e-02,  1.47729716e-02,  1.26175265e-02,  1.78768714e-02,
            1.79598362e-02,  1.53892730e-02,  1.58593448e-02,  1.28249077e-02,
            1.84054025e-02,  1.49754699e-02,  1.79232554e-02,  1.89091603e-02,
            1.82223043e-02,  1.89451605e-02,  1.35290061e-02,  1.72187296e-02,
            1.50152650e-02,  1.38927261e-02,  1.37595011e-02,  1.80317623e-02,
            2.12166828e-02,  2.16704408e-02,  1.97529782e-02,  1.77272156e-02,
            1.79678533e-02,  1.72478829e-02,  1.93340397e-02,  2.42999893e-02,
            2.98470553e-02,  3.07885000e-02,  2.41082341e-02,  2.28511123e-02,
            2.25007904e-02,  2.49555420e-02,  3.13775447e-02,  3.46038852e-02,
            3.50654920e-02,  3.89951020e-02,  3.79877632e-02,  3.98179606e-02,
            3.87461790e-02,  3.86466898e-02,  3.98755508e-02,  4.24180614e-02,
            4.11301617e-02,  3.98729124e-02,  3.70635886e-02,  3.68524392e-02,
            3.32893759e-02,  2.96930254e-02,  2.66612670e-02,  2.27630998e-02,
            2.02972849e-02,  2.67120806e-02,  2.25777209e-02,  2.08024877e-02,
            2.21440788e-02,  1.96254060e-02,  1.53842220e-02,  1.63202661e-02,
            1.24119557e-02,  4.93266574e-03,  3.05792619e-03,  6.00717751e-04,
            2.26241802e-03,  3.98090672e-03,  9.43660350e-03,  1.37109621e-02,
            8.41974369e-03,  4.79635810e-03,  6.79322572e-03,  5.65915227e-03,
            5.11819069e-03, -1.40437206e-03, -3.19479346e-03, -1.10215189e-03,
           -3.62444026e-03, -8.24113968e-03, -7.99147482e-03, -6.80337444e-03,
           -8.67473402e-03, -7.64685014e-03, -5.43004288e-03, -8.74611988e-03,
           -1.53328873e-02, -1.32786190e-02, -6.86497889e-03, -6.89871605e-03,
           -3.96098882e-03, -4.24721897e-03,  8.60345908e-04, -1.75339937e-03,
           -3.97648832e-03, -1.79228880e-03, -7.27401267e-03, -1.20367267e-02,
           -5.05863195e-03, -3.15668552e-03, -7.22031849e-03, -5.61891482e-03,
           -8.99394247e-03, -4.99008924e-03, -3.98181784e-03, -2.85079748e-03,
           -1.92110760e-03,  8.95187178e-04,  8.24409358e-04,  4.51452307e-04,
            2.35152468e-05, -7.83109591e-03, -1.03178645e-02, -9.82099154e-03,
           -7.32199765e-03, -8.60758853e-03, -8.41589900e-03, -3.20009668e-03,
           -5.03677163e-03, -2.46886380e-03, -2.31575398e-03,  1.65107042e-03,
            3.02728902e-03, -1.26912568e-03, -8.22190214e-03, -8.39204896e-03,
           -3.39896898e-03,  8.10999273e-04,  1.93569909e-04, -5.05952315e-03,
           -8.76862289e-03, -2.82366510e-03, -4.10693791e-03, -4.37199210e-03,
           -4.30273653e-03, -4.01604579e-03, -6.50536886e-03, -3.91802914e-03,
           -8.33658329e-03, -1.14231411e-02, -1.26956629e-02, -1.39092848e-02,
           -1.27417384e-02, -9.07064568e-03, -1.07042401e-02, -1.40611776e-02,
           -1.96930216e-02, -1.88447793e-02, -1.85306966e-02, -1.96383920e-02,
           -1.78831217e-02, -2.15872395e-02, -3.02901795e-02, -2.97436273e-02,
           -3.04903303e-02, -2.64786303e-02, -2.47558262e-02, -2.46386566e-02,
           -2.54256215e-02, -2.60930527e-02, -2.77996346e-02, -2.52396067e-02,
           -2.24246852e-02, -1.94018943e-02, -1.77361580e-02, -1.60319978e-02,
           -1.88930161e-02, -1.02486478e-02, -1.03076054e-02, -7.73786438e-03,
           -8.64283686e-03, -1.42762339e-02, -1.12050008e-02, -1.55817537e-02,
           -1.33180071e-02, -1.72238724e-02, -2.40749256e-02, -1.62914139e-02,
           -1.12832655e-02, -1.49591157e-02, -1.04310466e-02, -1.13856119e-02,
           -9.98774456e-03, -5.88120665e-03, -2.62506728e-03, -4.21640037e-03,
           -5.58938215e-03, -9.12321531e-03, -8.25956599e-03, -6.58248259e-03,
            7.77610425e-04, -4.56215219e-03,  5.10199559e-05,  3.31394515e-03,
            1.89356720e-04,  3.95849451e-03,  2.58431631e-03,  3.48188099e-03,
            3.07605988e-03,  7.10691144e-04,  6.24451308e-03,  1.80422320e-03,
           -1.53006315e-04,  4.45413362e-04,  7.98687632e-03,  4.32942780e-04,
           -2.89842736e-03, -4.10556290e-03,  1.40149129e-03,  7.93815524e-04]),
     'measures': array([[-0.990091, -1.164431, -1.308857, ..., -1.036424, -1.023179,
            -0.97675 ],
           [ 0.962689,  0.94269 ,  0.982925, ...,  1.379219,  1.096632,
             0.819753],
           [ 0.951407,  0.957691,  0.817552, ...,  0.924964,  0.786678,
             0.958438],
           ...,
           [-0.092554,  0.011183, -0.061816, ..., -0.373754, -0.387208,
            -0.198592],
           [-0.85584 , -0.771336, -0.903485, ..., -0.826768, -1.01682 ,
            -0.844019],
           [ 0.028634,  0.045196, -0.136293, ..., -0.071857, -0.175102,
             0.104608]]),
     'sigma_sqr': 0.48062716617008333}
#### GJgFxnlC1XVVPS4MCwDgrTLJswNUQDaOzSkt5g==.subsim:
    {'acceptance': 0.29435552,
     'measures': array([[-0.286315, -0.400628, -0.243522, ..., -0.371238, -0.176643,
            -0.238697],
           [ 0.559359,  0.226305,  0.069981, ...,  0.813273,  0.833461,
             0.734331],
           [-0.19828 , -0.152177, -0.314021, ..., -0.251951, -0.151915,
            -0.310796],
           ...,
           [-0.407731, -0.678754, -0.586019, ..., -0.261879, -0.277042,
            -0.219734],
           [-0.323849, -0.100641, -0.075855, ..., -0.618553, -0.568404,
            -0.478684],
           [-0.314655, -0.381097, -0.387656, ..., -0.283382, -0.162779,
            -0.223404]])}
#### .setup:
    {'V_tilde': ['if(y - floor(y) < 0.5) return 0; else return -1;'],
     'beta_tilde': [30],
     'd': [0.1,
           1.1473684210526316,
           2.194736842105263,
           3.2421052631578946,
           4.289473684210526,
           5.336842105263157,
           6.384210526315789,
           7.43157894736842,
           8.478947368421052,
           9.526315789473683,
           10.573684210526315,
           11.621052631578946,
           12.668421052631578,
           13.71578947368421,
           14.76315789473684,
           15.810526315789472,
           16.857894736842105,
           17.905263157894737,
           18.95263157894737,
           20.0],
     'eta': [0.03],
     'lambda': [0.04],
     'local_steps': [5],
     'measuration_to_take': [1000],
     'measure_every': [300],
     'seed': [42],
     'step_size': [0.6]}
#### P32bwPqVQv3OZqMKA6zGIZN5csUuPXaj9IH7ZA==.subsim:
    {'DeltaE': (0.11735538272023706, 0.0008948850949918433),
     'acceptance': 0.300107444,
     'correlations': array([0.56430636, 0.55068203, 0.53844318, 0.530131  , 0.52542051,
           0.52166941, 0.51261774, 0.51015177, 0.50603994, 0.49814426,
           0.49087368, 0.48574062, 0.48460365, 0.4800542 , 0.47858385,
           0.47162708, 0.47096838, 0.46194707, 0.44922669, 0.4437141 ,
           0.4385892 , 0.43092324, 0.42465406, 0.42098541, 0.4185159 ,
           0.41466248, 0.4049432 , 0.3992964 , 0.39133106, 0.38518569,
           0.3819116 , 0.37985795, 0.37898066, 0.37569239, 0.37469611,
           0.37518975, 0.37078826, 0.36852544, 0.36550913, 0.36170599,
           0.35789687, 0.3503214 , 0.34392451, 0.33934325, 0.33564479,
           0.33559756, 0.33032416, 0.32721277, 0.32365627, 0.32455968,
           0.3215363 , 0.31784727, 0.31597049, 0.31447136, 0.30553083,
           0.3006912 , 0.29505966, 0.2920372 , 0.28591787, 0.27975442,
           0.27939454, 0.2723338 , 0.27219096, 0.27036872, 0.26756094,
           0.26174771, 0.25719512, 0.2589014 , 0.25704614, 0.25442132,
           0.25324807, 0.25122015, 0.246726  , 0.24904953, 0.24088607,
           0.23937198, 0.23520788, 0.23225543, 0.23245614, 0.23142531,
           0.22672486, 0.2227995 , 0.2200939 , 0.21855642, 0.21156443,
           0.20863033, 0.20705227, 0.20292037, 0.19496657, 0.19111824,
           0.18753633, 0.18478238, 0.17953974, 0.18071599, 0.17738258,
           0.17385982, 0.17180342, 0.16926455, 0.16639258, 0.164029  ,
           0.16273704, 0.15800583, 0.1577801 , 0.15895323, 0.15218266,
           0.15196744, 0.14813357, 0.14381244, 0.14299657, 0.14094295,
           0.14391655, 0.1436795 , 0.14165434, 0.13698813, 0.13407898,
           0.13287259, 0.13430369, 0.13305048, 0.13829965, 0.13845917,
           0.13700302, 0.1359152 , 0.13618473, 0.13830395, 0.1359531 ,
           0.1329467 , 0.12908539, 0.13182878, 0.13645669, 0.13436709,
           0.13318399, 0.1312627 , 0.12963996, 0.13144247, 0.12897526,
           0.13209973, 0.13201979, 0.13234566, 0.13233684, 0.12875407,
           0.1259556 , 0.12961682, 0.12792881, 0.12806118, 0.12707362,
           0.12770858, 0.12823065, 0.12684872, 0.12323052, 0.12877886]),
     'measures': array([[-0.10962 , -0.182842, -0.376016, ..., -0.338578, -0.192524,
            -0.111345],
           [ 0.030942, -0.091037, -0.088928, ..., -0.190589, -0.206673,
             0.157685],
           [ 0.396702,  0.495481,  0.090774, ..., -0.13526 , -0.03119 ,
             0.149953],
           ...,
           [ 1.440614,  1.509055,  1.267049, ...,  1.697352,  1.804161,
             1.70886 ],
           [ 1.552706,  1.677142,  1.545659, ...,  1.526752,  1.445978,
             1.443655],
           [ 1.140082,  1.025372,  1.10964 , ...,  1.511037,  1.678758,
             1.58007 ]]),
     'meff': (2.9043835152525843, 0.02214716920258792),
     'sigma_sqr': 0.38722475634554737}
#### jmfHG7qv49uQWSX6ObmAZztj5vDlzTqFJY_GZg==.subsim:
    {'acceptance': 0.37961088,
     'measures': array([[-0.22464 , -0.697833, -0.643082, ..., -0.809505, -0.540468,
            -0.406914],
           [-0.193615, -0.060498,  0.035809, ..., -0.56093 , -0.66168 ,
            -0.245584],
           [-0.219566, -0.210865, -0.389684, ..., -0.050059,  0.007061,
             0.000986],
           ...,
           [ 0.695766,  0.770338,  0.706962, ...,  0.906742,  0.932494,
             0.71368 ],
           [ 0.22843 ,  0.464933,  0.52069 , ...,  0.112195,  0.001082,
            -0.027001],
           [ 0.660103,  0.063466, -0.152869, ...,  0.702389,  0.708249,
             0.543501]])}
#### QBIscaV5XuE93mbzqbxKmgCE_3HfisIB1fM56A==.subsim:
    {'acceptance': 0.22607491200000002,
     'measures': array([[-0.755311, -0.62972 , -0.499213, ..., -0.392453, -0.588779,
            -0.721076],
           [-0.644524, -0.673183, -0.737285, ..., -0.262721, -0.64284 ,
            -0.659176],
           [-0.541543, -0.435367, -0.325128, ..., -0.557921, -0.514325,
            -0.632871],
           ...,
           [ 1.376341,  1.523743,  1.577699, ...,  1.420305,  1.476988,
             1.464748],
           [ 1.337028,  1.393131,  1.178146, ...,  1.578612,  1.726826,
             1.591828],
           [ 1.434126,  1.560615,  1.756875, ...,  1.727411,  1.403706,
             1.39079 ]])}
#### G9xHH7Jk58cGd76L-ui9vL97EhSre5LV_aFdlA==.subsim:
    {'acceptance': 0.16885128500000002,
     'measures': array([[-0.210238, -0.283243, -0.25624 , ..., -0.148162, -0.285616,
            -0.235033],
           [-0.139099, -0.091949, -0.084361, ..., -0.356674, -0.406162,
            -0.253201],
           [-0.455325, -0.364296, -0.308663, ..., -0.295702, -0.443004,
            -0.435855],
           ...,
           [-0.395344, -0.271697, -0.308543, ..., -0.625205, -0.463694,
            -0.495197],
           [-0.383165, -0.447898, -0.464382, ..., -0.358886, -0.44877 ,
            -0.371487],
           [-0.812819, -0.908607, -1.036451, ..., -0.772069, -0.802664,
            -0.759237]])}
#### JAqGd_j1r2Nf-ugJoLw12p3RJ7QHdqaI6ORtIA==.subsim:
    {'acceptance': 0.32945846,
     'measures': array([[ 0.090919,  0.321668,  0.09826 , ..., -0.381814, -0.231346,
            -0.0957  ],
           [-0.196929, -0.091427, -0.382794, ..., -0.756333, -0.457807,
            -0.316602],
           [-0.375814, -0.454446, -0.428639, ..., -0.361368, -0.126081,
            -0.383958],
           ...,
           [ 1.160218,  1.114014,  0.935226, ...,  0.798505,  0.96661 ,
             1.031381],
           [ 1.965926,  2.135991,  1.932709, ...,  1.953653,  1.891413,
             1.996472],
           [ 1.899972,  1.800239,  1.610603, ...,  1.583547,  1.218532,
             1.389031]])}
#### .simulate_hash:
    '2dc79322d52e2c7af2ceef6759d52ef71a0e6f73c0bb86a45f4e3bad'
#### Fa9fHIKdJk6FjrJqI2GSTz233FiQKFd3tg1OBw==.subsim:
    {'DeltaE': (0.15764244285436302, 0.0007974070457110911),
     'acceptance': 0.30625596199999994,
     'correlations': array([0.60071474, 0.58578795, 0.57203964, 0.56504221, 0.55408394,
           0.54010407, 0.5302681 , 0.52308737, 0.51646493, 0.50751205,
           0.503055  , 0.49714951, 0.48696186, 0.47438779, 0.46694094,
           0.46151185, 0.45216062, 0.44310865, 0.43302657, 0.42835341,
           0.41371959, 0.40779059, 0.39663412, 0.39088102, 0.3856749 ,
           0.3788404 , 0.37398344, 0.36497581, 0.35336237, 0.34801073,
           0.34162208, 0.34065463, 0.33669403, 0.33382846, 0.32871377,
           0.32589862, 0.32118312, 0.32070724, 0.31281816, 0.31021536,
           0.30385917, 0.30223974, 0.29898319, 0.29091428, 0.28345277,
           0.27831087, 0.27742708, 0.27506709, 0.264175  , 0.26040591,
           0.25809612, 0.25100025, 0.24498996, 0.24577871, 0.24425961,
           0.23802337, 0.2422125 , 0.23749927, 0.23210112, 0.22782789,
           0.22340831, 0.21765947, 0.2149755 , 0.20816254, 0.20456262,
           0.20473764, 0.20070954, 0.20138963, 0.19697052, 0.18882897,
           0.18494526, 0.18125191, 0.18053017, 0.17929839, 0.17563606,
           0.17472338, 0.16897693, 0.1692088 , 0.16561003, 0.16333678,
           0.16349702, 0.16441536, 0.16295084, 0.15746672, 0.15062039,
           0.15131184, 0.15051472, 0.14852593, 0.14443776, 0.14097254,
           0.13892651, 0.13970932, 0.13972528, 0.14058214, 0.13833544,
           0.13389878, 0.13047798, 0.12878747, 0.12273539, 0.11406058,
           0.11638703, 0.11555642, 0.11450755, 0.11205996, 0.1121503 ,
           0.10894163, 0.1061012 , 0.10355977, 0.10210779, 0.10105834,
           0.09858581, 0.09671073, 0.09834171, 0.09808065, 0.0974399 ,
           0.10020618, 0.09873186, 0.09900798, 0.09368745, 0.09293534,
           0.09036637, 0.08935417, 0.08747766, 0.08259802, 0.07703103,
           0.0788313 , 0.07772952, 0.0762304 , 0.07495123, 0.07208529,
           0.07374953, 0.0698711 , 0.0680915 , 0.06471829, 0.06181423,
           0.06037834, 0.05639249, 0.05378718, 0.05245528, 0.05103654,
           0.05627796, 0.05674967, 0.05611702, 0.05295583, 0.05273139,
           0.0512201 , 0.05205293, 0.05501785, 0.05315552, 0.05248062]),
     'measures': array([[-0.360295, -0.432963, -0.39683 , ..., -0.275548, -0.092414,
            -0.232928],
           [-0.325859, -0.130265, -0.010981, ..., -0.674412, -0.643679,
            -0.333631],
           [-0.008121, -0.09609 , -0.113492, ..., -0.362569,  0.046266,
             0.102685],
           ...,
           [-1.08659 , -1.166063, -1.339485, ..., -0.974205, -1.148733,
            -1.053675],
           [-1.08755 , -1.052821, -1.013273, ..., -0.90638 , -1.118669,
            -1.16215 ],
           [-1.055197, -0.841132, -0.793953, ..., -1.170545, -1.082864,
            -1.033689]]),
     'meff': (1.6095841212707185, 0.00814180303049296),
     'sigma_sqr': 0.3518112103347627}
#### 8hlZSnCtW3gt6X6uW1F2e4BvH5rdtpChmDu2cQ==.subsim:
    {'acceptance': 0.172262635,
     'measures': array([[-0.403868, -0.426326, -0.333636, ..., -0.470341, -0.363657,
            -0.264803],
           [-0.217125, -0.258114, -0.065683, ..., -0.301437, -0.285094,
            -0.214199],
           [-0.435325, -0.366715, -0.384767, ..., -0.535903, -0.484789,
            -0.460713],
           ...,
           [-0.077558, -0.010295, -0.083659, ..., -0.046456, -0.055121,
            -0.013225],
           [ 0.032363,  0.099297,  0.137912, ..., -0.161016, -0.18368 ,
            -0.116138],
           [-0.268694, -0.364798, -0.377905, ..., -0.230085, -0.296045,
            -0.215703]])}
#### GOa_MVHH-kIrY4n2SfdS-OOYn_x9HtKTMSAwgw==.subsim:
    {'acceptance': 0.51928142,
     'correlations': array([ 2.20494986e+00,  2.14512372e+00,  2.09209183e+00,  2.05976742e+00,
            2.02613630e+00,  1.98495644e+00,  1.93372254e+00,  1.88102741e+00,
            1.82985538e+00,  1.80322598e+00,  1.77448215e+00,  1.75885948e+00,
            1.68654257e+00,  1.63939094e+00,  1.61199277e+00,  1.55684855e+00,
            1.53216672e+00,  1.50520235e+00,  1.47666081e+00,  1.44629595e+00,
            1.41308987e+00,  1.37662461e+00,  1.32738797e+00,  1.27506768e+00,
            1.25180297e+00,  1.21602635e+00,  1.20184591e+00,  1.16108307e+00,
            1.12408118e+00,  1.07257413e+00,  1.06166286e+00,  1.02939199e+00,
            1.01637981e+00,  9.88389017e-01,  9.72940023e-01,  9.33199039e-01,
            9.19883445e-01,  8.76475410e-01,  8.67534234e-01,  8.55081507e-01,
            8.27622477e-01,  7.97166612e-01,  7.67664069e-01,  7.78107166e-01,
            7.51703635e-01,  7.43481820e-01,  7.61916019e-01,  7.57608397e-01,
            7.34335896e-01,  7.30189838e-01,  7.08442099e-01,  6.77119872e-01,
            6.76282956e-01,  6.55369490e-01,  6.34758271e-01,  6.18815063e-01,
            6.03616858e-01,  6.07463655e-01,  6.19614707e-01,  6.00055954e-01,
            5.92927141e-01,  5.87523514e-01,  5.88276258e-01,  5.93861352e-01,
            5.80300119e-01,  5.94083509e-01,  5.66257459e-01,  5.48962355e-01,
            5.25695418e-01,  5.17949074e-01,  5.31028875e-01,  5.28417151e-01,
            4.99575358e-01,  5.02628342e-01,  5.05527855e-01,  4.86976183e-01,
            4.76506149e-01,  4.55823476e-01,  4.23342857e-01,  3.92414998e-01,
            3.84206507e-01,  3.60318477e-01,  3.73659951e-01,  3.76547656e-01,
            3.68419133e-01,  3.30383401e-01,  3.21527020e-01,  3.06800780e-01,
            2.87964214e-01,  2.72542583e-01,  2.55569728e-01,  2.54095505e-01,
            2.62976806e-01,  2.39661155e-01,  2.27786497e-01,  2.04527374e-01,
            1.77034963e-01,  1.66113592e-01,  1.53573020e-01,  1.38933148e-01,
            1.16677425e-01,  1.17935103e-01,  1.29270168e-01,  1.57713112e-01,
            1.47120786e-01,  1.38270297e-01,  1.35714660e-01,  1.34575954e-01,
            1.18140961e-01,  1.30828222e-01,  1.27362478e-01,  1.29406092e-01,
            1.50250548e-01,  1.43913252e-01,  1.32841132e-01,  1.09820478e-01,
            1.10192237e-01,  9.70914246e-02,  1.16283858e-01,  9.25720074e-02,
            9.48504237e-02,  6.93550596e-02,  7.08177989e-02,  6.35519355e-02,
            7.86047758e-02,  7.72357023e-02,  1.07833759e-01,  1.04689017e-01,
            9.45588285e-02,  9.97005710e-02,  8.53672326e-02,  8.79495300e-02,
            6.45343571e-02,  5.25830426e-02,  5.33431445e-02,  6.94169890e-02,
            6.21886886e-02,  4.84490358e-02,  5.34595448e-02,  6.04898231e-02,
            3.68047581e-02,  3.42546075e-02,  3.44703823e-02,  1.45177016e-02,
            1.45329444e-03, -1.35725306e-03, -1.93492388e-02, -8.77760706e-03,
           -2.81659944e-02, -4.18081894e-02, -2.14624820e-02, -5.25836872e-02,
           -5.91918916e-02, -4.90145168e-02, -4.43044211e-02, -4.96010316e-02,
           -3.70039154e-02, -1.58299308e-02,  6.38715970e-03,  2.17181909e-02,
            1.04622253e-02,  3.09191499e-02,  4.09442433e-02,  3.27873916e-02,
            4.60428871e-02,  3.88421800e-02,  3.61808958e-02,  4.66597897e-02,
            3.21980042e-02,  2.80483173e-02,  2.15347452e-02,  4.23204416e-02,
            3.06527293e-02,  1.80576939e-02,  2.99186068e-02,  6.19359363e-02,
            4.71724096e-02,  3.92619821e-02,  3.33193028e-02,  3.87682566e-02,
            3.15943199e-02,  3.75057847e-02,  4.37435815e-02,  5.19030602e-02,
            4.90663080e-02,  6.96574800e-02,  8.35632182e-02,  7.64150691e-02,
            6.96013495e-02,  6.40007777e-02,  6.04546927e-02,  4.75094477e-02,
            5.50266672e-02,  4.49772774e-02,  4.17804612e-02,  6.16507877e-02,
            5.99893112e-02,  5.87820491e-02,  6.80190166e-02,  7.85440983e-02,
            5.69565195e-02,  5.32746210e-02,  2.52828094e-02,  1.49771865e-02,
            3.51176729e-02,  3.32722133e-02,  2.71237930e-02,  1.11453190e-02,
           -1.53183596e-02, -2.33064364e-02, -1.77870787e-02, -2.50329376e-02,
           -1.58510216e-02, -1.74998926e-02, -2.25910759e-02, -2.53903336e-02,
           -4.54687300e-03,  5.50885232e-03, -5.48736290e-03, -2.35334666e-02,
           -4.44540321e-02, -4.67190338e-02, -4.35927409e-02, -6.56669289e-02,
           -7.66958289e-02, -1.19618179e-01, -1.41565197e-01, -1.67332852e-01,
           -1.60848615e-01, -1.64176988e-01, -1.72205032e-01, -1.66195568e-01,
           -1.79006328e-01, -1.85360942e-01, -1.62995458e-01, -1.53436164e-01,
           -1.59131402e-01, -1.47445215e-01, -1.58945884e-01, -1.36292994e-01,
           -1.39553363e-01, -1.37223961e-01, -1.33773889e-01, -1.26518439e-01,
           -9.52902050e-02, -9.39444035e-02, -1.01613413e-01, -9.89067829e-02,
           -9.16540440e-02, -8.20635445e-02, -7.30184292e-02, -5.00696171e-02,
           -5.78135306e-02, -6.39623573e-02, -5.59893498e-02, -4.39353939e-02,
           -5.91079847e-02, -6.16781778e-02, -3.51510258e-02, -4.16615648e-02,
           -4.37671960e-02, -5.01627870e-02, -5.91793075e-02, -6.53598901e-02,
           -5.30204922e-02, -4.87279203e-02, -5.56953551e-02, -6.46217260e-02,
           -7.50154817e-02, -5.99611159e-02, -7.18445720e-02, -8.17523889e-02,
           -9.02589202e-02, -7.58816952e-02, -7.56992524e-02, -7.95277308e-02,
           -6.23374939e-02, -4.75329491e-02, -6.40796352e-02, -4.16044583e-02,
           -3.25257704e-02, -1.98063399e-02, -2.14492833e-04,  1.49805539e-02,
            2.04611282e-02,  1.47958845e-02,  1.82602448e-02,  3.91987118e-02,
            2.47700350e-02,  2.66942838e-02,  1.51316330e-02,  3.45919756e-02,
            6.12934154e-02,  3.30170280e-02,  4.93161788e-02,  2.87295235e-02,
            3.21346773e-02,  2.66811292e-02,  9.81320793e-03,  3.54440187e-03,
           -2.66184832e-03, -2.32748185e-02,  7.89166617e-05,  1.91458050e-02,
            9.77133561e-03,  1.60478633e-02,  3.06260225e-02,  1.12196036e-02,
            2.21877421e-02,  1.58222898e-02,  2.04388429e-02,  9.57051755e-03,
            5.64342239e-03,  1.03669779e-03,  2.32247474e-03, -6.35262208e-03,
           -3.09926121e-03,  9.79091331e-03,  1.21229362e-02, -1.66273624e-03,
           -7.19384618e-03, -1.49568323e-02,  6.08949296e-03,  2.11574742e-03,
           -1.69155109e-02, -2.14133363e-02, -2.94673307e-02, -2.66848947e-02,
           -1.31076270e-02, -9.03383848e-03, -1.39321368e-03,  1.88700575e-02,
           -1.32312306e-03,  9.03151412e-03,  2.86320468e-02,  4.61066630e-02,
            3.45822587e-02,  3.40546277e-02,  1.20798081e-02, -2.49721709e-03,
           -4.07065499e-03, -3.08432676e-03, -1.07008513e-02, -2.55642685e-03,
            1.40809901e-02,  3.14366786e-03, -6.15683442e-03, -1.29093087e-02,
           -1.52192925e-02, -1.61781059e-02, -5.03131396e-02, -3.86689391e-02,
           -4.80331722e-02, -2.69220376e-02, -3.46504699e-02, -5.43309925e-02,
           -5.04548840e-02, -5.62890827e-02, -9.40382805e-02, -1.00522788e-01,
           -9.02694015e-02, -8.94306687e-02, -1.01326584e-01, -1.01893719e-01,
           -1.01190198e-01, -8.38589918e-02, -8.94771759e-02, -8.19896176e-02,
           -5.66758614e-02, -5.67790996e-02, -4.06675060e-02, -4.65048917e-02,
           -5.63137587e-02, -4.94494841e-02, -3.70707039e-02, -3.21414706e-02,
           -4.31899572e-02, -6.36875278e-02, -8.12007094e-02, -7.56104623e-02,
           -4.24748678e-02, -5.30824400e-02, -4.73785831e-02, -3.74860791e-02,
           -4.36235493e-02, -4.64436751e-02, -3.26792868e-02, -3.40170907e-02,
           -2.85028896e-02, -4.09144664e-02, -4.49026749e-02, -4.98205327e-02,
           -2.96801701e-02, -4.67785614e-02, -5.69438578e-02, -6.16353483e-02,
           -4.62960512e-02, -5.49347133e-02, -3.88038305e-02, -4.22882028e-02,
           -5.11834621e-03, -1.82204970e-02, -3.83268628e-02, -3.53842617e-02,
           -3.58206142e-02, -3.65767896e-02, -2.01301348e-02, -1.31787966e-02,
            3.58690377e-03, -2.74202176e-02, -4.04062629e-02, -3.50650623e-02,
           -1.67590037e-02, -4.97524250e-03, -7.02508224e-03, -2.57730641e-02,
           -2.16396169e-02, -1.51434284e-02, -7.17952755e-03, -7.29133711e-03,
           -2.00017842e-02, -4.80763734e-02, -4.12132142e-02, -7.53892194e-02,
           -7.64324813e-02, -6.78002966e-02, -7.34654679e-02, -6.91286288e-02,
           -5.40458151e-02, -6.99480001e-02, -6.46183102e-02, -6.85475800e-02,
           -5.26394254e-02, -2.05112062e-02, -1.16946208e-02, -6.66178297e-03,
            7.95549148e-03,  9.93946088e-03,  8.54176327e-04, -6.98389955e-03,
           -6.11520199e-03,  1.54189777e-02,  1.57495728e-02,  3.04987394e-03,
            5.79763929e-03, -2.10734888e-02, -2.47888270e-02, -2.00091551e-02,
           -1.44372217e-02, -2.37049732e-02, -3.10156963e-02, -3.04524082e-02,
           -5.02669087e-02, -7.38929461e-02, -7.88860828e-02, -8.76526837e-02,
           -1.05368774e-01, -1.05887330e-01, -1.10089051e-01, -1.29999661e-01,
           -1.16966736e-01, -1.11504548e-01, -1.28550847e-01, -1.42058669e-01,
           -1.48791051e-01, -1.46397587e-01, -1.44215121e-01, -1.36177622e-01,
           -1.32908699e-01, -1.49882473e-01, -1.38803940e-01, -1.32370026e-01,
           -1.25153287e-01, -1.30842047e-01, -1.17521334e-01, -1.13175298e-01,
           -1.47064597e-01, -1.70203890e-01, -1.73488502e-01, -1.47494791e-01,
           -1.72937549e-01, -1.74617505e-01, -1.60035268e-01, -1.69779646e-01,
           -1.74845956e-01, -1.80385960e-01, -1.50708485e-01, -1.43467320e-01,
           -1.55622833e-01, -1.85519256e-01, -1.56763869e-01, -1.37762899e-01,
           -1.57964416e-01, -1.56452411e-01, -1.27566965e-01, -1.15691455e-01,
           -1.01729419e-01, -7.64300058e-02, -5.98605633e-02, -6.61522078e-02]),
     'measures': array([[-0.524058, -0.237223, -0.347507, ..., -1.390519, -0.928979,
            -0.726051],
           [-1.476175, -1.458456, -1.760469, ..., -0.876013, -1.289078,
            -1.081598],
           [-1.024416, -0.966707, -1.045026, ..., -0.944412, -1.154032,
            -1.576771],
           ...,
           [ 0.69056 ,  0.475711,  0.164924, ...,  0.522445,  0.462672,
             0.532443],
           [ 1.615655,  1.881237,  2.445143, ...,  1.23183 ,  1.507852,
             1.362721],
           [ 0.662983,  0.425615,  0.567927, ...,  0.233655,  0.584652,
             0.652548]]),
     'sigma_sqr': 2.089786982539251}
#### jDYQUct7_lbmHxrrJrRgRIx639DBSWC0tHjZwA==.subsim:
    {'acceptance': 0.165727756,
     'measures': array([[-0.272272, -0.345211, -0.324991, ..., -0.361324, -0.319065,
            -0.192975],
           [-0.54993 , -0.623713, -0.598655, ..., -0.258297, -0.31149 ,
            -0.397693],
           [-0.33629 , -0.311514, -0.401559, ..., -0.273586, -0.448745,
            -0.419189],
           ...,
           [-0.001666, -0.031138,  0.084773, ..., -0.155208, -0.04589 ,
             0.048416],
           [-0.172929, -0.20663 , -0.225281, ..., -0.050581, -0.052479,
            -0.1281  ],
           [-0.534067, -0.45561 , -0.548958, ..., -0.501885, -0.385664,
            -0.495255]])}
#### KZIKr-eRoAxMYI9vVmXrcGR3DVs2q465c0Zs7g==.subsim:
    {'acceptance': 0.31323011500000003,
     'measures': array([[ 0.102917,  0.049068,  0.149333, ...,  0.371914,  0.186179,
             0.110041],
           [ 0.862427,  0.916372,  0.641906, ...,  1.210764,  0.914935,
             0.81779 ],
           [ 1.108966,  1.335183,  1.260027, ...,  0.938158,  0.774941,
             0.950082],
           ...,
           [-2.590047, -2.580694, -2.688287, ..., -2.920602, -2.828965,
            -2.562854],
           [-2.976622, -3.081599, -3.082208, ..., -2.975771, -2.823418,
            -2.969988],
           [-2.877754, -2.992792, -3.021446, ..., -2.875212, -2.629558,
            -2.574788]])}
#### xWouCAv2nlekpemieq6nv4DQp5w2efiN4zh4Gg==.subsim:
    {'acceptance': 0.329470014,
     'correlations': array([ 6.40079974e-01,  6.29338970e-01,  6.17924521e-01,  6.07664235e-01,
            6.00062526e-01,  5.89619458e-01,  5.85840108e-01,  5.72683993e-01,
            5.69039201e-01,  5.56057527e-01,  5.40650046e-01,  5.31674712e-01,
            5.18869266e-01,  5.10432393e-01,  4.98324666e-01,  4.88086413e-01,
            4.82357181e-01,  4.71790779e-01,  4.57140721e-01,  4.48321050e-01,
            4.41660639e-01,  4.31532443e-01,  4.23735674e-01,  4.12257832e-01,
            4.03105166e-01,  3.97446288e-01,  3.94707045e-01,  3.87082639e-01,
            3.85535048e-01,  3.76746770e-01,  3.63396662e-01,  3.52248771e-01,
            3.42467020e-01,  3.39186730e-01,  3.28745432e-01,  3.24751408e-01,
            3.17937180e-01,  3.07159944e-01,  3.00424900e-01,  2.95394448e-01,
            2.88187948e-01,  2.80374179e-01,  2.79137448e-01,  2.77319460e-01,
            2.67596307e-01,  2.61849114e-01,  2.58749777e-01,  2.58384289e-01,
            2.57618290e-01,  2.55762980e-01,  2.42562397e-01,  2.40069042e-01,
            2.34343702e-01,  2.34861813e-01,  2.33980314e-01,  2.30640996e-01,
            2.28295991e-01,  2.29182117e-01,  2.20221514e-01,  2.15814416e-01,
            2.13432721e-01,  2.16799034e-01,  2.06962624e-01,  2.04071626e-01,
            2.03450703e-01,  2.00637158e-01,  1.86526204e-01,  1.81300292e-01,
            1.78553318e-01,  1.74716836e-01,  1.71934904e-01,  1.77676025e-01,
            1.78363139e-01,  1.74613848e-01,  1.72742119e-01,  1.66313495e-01,
            1.65776129e-01,  1.56692174e-01,  1.59211578e-01,  1.57791161e-01,
            1.51553958e-01,  1.51557731e-01,  1.42253476e-01,  1.42150267e-01,
            1.35912149e-01,  1.28265817e-01,  1.27551730e-01,  1.30202519e-01,
            1.24528722e-01,  1.21529989e-01,  1.19919613e-01,  1.17197853e-01,
            1.18271928e-01,  1.12774268e-01,  1.08284650e-01,  1.06699196e-01,
            1.05890103e-01,  1.07360776e-01,  1.07950996e-01,  1.05789308e-01,
            9.94303087e-02,  9.90425260e-02,  9.37019862e-02,  8.78108369e-02,
            8.86395544e-02,  9.43171084e-02,  9.29005386e-02,  9.39852874e-02,
            9.40350864e-02,  9.10260783e-02,  7.67941201e-02,  7.60123554e-02,
            7.28504901e-02,  6.97901354e-02,  6.29746119e-02,  6.18668448e-02,
            5.89420247e-02,  4.68459901e-02,  4.24680216e-02,  3.85540481e-02,
            3.07329176e-02,  3.41426908e-02,  3.62053122e-02,  3.39296252e-02,
            3.86483036e-02,  2.92763046e-02,  2.85913108e-02,  2.53440843e-02,
            2.93984435e-02,  2.85713883e-02,  2.71259055e-02,  2.54584486e-02,
            2.67079987e-02,  2.76232061e-02,  2.62518877e-02,  2.40627432e-02,
            3.14704681e-02,  3.01997074e-02,  2.53538049e-02,  2.73501002e-02,
            3.08132624e-02,  2.62748876e-02,  2.53471458e-02,  2.10997211e-02,
            2.25572784e-02,  2.33876870e-02,  2.20798336e-02,  2.26136293e-02,
            2.24068386e-02,  2.52390290e-02,  2.82403478e-02,  2.30431100e-02,
            2.51832408e-02,  2.76956994e-02,  2.17005779e-02,  2.76735708e-02,
            2.22233512e-02,  2.71766669e-02,  1.88586695e-02,  1.56601746e-02,
            8.48639725e-03,  6.17519995e-03,  1.46426602e-02,  1.33447770e-02,
            1.92428501e-02,  1.48335912e-02,  1.04254435e-02,  6.64891049e-03,
            9.86696720e-03,  8.26435872e-03,  2.66831302e-03, -1.03514354e-03,
           -3.37726565e-03, -4.46063301e-04, -6.24103218e-03, -8.32278445e-04,
            8.22753290e-04,  5.67564737e-03,  1.07203124e-02,  1.53196320e-02,
            1.31447208e-02,  1.53934897e-02,  1.62002957e-02,  7.91475669e-03,
           -4.81121131e-03, -2.18812373e-03, -1.74294133e-03,  2.20381648e-03,
            4.02570802e-03, -4.20431791e-03, -5.25303755e-05, -1.39748135e-04,
           -1.11053221e-03,  6.16573758e-04,  3.49153803e-03,  4.56123190e-03,
            3.21238621e-03,  6.57012243e-03,  4.41013312e-03,  7.64346205e-04,
            7.21648472e-05,  4.14158853e-03,  8.50014431e-03,  1.25254100e-02,
            5.24124196e-03,  6.05769334e-03,  6.68874503e-03,  1.08510847e-02,
            1.53152475e-02,  1.76281202e-02,  1.07324708e-02,  1.41786329e-02,
            5.53809271e-03,  4.69493770e-03,  4.61661258e-03, -3.89280184e-04,
            2.82481630e-03, -4.62129434e-04,  1.95741421e-03,  5.65427618e-03,
            5.44431124e-03,  3.79881279e-03, -2.77227771e-05, -3.40976827e-03,
           -4.60221347e-04, -7.88138472e-03,  2.16777071e-03, -5.25615077e-04,
           -2.18368981e-03, -1.12071884e-02, -6.67598505e-03, -7.69173747e-03,
           -6.42217280e-03, -8.56902731e-03, -6.46257929e-03, -7.03287838e-03,
           -1.53449585e-02, -1.61075439e-02, -1.59223421e-02, -1.79460094e-02,
           -1.25554498e-02, -4.87831026e-03,  8.16790929e-03,  5.23805080e-03,
            9.73389515e-03,  3.28163594e-04,  3.24763356e-04, -1.93111588e-03,
            2.03965446e-03,  3.61597092e-04, -7.05051950e-03, -3.89391095e-03,
           -6.58884724e-03, -1.12212497e-02, -8.06760109e-03, -1.29264997e-02,
           -1.36461430e-02, -1.18770884e-02, -1.10383469e-02, -1.38758500e-02,
           -1.97595437e-02, -2.45660190e-02, -1.90805587e-02, -2.16471869e-02,
           -1.77266184e-02, -2.25967927e-02, -2.41749486e-02, -2.11237570e-02,
           -1.70943001e-02, -8.87909675e-03, -1.10977093e-02, -8.56541307e-03,
           -7.61273795e-03, -1.55263065e-02, -1.84270272e-02, -1.69711320e-02,
           -2.32796476e-02, -2.39991879e-02, -1.95452736e-02, -1.64728347e-02,
           -1.64711589e-02, -1.53951902e-02, -1.52705098e-02, -2.32335115e-03,
           -7.39567759e-03, -8.63603284e-03, -1.41155869e-02, -1.19493391e-02,
           -1.78940755e-02, -2.12182445e-02, -2.20390711e-02, -2.21339714e-02,
           -1.73577438e-02, -1.73578599e-02, -1.30755656e-02, -1.29324166e-02,
           -9.58315712e-03, -1.24651429e-02, -1.54909556e-02, -2.07445889e-02,
           -2.11850093e-02, -1.65292166e-02, -2.46492490e-02, -2.80236607e-02,
           -2.93242563e-02, -2.59173634e-02, -2.04239231e-02, -2.16079572e-02,
           -2.10888386e-02, -2.15497537e-02, -2.28628021e-02, -2.73230669e-02,
           -2.01815417e-02, -2.09783619e-02, -1.72993399e-02, -2.46616314e-02,
           -2.91598756e-02, -2.75010562e-02, -1.32169537e-02, -1.39356845e-02,
           -1.05823468e-02, -5.56336074e-03,  4.63358488e-04, -5.02225370e-03,
           -1.20028026e-02, -1.48650230e-02, -1.56795767e-02, -1.32541763e-02,
           -2.11989054e-02, -2.45761473e-02, -2.46557948e-02, -2.64865446e-02,
           -3.54000740e-02, -2.90433578e-02, -2.20971711e-02, -2.37269377e-02,
           -1.60504549e-02, -1.73029602e-02, -1.24657624e-02, -8.83579082e-03,
           -9.81328340e-03, -9.26031953e-03, -8.34852853e-03, -1.18280050e-02,
           -7.57268184e-03, -9.13048326e-03, -1.41627860e-02, -7.75395967e-03,
           -2.26139845e-03, -1.17783519e-03,  3.21788944e-03,  9.52368935e-03,
            6.31781165e-03,  6.04288649e-03,  6.69263433e-03, -3.56113638e-03,
            1.52630873e-03, -3.78797892e-03,  8.39561732e-03,  5.06712462e-03,
            8.73352220e-03,  4.98461500e-03,  4.42485352e-03, -1.65886134e-03,
           -2.34493567e-03,  5.63045742e-03, -8.63919798e-04,  5.66196289e-03,
            4.26993415e-03,  1.48508695e-02,  1.56899084e-02,  1.92805901e-02,
            2.35057797e-02,  2.47279593e-02,  1.71393141e-02,  1.88799267e-02,
            1.30380654e-02,  1.23430738e-02,  2.34781768e-02,  2.44702740e-02,
            2.40068631e-02,  2.17485123e-02,  2.53611505e-02,  2.68525576e-02,
            3.61511395e-02,  3.81127823e-02,  4.42592332e-02,  3.65001672e-02,
            3.62543610e-02,  3.08408035e-02,  3.39979046e-02,  3.79365504e-02,
            3.70967570e-02,  2.85422321e-02,  3.94466702e-02,  3.66346974e-02,
            3.72729467e-02,  3.79359452e-02,  4.16987276e-02,  3.74939561e-02,
            4.38523311e-02,  4.57469852e-02,  4.85276061e-02,  4.51914844e-02,
            4.41166273e-02,  3.81976557e-02,  4.97943539e-02,  5.08550484e-02,
            4.91112663e-02,  5.34503180e-02,  5.20051265e-02,  5.69298961e-02,
            5.73207317e-02,  4.97202883e-02,  5.03108041e-02,  5.40057557e-02,
            4.94482801e-02,  4.33109528e-02,  3.56727215e-02,  3.23692534e-02,
            3.36188584e-02,  3.26150340e-02,  2.50578013e-02,  2.06233811e-02,
            1.76119387e-02,  2.16372381e-02,  2.20554791e-02,  2.50129427e-02,
            2.28318028e-02,  1.39323243e-02,  1.55072968e-02,  1.31176164e-02,
            7.04830960e-03,  1.44981515e-02,  1.45521512e-02,  1.36612223e-02,
            1.62451465e-02,  1.97175045e-02,  1.59371701e-02,  1.34794181e-02,
            1.06021035e-02,  1.18251250e-02,  5.26990433e-03, -5.86248967e-03,
           -7.57270867e-03, -6.74619352e-03, -5.28090834e-03, -1.29145070e-02,
           -1.15470913e-02, -8.11014967e-03, -7.46975791e-03, -1.43263131e-02,
           -1.53730793e-02, -1.98120489e-02, -1.70309507e-02, -1.35836062e-02,
           -1.41064094e-02, -6.61824805e-05, -4.31066720e-03, -1.24693157e-02,
           -1.42794591e-02, -1.99006705e-02, -1.08195045e-02, -1.71344317e-02,
           -2.01796901e-02, -1.96664683e-02, -2.79322920e-02, -2.80969911e-02,
           -2.36549555e-02, -2.55321881e-02, -2.86412532e-02, -2.46248519e-02,
           -2.48073927e-02, -2.38031448e-02, -3.09451313e-02, -3.13112915e-02,
           -3.12625641e-02, -3.39901084e-02, -3.26630794e-02, -3.17685912e-02,
           -2.54582058e-02, -3.27622548e-02, -3.36476998e-02, -3.58457642e-02,
           -3.24797345e-02, -3.06658412e-02, -3.65291415e-02, -3.87678840e-02,
           -4.11710674e-02, -3.60893258e-02, -2.68419786e-02, -2.27506782e-02,
           -1.98682770e-02, -1.25551059e-02, -1.18919167e-02, -9.83782638e-03,
           -1.49822905e-02, -1.11766027e-02, -1.36479851e-02, -1.83897129e-02]),
     'measures': array([[-1.994953, -2.081618, -2.1844  , ..., -1.628201, -1.860676,
            -1.79104 ],
           [-0.4179  , -0.369688, -0.289186, ..., -0.41898 , -0.743179,
            -0.314773],
           [-0.421037, -0.323131, -0.360733, ..., -0.246301, -0.351586,
            -0.244228],
           ...,
           [ 0.66565 ,  0.565105,  0.485788, ...,  0.933244,  0.65988 ,
             0.555626],
           [ 0.26129 ,  0.608817,  0.848425, ...,  0.390629,  0.531084,
             0.249713],
           [ 0.257729,  0.163437,  0.058017, ..., -0.049312, -0.051645,
             0.123572]]),
     'sigma_sqr': 0.6494756144392339}
#### uPotQctLgmPJKIbfwdnIbisBHUsIUu0scC5Urg==.subsim:
    {'acceptance': 0.39907391200000003,
     'correlations': array([ 1.12379865e+00,  1.11370197e+00,  1.08928130e+00,  1.06232413e+00,
            1.03281365e+00,  1.01663801e+00,  9.91934966e-01,  9.71555746e-01,
            9.51704522e-01,  9.36061768e-01,  9.08550503e-01,  8.88577978e-01,
            8.75580895e-01,  8.43896371e-01,  8.28741084e-01,  8.21079218e-01,
            8.12931361e-01,  7.94794570e-01,  7.70487254e-01,  7.48878312e-01,
            7.38718389e-01,  7.26925557e-01,  7.11952830e-01,  6.92894690e-01,
            6.84482024e-01,  6.70475872e-01,  6.64037714e-01,  6.43734470e-01,
            6.44103542e-01,  6.36810423e-01,  6.29611120e-01,  6.01526124e-01,
            5.92296650e-01,  5.83300946e-01,  5.86479766e-01,  5.94211940e-01,
            5.79037965e-01,  5.74294850e-01,  5.52172637e-01,  5.41013178e-01,
            5.21209970e-01,  5.06440308e-01,  5.01282389e-01,  4.91165804e-01,
            4.80982918e-01,  4.69420448e-01,  4.68086208e-01,  4.58528805e-01,
            4.49416456e-01,  4.43144775e-01,  4.29134699e-01,  4.18007394e-01,
            4.06844066e-01,  3.90903217e-01,  3.77807350e-01,  3.72800095e-01,
            3.58465635e-01,  3.57292376e-01,  3.52491303e-01,  3.47345184e-01,
            3.34948495e-01,  3.36176999e-01,  3.36125008e-01,  3.35007252e-01,
            3.21973041e-01,  3.03945361e-01,  2.95470586e-01,  2.92766592e-01,
            2.95339881e-01,  2.87328682e-01,  2.92059807e-01,  2.84107129e-01,
            2.78300222e-01,  2.75640603e-01,  2.68438546e-01,  2.50347931e-01,
            2.43997842e-01,  2.44593189e-01,  2.44678923e-01,  2.35102711e-01,
            2.27379789e-01,  2.30342989e-01,  2.21556382e-01,  2.25951274e-01,
            2.18111489e-01,  2.08176950e-01,  1.93916651e-01,  1.89727947e-01,
            1.91749164e-01,  2.00028893e-01,  1.96362711e-01,  1.99664728e-01,
            2.03959390e-01,  2.00284267e-01,  2.00788753e-01,  1.91896000e-01,
            1.86010391e-01,  1.88419439e-01,  1.69484519e-01,  1.76768598e-01,
            1.55809416e-01,  1.60284388e-01,  1.60050308e-01,  1.65278103e-01,
            1.65830735e-01,  1.57206638e-01,  1.54472647e-01,  1.59858442e-01,
            1.57847135e-01,  1.62651626e-01,  1.49244258e-01,  1.53454943e-01,
            1.41672588e-01,  1.38490336e-01,  1.25506570e-01,  1.35607030e-01,
            1.39787330e-01,  1.39993792e-01,  1.36439172e-01,  1.25659519e-01,
            1.23029141e-01,  1.34924433e-01,  1.35904336e-01,  1.42522747e-01,
            1.44997289e-01,  1.41228552e-01,  1.32317294e-01,  1.23341590e-01,
            1.28125800e-01,  1.27996533e-01,  1.55155514e-01,  1.53369307e-01,
            1.54412775e-01,  1.53702292e-01,  1.54169125e-01,  1.33612902e-01,
            1.40489614e-01,  1.51861018e-01,  1.57728752e-01,  1.61222817e-01,
            1.61068897e-01,  1.59427775e-01,  1.55400037e-01,  1.52651900e-01,
            1.47737645e-01,  1.37395931e-01,  1.40340627e-01,  1.50226572e-01,
            1.46197483e-01,  1.40308029e-01,  1.21421197e-01,  1.38930241e-01,
            1.40184131e-01,  1.39839831e-01,  1.24055865e-01,  1.09806348e-01,
            1.00856127e-01,  9.88510170e-02,  9.02341547e-02,  9.17257582e-02,
            8.27811774e-02,  7.00080534e-02,  5.70555912e-02,  6.32055988e-02,
            3.94764830e-02,  4.28522045e-02,  3.74137114e-02,  3.05187729e-02,
            3.87252051e-02,  3.94923267e-02,  3.74015668e-02,  2.50962459e-02,
            2.50963821e-02,  2.89683646e-02,  2.68798930e-02,  2.80748600e-02,
            2.84355434e-02,  2.18215262e-02,  1.07715823e-02,  1.61159555e-02,
            1.95811017e-02,  9.57514507e-03,  6.69807430e-03,  8.12020741e-03,
            1.69243870e-02,  9.78126752e-03,  1.55833495e-02,  1.40818389e-02,
            6.55340848e-03,  1.31012315e-02,  5.62701624e-03,  1.34499039e-03,
            4.48043205e-05,  4.17094356e-03,  6.18823227e-03,  3.72753402e-03,
           -7.07673969e-03, -3.91572916e-03, -5.91958814e-03, -1.08428006e-02,
           -6.29525371e-03, -1.59658530e-02, -1.48750412e-02, -1.57181127e-02,
           -1.58865268e-02, -1.60984234e-02, -2.15183969e-02, -1.68806988e-02,
           -3.08449691e-02, -1.79618597e-02, -1.82477549e-02, -2.06850538e-02,
           -2.33531376e-02, -2.87122992e-02, -3.44351356e-02, -3.35055489e-02,
           -2.38905310e-02, -3.34378351e-02, -4.26313496e-02, -3.33864232e-02,
           -3.74920349e-02, -4.17238496e-02, -4.24196704e-02, -3.69215407e-02,
           -2.35839315e-02, -2.64434441e-02, -3.17345454e-02, -2.83989566e-02,
           -1.87541097e-02, -2.61629838e-02, -3.39542913e-02, -4.22637339e-02,
           -4.99137560e-02, -5.03147044e-02, -4.42600704e-02, -5.29215694e-02,
           -5.77002723e-02, -6.81922378e-02, -8.15104171e-02, -8.28865403e-02,
           -9.33544664e-02, -9.30891186e-02, -7.77065561e-02, -8.17129129e-02,
           -7.91779686e-02, -7.77687168e-02, -9.09115401e-02, -9.14714036e-02,
           -8.99412638e-02, -8.55094130e-02, -8.35452516e-02, -8.03473648e-02,
           -7.05145339e-02, -6.84711029e-02, -7.56504424e-02, -7.25907429e-02,
           -8.77148850e-02, -8.63460829e-02, -8.34231392e-02, -8.65041869e-02,
           -7.72857830e-02, -7.68519423e-02, -6.77975265e-02, -6.61111143e-02,
           -6.95284094e-02, -6.66246092e-02, -7.00112007e-02, -7.14564370e-02,
           -7.65183350e-02, -8.22818187e-02, -8.55406265e-02, -7.80099161e-02,
           -6.88097082e-02, -5.25031325e-02, -4.48062247e-02, -5.41634457e-02,
           -5.06366016e-02, -3.97659351e-02, -4.12474245e-02, -5.54638103e-02,
           -5.30271222e-02, -5.96742005e-02, -5.45491414e-02, -4.65072432e-02,
           -4.83353690e-02, -3.52817373e-02, -3.92975969e-02, -3.67402942e-02,
           -2.58188999e-02, -2.26709987e-02, -2.64021111e-02, -2.99131311e-02,
           -2.85046892e-02, -2.56431114e-02, -3.60401184e-02, -3.27670081e-02,
           -4.20352215e-02, -4.20014420e-02, -3.65937837e-02, -3.06188162e-02,
           -2.33213638e-02, -2.56598279e-02, -2.12018109e-02, -1.62071416e-02,
           -1.65480096e-02, -1.03236883e-02, -1.50273594e-02, -1.71107249e-02,
           -1.74745305e-02, -1.82901330e-02, -2.03241731e-02, -2.37090930e-02,
           -2.73749272e-02, -3.51617078e-02, -2.12855465e-02, -2.67750629e-02,
           -2.59360581e-02, -3.00955922e-02, -9.37465772e-03, -7.45605973e-03,
           -2.74318193e-03, -1.17531972e-03, -6.92865550e-03, -2.54739550e-03,
           -1.27814504e-03, -5.88773079e-03, -8.08140165e-03, -7.36411787e-03,
            1.03961565e-03, -5.37335350e-03, -3.00375505e-03, -1.53300773e-02,
           -1.67501859e-02, -2.30036846e-02, -2.06013621e-02, -2.63202231e-03,
            2.01621782e-04, -1.32828132e-02, -1.16652234e-02, -1.65695406e-02,
           -2.36082523e-02, -1.79926575e-02, -1.24808324e-02, -1.75966273e-02,
           -1.39167911e-02, -2.69280199e-03,  1.85101865e-03,  5.91914082e-03,
            2.41367382e-03, -8.83765530e-03,  2.47671443e-03,  1.66400445e-03,
            1.24510033e-02,  4.31578726e-03,  2.28869350e-03, -1.65509240e-03,
           -6.50022629e-03,  2.20389736e-03, -3.45843272e-03, -1.21330069e-02,
           -1.71508676e-02, -1.21148746e-02, -1.56239778e-02, -2.19343240e-02,
           -1.78771283e-02, -1.85907945e-02, -1.60753977e-02, -1.35086261e-02,
           -9.76576414e-03, -1.50921977e-02, -1.78684385e-02, -3.36814164e-02,
           -3.16311209e-02, -4.14410373e-02, -2.89914215e-02, -2.25473301e-02,
           -2.57629369e-02, -1.76165781e-02, -2.32401748e-02, -2.30103829e-02,
           -1.71034957e-02, -2.34701330e-02, -2.18761454e-02, -1.74124711e-02,
           -8.45249334e-03, -1.73221684e-02, -2.08217833e-02, -6.58711057e-03,
           -8.93193874e-03, -1.28541805e-02, -2.10296113e-02, -2.75582436e-02,
           -2.96892786e-02, -4.28908107e-02, -4.65359855e-02, -4.21832157e-02,
           -3.89523468e-02, -4.23730403e-02, -4.35234447e-02, -3.74554706e-02,
           -4.31666932e-02, -5.03827179e-02, -5.06649512e-02, -4.16885663e-02,
           -3.50343839e-02, -1.59066900e-02,  2.05181417e-03,  1.71358670e-02,
            2.21898803e-02,  9.31244297e-03,  2.65607188e-03,  5.22659820e-03,
            4.18345569e-03,  1.08196451e-02,  1.20213135e-02,  8.40482704e-03,
           -4.68395928e-03, -1.05526245e-02, -1.33883245e-02, -6.15019733e-03,
           -6.45501228e-03, -5.37983949e-03, -1.45853639e-03, -3.43736508e-03,
           -1.68546078e-03,  8.74875036e-03,  3.61437067e-03,  4.07496288e-03,
            1.98866100e-03, -1.17937867e-02, -9.04580538e-03, -1.56106330e-02,
           -2.54882643e-02, -2.68863603e-02, -3.68644367e-02, -3.51351123e-02,
           -3.53528946e-02, -4.51757495e-02, -4.14204979e-02, -3.40504181e-02,
           -4.07807191e-02, -4.77370555e-02, -3.68161253e-02, -4.54816563e-02,
           -3.34774141e-02, -3.35884310e-02, -2.99974828e-02, -4.20394554e-02,
           -4.90762879e-02, -4.01144338e-02, -3.73313712e-02, -3.42878109e-02,
           -4.15539085e-02, -4.15586196e-02, -3.05980704e-02, -3.37577207e-02,
           -4.74569799e-02, -4.75269591e-02, -4.79279845e-02, -4.56522298e-02,
           -4.91065192e-02, -4.59344263e-02, -5.30669317e-02, -6.51494738e-02,
           -7.66573191e-02, -8.03201754e-02, -7.24630315e-02, -6.96635612e-02,
           -6.63629314e-02, -5.95251880e-02, -5.78904894e-02, -5.09333992e-02,
           -4.67477975e-02, -5.43290062e-02, -4.20522383e-02, -5.68007343e-02,
           -5.44827378e-02, -4.44252010e-02, -3.47260398e-02, -3.10970761e-02,
           -3.48853551e-02, -3.62750825e-02, -2.81176060e-02, -3.49787277e-02,
           -3.87246143e-02, -3.86092174e-02, -5.65140770e-02, -4.99958377e-02,
           -4.80609083e-02, -5.56981037e-02, -5.42298029e-02, -5.68594999e-02,
           -4.55288444e-02, -4.68104325e-02, -4.28038333e-02, -4.16365153e-02,
           -3.30788653e-02, -2.81033254e-02, -2.76490071e-02, -4.54881841e-02]),
     'measures': array([[ 0.544666,  0.332045,  0.791737, ...,  1.086002,  0.788513,
             0.621495],
           [ 1.001204,  1.069514,  0.984171, ...,  0.280087,  0.548398,
             1.145353],
           [-0.536792, -0.329686, -0.385817, ..., -0.548099, -0.537048,
            -0.523672],
           ...,
           [-1.275065, -0.993795, -0.883691, ..., -1.154584, -1.429857,
            -1.67794 ],
           [-1.476324, -1.718844, -1.346479, ..., -1.171252, -1.354426,
            -1.505219],
           [-0.115574, -0.060583, -0.216165, ..., -0.345931, -0.375502,
            -0.130501]]),
     'sigma_sqr': 1.0694258748350116}
#### A7_joszXWWbTgD7wOIaKUdRpRITKlk8Y5pGeVQ==.subsim:
    {'acceptance': 0.363666,
     'measures': array([[-0.256952,  0.025869, -0.135731, ..., -0.673818, -0.500292,
            -0.205867],
           [-1.416534, -1.375232, -1.48476 , ..., -0.672134, -0.973401,
            -1.214434],
           [-0.880511, -1.105401, -1.100565, ..., -1.014927, -0.787377,
            -0.697005],
           ...,
           [-0.824926, -1.106416, -0.963155, ..., -0.62079 , -0.452524,
            -0.445875],
           [ 0.28679 ,  0.512938,  0.564924, ..., -0.179769, -0.131845,
             0.407193],
           [ 0.039746,  0.309822, -0.044813, ...,  0.141513, -0.040296,
             0.019375]])}
#### 8ojNvSloPMbYG1S8BZhsC-QEe2nOlCPrpUcs7Q==.subsim:
    {'acceptance': 0.36369043599999995,
     'correlations': array([ 1.01750879e+00,  9.96139464e-01,  9.81740062e-01,  9.52124763e-01,
            9.44237401e-01,  9.24586003e-01,  9.20660371e-01,  9.05463937e-01,
            8.89198915e-01,  8.70596403e-01,  8.43048941e-01,  8.39798291e-01,
            8.27069424e-01,  8.08300319e-01,  7.98306888e-01,  7.87771502e-01,
            7.63988823e-01,  7.56700569e-01,  7.35216641e-01,  7.20903768e-01,
            7.13103240e-01,  6.97873452e-01,  6.85059555e-01,  6.68566882e-01,
            6.57322627e-01,  6.41154258e-01,  6.30038108e-01,  6.15725620e-01,
            6.06906964e-01,  6.01872613e-01,  5.95389707e-01,  5.94435891e-01,
            5.73219472e-01,  5.62766695e-01,  5.51531204e-01,  5.42856422e-01,
            5.18603526e-01,  5.01412788e-01,  4.98962334e-01,  4.81082526e-01,
            4.66949522e-01,  4.72505402e-01,  4.70286882e-01,  4.59639305e-01,
            4.35995278e-01,  4.36893458e-01,  4.29426521e-01,  4.17720525e-01,
            4.11303195e-01,  4.03842087e-01,  3.81802153e-01,  3.80009750e-01,
            3.73482591e-01,  3.73812220e-01,  3.69390724e-01,  3.61068030e-01,
            3.45621019e-01,  3.32545163e-01,  3.20751389e-01,  3.19696348e-01,
            3.11957211e-01,  3.05275283e-01,  2.91728565e-01,  2.84623700e-01,
            2.84992223e-01,  2.72732762e-01,  2.63077291e-01,  2.60945531e-01,
            2.61793308e-01,  2.52107826e-01,  2.41348820e-01,  2.29320282e-01,
            2.25575372e-01,  2.28512960e-01,  2.21940227e-01,  2.18170464e-01,
            2.23078761e-01,  2.13701864e-01,  2.10378611e-01,  2.14966723e-01,
            2.11999224e-01,  2.02446569e-01,  2.01442047e-01,  1.96039097e-01,
            1.89602993e-01,  1.86132767e-01,  1.88406059e-01,  1.80080945e-01,
            1.92242549e-01,  2.09120319e-01,  2.02685135e-01,  1.92075184e-01,
            1.89360218e-01,  1.81350266e-01,  1.75625313e-01,  1.78023507e-01,
            1.77134083e-01,  1.71950411e-01,  1.73317152e-01,  1.60005869e-01,
            1.56330130e-01,  1.53009447e-01,  1.55720129e-01,  1.48626825e-01,
            1.53112332e-01,  1.58529434e-01,  1.54506020e-01,  1.54337138e-01,
            1.61011550e-01,  1.55595517e-01,  1.52654057e-01,  1.46839408e-01,
            1.42077086e-01,  1.42669203e-01,  1.32661779e-01,  1.31788755e-01,
            1.32638072e-01,  1.31243034e-01,  1.36082585e-01,  1.16754723e-01,
            1.16564268e-01,  1.07145628e-01,  1.04575897e-01,  1.08364658e-01,
            1.08829748e-01,  1.05962176e-01,  9.93676081e-02,  9.16864256e-02,
            8.54355247e-02,  9.00502213e-02,  9.56011570e-02,  1.02031144e-01,
            8.36637741e-02,  7.29699964e-02,  7.72624321e-02,  7.37903997e-02,
            7.25586459e-02,  6.01888978e-02,  5.07839663e-02,  4.29343769e-02,
            3.94745805e-02,  4.59480766e-02,  4.21497684e-02,  5.01762274e-02,
            4.86794133e-02,  5.17240723e-02,  3.97498293e-02,  4.09532002e-02,
            2.91265131e-02,  2.84296278e-02,  3.10756127e-02,  3.50052283e-02,
            2.73761427e-02,  2.37730499e-02,  3.19601585e-02,  2.18235383e-02,
            2.08849301e-02,  2.90234102e-02,  3.36162212e-02,  2.84991617e-02,
            2.41354035e-02,  1.55522958e-02,  2.04235374e-02,  1.76609059e-02,
            1.34612747e-02,  2.55080502e-02,  1.42464331e-02,  1.10559895e-02,
            9.05378234e-03,  7.99629975e-03,  1.67444413e-02,  3.29984311e-02,
            2.75697179e-02,  3.32735097e-02,  3.13789034e-02,  4.10326380e-02,
            2.99794460e-02,  2.74090639e-02,  2.52813064e-02,  2.83241460e-02,
            2.26179379e-02,  1.51305722e-02,  2.08041768e-02,  2.60474962e-02,
            3.20354443e-02,  3.55028983e-02,  3.23578815e-02,  2.32284279e-02,
            2.54564913e-02,  2.31237768e-02,  2.41547496e-02,  3.10455197e-02,
            3.16952913e-02,  2.97839418e-02,  2.17150494e-02,  2.66724032e-02,
            2.75359085e-02,  3.30122905e-02,  3.64711653e-02,  2.99320682e-02,
            3.25092841e-02,  3.00737959e-02,  2.17187276e-02,  1.72077280e-02,
            1.67949573e-02,  1.14499539e-02,  2.07381528e-03, -2.02547725e-03,
           -1.35730974e-02, -9.98585619e-03, -5.27863688e-03,  1.42280390e-03,
           -3.20863850e-03, -3.31244291e-04,  1.11693618e-02, -3.48627036e-03,
           -7.62865786e-03, -1.38602721e-02, -1.75718056e-02, -1.65010518e-02,
           -1.06302797e-02, -6.33107332e-03, -8.05942429e-03, -5.54444405e-04,
           -1.55608719e-03, -7.05406237e-03, -4.92852975e-03, -1.56292948e-02,
           -1.50151602e-02, -1.82212000e-02, -1.04386689e-02, -1.64627549e-03,
            4.89908007e-03,  1.36616800e-02,  2.23679427e-02,  1.84939360e-02,
            3.45292831e-02,  3.52437725e-02,  4.21528418e-02,  3.15537796e-02,
            3.91993294e-02,  3.20137276e-02,  3.19216200e-02,  3.10216106e-02,
            2.39325428e-02,  3.01687477e-02,  3.28292926e-02,  3.59712761e-02,
            3.25771721e-02,  2.93541868e-02,  2.99751732e-02,  4.33733924e-02,
            3.41193571e-02,  2.94011381e-02,  2.37060996e-02,  2.02223422e-02,
            1.93207317e-02,  9.04064071e-03,  1.78294446e-02,  1.35191799e-02,
            8.65905894e-03, -3.24289867e-03, -1.24874570e-02, -8.81608740e-03,
           -1.27662349e-02, -2.65201802e-02, -4.59282215e-02, -4.32115185e-02,
           -3.85339093e-02, -4.21326106e-02, -3.57608967e-02, -3.53934097e-02,
           -4.20886370e-02, -4.14736200e-02, -4.65858879e-02, -3.93428743e-02,
           -3.14825002e-02, -3.74497956e-02, -3.41414054e-02, -3.71699125e-02,
           -4.23368552e-02, -3.82096136e-02, -2.75663802e-02, -1.93714016e-02,
           -1.28412976e-02, -7.47397341e-03, -1.00596199e-02, -1.95384911e-02,
           -1.57514293e-02, -1.46657633e-02, -1.74475301e-02, -1.88430733e-02,
           -3.02855775e-02, -3.13336588e-02, -2.18990409e-02, -1.01430604e-02,
           -4.82709076e-03,  1.86551729e-03,  6.21170608e-03,  2.99392801e-06,
            9.31373720e-03,  1.76195761e-02,  9.66726916e-03, -1.94840958e-03,
            4.32634614e-03,  6.97740871e-03, -2.95899427e-04, -6.81919692e-03,
           -1.82193125e-02, -1.78277920e-02, -2.10700472e-02, -2.02591467e-02,
           -2.61423113e-02, -2.60845300e-02, -2.40742447e-02, -4.12992665e-03,
           -3.61030036e-03,  6.03951925e-04, -5.71523437e-03, -1.34378570e-02,
           -1.39158192e-02, -2.11956944e-02, -1.41766979e-02, -1.56431443e-02,
           -1.65512887e-02, -1.06988635e-02, -1.38193054e-02, -1.38506583e-02,
           -1.44644291e-02, -1.75238128e-02, -1.55298789e-02, -2.03227186e-02,
           -1.92499987e-02, -8.41763660e-03, -6.03299948e-03, -8.61976333e-03,
           -1.32199034e-02, -3.21905428e-02, -3.10943237e-02, -3.75679870e-02,
           -3.51947371e-02, -4.23135773e-02, -4.11296639e-02, -4.71066443e-02,
           -2.83376107e-02, -3.09889617e-02, -4.54387884e-02, -4.79182115e-02,
           -5.40133417e-02, -4.21264621e-02, -3.83450248e-02, -4.49105790e-02,
           -4.55511090e-02, -4.16238874e-02, -4.10885368e-02, -4.29407972e-02,
           -4.07585903e-02, -4.01417640e-02, -3.47785861e-02, -3.01352270e-02,
           -3.69394466e-02, -3.60926008e-02, -3.18365250e-02, -2.74048339e-02,
           -2.81998271e-02, -3.15259332e-02, -3.77766704e-02, -2.12431344e-02,
           -2.08259740e-02, -1.66357035e-02, -1.51117379e-02, -2.14659152e-02,
           -1.51485322e-02, -2.09932039e-02, -1.95287215e-02, -2.05343922e-02,
           -2.07695540e-02, -2.54196957e-02, -2.60444191e-02, -2.41184184e-02,
           -2.58946684e-02, -2.87841119e-02, -2.46388663e-02, -2.29856607e-02,
           -1.75399093e-02, -1.42251692e-02, -1.81082545e-02, -1.82733563e-02,
           -2.47838337e-02, -3.57693423e-02, -3.03883903e-02, -3.35640706e-02,
           -3.36944360e-02, -4.01643117e-02, -3.82528249e-02, -2.17784389e-02,
           -1.72699169e-02, -1.76258316e-02, -2.10435609e-02, -2.14974288e-02,
           -2.09245940e-02, -2.35577257e-02, -2.41833670e-02, -2.29522035e-02,
           -2.04493647e-02, -4.63971868e-03, -1.97604295e-03, -6.02841538e-03,
           -1.33053072e-02, -1.59503581e-02,  2.79977051e-03,  6.11655398e-03,
            1.48724953e-02,  4.44183078e-04,  2.55026132e-03,  9.11902834e-03,
            1.18852715e-02,  1.32109732e-02,  1.38148042e-02,  1.08641281e-02,
            1.00241548e-02,  1.66232373e-02,  1.87338245e-02,  1.81291248e-02,
            2.96362553e-02,  1.44219562e-02,  5.31975094e-03,  2.84894046e-03,
            3.84596322e-03,  4.71675439e-04,  5.14423032e-03,  9.79548976e-03,
            2.72195650e-04,  1.23498337e-03, -6.37402517e-04,  2.75739008e-03,
            2.21211743e-03,  7.90002127e-04,  3.94846305e-03,  9.93094820e-03,
            7.74629961e-03,  1.41048972e-02,  1.69720588e-02,  1.74682848e-02,
            1.55085173e-02,  8.15985981e-03,  1.41925737e-02,  5.65890706e-03,
            1.20714996e-02,  2.73799179e-02,  2.83772330e-02,  2.85376364e-02,
            3.00083261e-02,  3.33207384e-02,  3.56924314e-02,  3.17231336e-02,
            2.38882200e-02,  3.48255674e-02,  3.09388679e-02,  3.19077908e-02,
            3.19082786e-02,  3.52633271e-02,  4.18204253e-02,  3.46598862e-02,
            3.23695915e-02,  3.33204381e-02,  3.54598344e-02,  2.70041556e-02,
            2.70766334e-02,  2.15530947e-02,  1.42206416e-02,  2.06257423e-02,
            1.00615421e-02,  1.26236370e-02,  9.49081789e-03,  6.69516469e-03,
            5.22859460e-03,  2.92239103e-03,  4.17839299e-03,  1.20298888e-02,
            9.46838851e-03,  4.22191375e-03,  2.24194806e-03,  1.00117887e-03,
            2.34847115e-03,  6.80353915e-04,  6.81439312e-03,  5.15954745e-03,
            1.10961622e-02,  1.16896782e-02,  1.02135033e-02,  7.09372065e-03,
           -2.61551413e-03, -1.11156559e-02,  6.24324305e-04, -2.17676421e-03,
            5.03401515e-04,  1.67483369e-03,  1.14935677e-02,  5.56365272e-03]),
     'measures': array([[-0.566443, -0.431104, -0.252759, ..., -0.355003, -0.385949,
            -0.303025],
           [-0.030588, -0.008156,  0.009422, ..., -0.383157, -0.183157,
            -0.191034],
           [-1.304676, -1.273468, -1.217371, ..., -1.101134, -1.111586,
            -1.262218],
           ...,
           [ 0.184222, -0.387785, -0.151873, ...,  0.576839,  0.265744,
             0.153551],
           [ 0.840467,  0.555614,  0.58345 , ...,  0.555548,  0.680772,
             0.89337 ],
           [-0.030013,  0.140188,  0.036443, ..., -0.319547, -0.244869,
            -0.208579]]),
     'sigma_sqr': 0.836172367580062}
#### ulSag_5IDyqA7nWMQY5Vrrnz9vQn-OlwQXzm0Q==.subsim:
    {'DeltaE': (0.18313165194537714, 0.0009761644799763619),
     'acceptance': 0.46023989,
     'correlations': array([1.81750659, 1.788075  , 1.75734239, 1.72385612, 1.68649217,
           1.64304695, 1.61198076, 1.58678782, 1.55264402, 1.5260032 ,
           1.50199951, 1.47352558, 1.43962899, 1.42149985, 1.3909553 ,
           1.3610176 , 1.33514097, 1.30448226, 1.29757422, 1.2701826 ,
           1.24746499, 1.22563986, 1.19370639, 1.17223082, 1.1484893 ,
           1.13733108, 1.11257006, 1.09341904, 1.08286293, 1.04986515,
           1.01851176, 0.99561341, 0.98013878, 0.97668096, 0.95059324,
           0.93773408, 0.92253878, 0.91446979, 0.8899671 , 0.86627221,
           0.8414291 , 0.81592068, 0.80399659, 0.79471034, 0.79586819,
           0.77647566, 0.77594313, 0.75980475, 0.74392315, 0.72508813,
           0.71584361, 0.70086885, 0.70153002, 0.69077941, 0.67291461,
           0.66657425, 0.63800236, 0.61268878, 0.59184665, 0.57580067,
           0.56715672, 0.56126306, 0.55169502, 0.5437152 , 0.52494829,
           0.51807468, 0.50753412, 0.50005324, 0.49069905, 0.4714788 ,
           0.46307091, 0.46032544, 0.46192183, 0.45791568, 0.45232783,
           0.4539375 , 0.46764833, 0.44973752, 0.4454759 , 0.43656991,
           0.42852094, 0.40752026, 0.40176696, 0.39199369, 0.39123274,
           0.38313062, 0.38514873, 0.3629576 , 0.35215237, 0.33746114,
           0.33179711, 0.33322869, 0.32733713, 0.33857992, 0.33768473,
           0.34170026, 0.31964946, 0.3071853 , 0.30509053, 0.30950055,
           0.29733192, 0.27561031, 0.28059282, 0.26195488, 0.24841997,
           0.25178792, 0.23524218, 0.22871326, 0.22807265, 0.2139173 ,
           0.21596202, 0.20539464, 0.21013461, 0.2142169 , 0.20463093,
           0.21061914, 0.21708212, 0.21043316, 0.204318  , 0.19284453,
           0.19145722, 0.18944363, 0.17872384, 0.17510093, 0.1708679 ,
           0.18095169, 0.18987683, 0.188163  , 0.1845018 , 0.16838469,
           0.1621017 , 0.16118114, 0.16398518, 0.16325228, 0.17212326,
           0.17222702, 0.1660985 , 0.16144934, 0.15849264, 0.15747229,
           0.15777342, 0.16459533, 0.15806911, 0.14544123, 0.14555304,
           0.14852368, 0.1392524 , 0.16050294, 0.14855909, 0.13934705]),
     'measures': array([[-0.333807, -0.424492, -0.41733 , ..., -0.18398 , -0.636275,
            -0.357849],
           [ 0.766624,  0.730022,  0.352318, ...,  0.432383,  0.426824,
             0.7187  ],
           [ 1.576197,  1.573124,  1.532875, ...,  1.506551,  1.799419,
             1.691761],
           ...,
           [-2.309596, -2.386343, -2.423862, ..., -1.420566, -1.688527,
            -1.864865],
           [-0.762359, -0.914134, -0.647599, ..., -1.329545, -1.008645,
            -0.309593],
           [-0.126036, -0.143064,  0.086453, ..., -0.625272, -0.167664,
            -0.199029]]),
     'meff': (1.1927053445454978, 0.006357593458341944),
     'sigma_sqr': 1.119118507962507}
#### -ltzAh3tG74yZFezyLNOHEXv9VrWDX2T1jLVNw==.subsim:
    {'acceptance': 0.180252691,
     'measures': array([[-0.165936, -0.309701, -0.275167, ..., -0.218889, -0.132828,
            -0.15523 ],
           [-0.160246, -0.194972, -0.187411, ..., -0.134407, -0.00632 ,
            -0.075996],
           [-0.244019, -0.227215, -0.236562, ..., -0.170696, -0.110471,
            -0.138969],
           ...,
           [-2.101008, -2.117342, -2.065551, ..., -2.271176, -2.139993,
            -2.236199],
           [-1.87391 , -1.960895, -1.971233, ..., -1.563702, -1.584335,
            -1.628969],
           [-1.715206, -1.781849, -1.920868, ..., -1.49753 , -1.642074,
            -1.612512]])}
#### wB7MuO5TfxuzLlf-sKsrbEPK10gU8_AEr1jynA==.subsim:
    {'acceptance': 0.3506294,
     'measures': array([[ 0.78198 ,  0.775101,  0.33541 , ...,  0.4094  ,  0.558025,
             0.576121],
           [-0.263129, -0.039694, -0.190398, ..., -0.5664  , -0.672515,
            -0.531953],
           [-0.271833, -0.453915, -0.220909, ..., -0.247951,  0.004377,
            -0.080586],
           ...,
           [ 1.867221,  1.782414,  1.645182, ...,  1.54808 ,  1.667566,
             1.795463],
           [ 1.284702,  1.192405,  1.322957, ...,  1.346533,  1.424533,
             1.181602],
           [-0.298834, -0.233115, -0.180442, ..., -0.022274, -0.119989,
            -0.095336]])}
#### bDVlaklZqKwHMN_kZdgAN3LgNzx5Pq8cmZNaoA==.subsim:
    {'DeltaE': (0.2024360374796034, 0.001474468014959838),
     'acceptance': 0.289128435,
     'correlations': array([0.40074433, 0.39096981, 0.37901507, 0.37312087, 0.36963993,
           0.36408707, 0.35649575, 0.34655611, 0.34005123, 0.33372264,
           0.32526428, 0.31678108, 0.31338059, 0.30771669, 0.30596112,
           0.30215024, 0.30026332, 0.29377919, 0.2862956 , 0.28190632,
           0.27485757, 0.26950464, 0.26560326, 0.26496625, 0.26211098,
           0.25969641, 0.25224926, 0.24368733, 0.23949229, 0.2289117 ,
           0.22511675, 0.21804932, 0.20846238, 0.20582941, 0.20644128,
           0.20467481, 0.20202357, 0.19708713, 0.19511535, 0.19288485,
           0.18888908, 0.18332554, 0.18469658, 0.18000017, 0.17391336,
           0.17427591, 0.16784069, 0.16458608, 0.16552049, 0.16136564,
           0.15424652, 0.14581922, 0.13990879, 0.13912899, 0.13495134,
           0.13556986, 0.13179786, 0.13035123, 0.126805  , 0.12070141,
           0.11899912, 0.11264323, 0.11063134, 0.11207777, 0.10963003,
           0.10483973, 0.10582973, 0.10277361, 0.0985263 , 0.09410237,
           0.09086792, 0.09044955, 0.08964097, 0.08837521, 0.08679507,
           0.08326268, 0.08366317, 0.08348078, 0.08172607, 0.0799158 ,
           0.07858812, 0.07644669, 0.07219616, 0.07279424, 0.07057861,
           0.07226933, 0.06767651, 0.0670956 , 0.06519417, 0.06819077,
           0.06885298, 0.06731726, 0.06545652, 0.06429886, 0.06236451,
           0.06240201, 0.06416223, 0.06233972, 0.06457261, 0.06209661,
           0.06527158, 0.06592476, 0.05966207, 0.05868453, 0.05828252,
           0.05561076, 0.06134293, 0.0608468 , 0.060006  , 0.06089427,
           0.05970587, 0.05738796, 0.05749628, 0.05650144, 0.06013367,
           0.06143173, 0.06040795, 0.06399353, 0.06173369, 0.06321691,
           0.06128272, 0.06043995, 0.0590951 , 0.06249874, 0.06207018,
           0.06254032, 0.06155086, 0.06037987, 0.05820371, 0.05273648,
           0.05048619, 0.04932454, 0.05195856, 0.05214975, 0.05123077,
           0.05141768, 0.0513012 , 0.05201131, 0.05211511, 0.05082634,
           0.05040628, 0.05137103, 0.04813842, 0.04661449, 0.04610965,
           0.04831977, 0.04632131, 0.04229961, 0.04229229, 0.0440386 ]),
     'measures': array([[ 0.613295,  0.630794,  0.721108, ..., -0.346923, -0.139877,
             0.057677],
           [ 0.631077,  0.768148,  0.594056, ...,  0.844524,  1.100046,
             0.891485],
           [-0.300926, -0.376918, -0.328615, ..., -0.308055, -0.292408,
            -0.284571],
           ...,
           [-1.161671, -1.204543, -1.030731, ..., -1.075141, -1.080358,
            -1.078972],
           [-1.589957, -1.75983 , -1.859681, ..., -1.335098, -1.319095,
            -1.409655],
           [-1.348414, -1.358502, -1.334321, ..., -1.151346, -1.154552,
            -1.507229]]),
     'meff': (0.9760775764995626, 0.0071093822255492755),
     'sigma_sqr': 0.2396446682853267}
#### MrthznqgEA2aNpk-X15HZ-huDEIi6WrOHn1eeA==.subsim:
    {'DeltaE': (0.13220920595656682, 0.002737102710297815),
     'acceptance': 0.27575105,
     'correlations': array([0.3235837 , 0.3165299 , 0.30846755, 0.30364637, 0.29833793,
           0.29602246, 0.28730207, 0.28353636, 0.27931814, 0.27913853,
           0.27434762, 0.26863169, 0.26410902, 0.25876573, 0.25491335,
           0.2511699 , 0.24809659, 0.24521102, 0.24069447, 0.23643588,
           0.23549459, 0.23249312, 0.23026594, 0.2260175 , 0.22549754,
           0.22258981, 0.21654841, 0.21088371, 0.20554511, 0.20298635,
           0.19839268, 0.19507822, 0.1899376 , 0.18520553, 0.18207865,
           0.17862959, 0.1780985 , 0.17487818, 0.17145571, 0.16490067,
           0.16410241, 0.15940419, 0.15873816, 0.15358658, 0.15125526,
           0.15224065, 0.15015059, 0.14543308, 0.14137346, 0.14183522,
           0.13837604, 0.13858581, 0.13646717, 0.13317348, 0.12963947,
           0.12811444, 0.12118537, 0.11771244, 0.11860005, 0.11927491,
           0.11603966, 0.11643602, 0.11349035, 0.11357997, 0.1134393 ,
           0.11380101, 0.11334534, 0.1106852 , 0.11372911, 0.11271941,
           0.10993068, 0.10997205, 0.10771689, 0.10662466, 0.10672901,
           0.10611099, 0.1059565 , 0.1029988 , 0.10302999, 0.1064822 ,
           0.10744071, 0.1061269 , 0.10641899, 0.10193961, 0.10152084,
           0.10270942, 0.10195791, 0.09958706, 0.09770004, 0.09321053,
           0.09524552, 0.09554165, 0.09612181, 0.09751594, 0.09751751,
           0.09803184, 0.09803785, 0.0958474 , 0.0922455 , 0.09109423,
           0.09324183, 0.0926071 , 0.09384861, 0.09431677, 0.09974381,
           0.0995462 , 0.09587952, 0.09505294, 0.0945329 , 0.09422887,
           0.09152108, 0.09268855, 0.08856385, 0.08542727, 0.08178771,
           0.08068029, 0.08111805, 0.07487542, 0.0732939 , 0.0738255 ,
           0.07352561, 0.07500826, 0.07374083, 0.07250141, 0.07143253,
           0.07381225, 0.0700508 , 0.07214961, 0.07136125, 0.06856551,
           0.06785512, 0.07020284, 0.07244455, 0.06988483, 0.07078442,
           0.07311794, 0.07381915, 0.07216592, 0.06979279, 0.07221271,
           0.07077821, 0.0713045 , 0.07500253, 0.07420687, 0.07288542,
           0.0704538 , 0.07466683, 0.07631939, 0.07758379, 0.07687589]),
     'measures': array([[ 0.07596 ,  0.312531,  0.307754, ..., -0.251842, -0.165197,
            -0.070962],
           [-0.238077, -0.331594, -0.43573 , ..., -0.383412, -0.249453,
            -0.267987],
           [-0.884739, -1.212005, -0.762165, ..., -0.896847, -1.042854,
            -1.11974 ],
           ...,
           [-0.377464, -0.484444, -0.364321, ..., -0.296744, -0.110014,
            -0.370709],
           [-0.138414, -0.2032  , -0.100977, ..., -0.342481, -0.221975,
            -0.282176],
           [-0.230185, -0.113072, -0.346233, ..., -0.053516, -0.309252,
            -0.277867]]),
     'meff': (2.2884245467165925, 0.04737682965275322),
     'sigma_sqr': 0.19868423055296586}
#### wuIxFYiwx9rWLPjp4AG1rHfGmi1ya2kaJxfJLQ==.subsim:
    {'acceptance': 0.16289526499999998,
     'measures': array([[-0.045576, -0.062578, -0.188751, ..., -0.022611,  0.059858,
            -0.111471],
           [-0.160843, -0.155515, -0.166859, ..., -0.255182, -0.204955,
            -0.241418],
           [-0.068252, -0.087195, -0.255208, ..., -0.122721, -0.123471,
            -0.066254],
           ...,
           [ 0.688999,  0.756092,  0.731979, ...,  0.698559,  0.652316,
             0.754754],
           [ 0.940338,  0.856365,  0.900829, ...,  0.956638,  0.976607,
             1.009988],
           [ 0.919023,  0.845768,  0.684881, ...,  0.961919,  0.858712,
             0.812534]])}
#### 7oIjbBOvisFod-VFfQ48beVF7P8vgighY6bG_A==.subsim:
    {'acceptance': 0.424489556,
     'correlations': array([ 1.37590025e+00,  1.34920494e+00,  1.30929878e+00,  1.30174529e+00,
            1.27492861e+00,  1.25435533e+00,  1.22038849e+00,  1.19312552e+00,
            1.17006463e+00,  1.14897176e+00,  1.13033203e+00,  1.10440569e+00,
            1.08277381e+00,  1.06132468e+00,  1.03676404e+00,  9.98350400e-01,
            9.93037840e-01,  9.75727690e-01,  9.55754454e-01,  9.31661186e-01,
            9.11691085e-01,  9.01883504e-01,  8.76462307e-01,  8.59374609e-01,
            8.45405105e-01,  8.35013862e-01,  8.12586139e-01,  7.91599629e-01,
            7.67672628e-01,  7.58156045e-01,  7.51798504e-01,  7.43519745e-01,
            7.34488953e-01,  7.15746009e-01,  6.92594574e-01,  6.81186682e-01,
            6.65691413e-01,  6.50089489e-01,  6.29897167e-01,  6.13452895e-01,
            5.97190756e-01,  5.88842745e-01,  5.83741385e-01,  5.63333002e-01,
            5.46604132e-01,  5.43230044e-01,  5.21329884e-01,  5.14646634e-01,
            5.01898019e-01,  4.79588240e-01,  4.67737893e-01,  4.46699042e-01,
            4.42745361e-01,  4.24899408e-01,  4.21367125e-01,  4.26613366e-01,
            4.21393389e-01,  4.09847214e-01,  4.03875015e-01,  3.98282473e-01,
            3.90820881e-01,  3.88801720e-01,  3.80898710e-01,  3.69102122e-01,
            3.39925988e-01,  3.41634137e-01,  3.40837672e-01,  3.38598722e-01,
            3.40602308e-01,  3.27075758e-01,  3.23494438e-01,  3.10841590e-01,
            2.97174807e-01,  2.89964039e-01,  2.88293181e-01,  2.75686989e-01,
            2.67053776e-01,  2.47241560e-01,  2.48074290e-01,  2.36952577e-01,
            2.33962141e-01,  2.24731366e-01,  2.06621633e-01,  1.89970409e-01,
            2.00417241e-01,  1.95321075e-01,  1.86994028e-01,  1.92542949e-01,
            1.64541904e-01,  1.59856393e-01,  1.45634541e-01,  1.39103679e-01,
            1.45804405e-01,  1.54204028e-01,  1.47517618e-01,  1.50356195e-01,
            1.56049430e-01,  1.50529326e-01,  1.38995850e-01,  1.43254942e-01,
            1.47462404e-01,  1.30783498e-01,  1.20316958e-01,  1.10032873e-01,
            9.47837148e-02,  9.23285822e-02,  9.41896446e-02,  1.00013127e-01,
            9.08936441e-02,  1.08302667e-01,  1.14527950e-01,  1.00369174e-01,
            1.00147973e-01,  1.06553710e-01,  1.13182038e-01,  9.78517170e-02,
            1.03319072e-01,  1.15425619e-01,  1.11122541e-01,  1.15143601e-01,
            1.10426552e-01,  1.04782097e-01,  9.28375814e-02,  7.06268400e-02,
            5.99744324e-02,  6.35296108e-02,  5.29161014e-02,  6.61081077e-02,
            5.51766266e-02,  5.33233891e-02,  6.43048473e-02,  6.85785439e-02,
            6.97335665e-02,  7.12050297e-02,  7.00472148e-02,  7.66497158e-02,
            6.52367299e-02,  6.12441001e-02,  3.87892058e-02,  3.66497808e-02,
            1.89566484e-02,  2.02514907e-02,  2.91145609e-02,  3.50840929e-02,
            4.41831872e-02,  4.76567807e-02,  4.12853177e-02,  2.06039908e-02,
            2.55249983e-02,  2.43943517e-02,  2.79612737e-02,  3.29654462e-02,
            4.05583581e-02,  4.17618171e-02,  4.11669207e-02,  3.29617654e-02,
            4.18714383e-02,  2.94584249e-02,  3.23596618e-02,  2.61523582e-02,
            3.32458645e-02,  3.79759008e-02,  3.22226952e-02,  3.12505957e-02,
            4.01991439e-02,  2.68369516e-02,  3.74164668e-02,  3.87483780e-02,
            4.50368799e-02,  3.34519122e-02,  1.53667869e-02,  2.19558845e-02,
            1.99579191e-02,  1.60272411e-02,  5.50958318e-03, -3.48976589e-04,
           -3.41883453e-03,  1.28097046e-02,  6.72557178e-03, -2.18491172e-03,
            4.72383876e-03,  1.25583304e-03,  2.41694448e-03,  1.00742008e-02,
            1.49175067e-02,  1.98493061e-02,  3.15161999e-02,  2.75698421e-02,
            3.91286545e-02,  4.26991442e-02,  2.97189141e-02,  4.52478007e-02,
            4.84159270e-02,  4.54046226e-02,  3.70950007e-02,  3.13157592e-02,
            3.08278211e-02,  3.12587358e-02,  2.64623321e-02,  4.32992131e-02,
            5.07375296e-02,  5.71877439e-02,  5.68976863e-02,  6.39006441e-02,
            5.90973928e-02,  4.56593995e-02,  5.04015609e-02,  5.70350519e-02,
            5.73842475e-02,  4.85192822e-02,  3.83337855e-02,  3.24239754e-02,
            3.85354832e-02,  5.08294312e-02,  4.02034433e-02,  3.13567598e-02,
            2.37159911e-02,  1.38381800e-02,  9.23435754e-03,  6.21836741e-04,
            5.90888139e-03,  6.07027827e-04, -1.44854968e-03, -1.20987807e-02,
           -2.18390325e-02, -3.41480207e-02, -3.33270687e-02, -1.46118588e-02,
           -1.77230599e-02, -7.18110528e-03, -5.09826058e-03, -1.73243354e-02,
           -1.70973804e-04,  1.43469659e-02,  1.79758275e-02,  6.78870812e-03,
           -1.50116390e-03,  5.89714615e-03,  2.04739065e-02,  2.11198049e-02,
            1.59688644e-02,  2.03613051e-02,  2.34778267e-02,  3.11047057e-02,
            3.94685789e-02,  3.53649590e-02,  5.12884170e-02,  5.57748677e-02,
            5.29916853e-02,  5.98990358e-02,  5.36332327e-02,  5.94977002e-02,
            5.76283296e-02,  5.02398947e-02,  3.69049209e-02,  2.31007858e-02,
            2.72780826e-02,  2.82762920e-02,  2.71374038e-02,  2.77024324e-02,
            1.70288500e-02,  2.15516365e-02,  1.88414559e-02,  1.90069534e-02,
            2.43662732e-03,  1.88004621e-02,  1.36678625e-02,  1.79486182e-02,
            1.90042877e-02,  2.08642092e-02,  2.51011979e-02,  2.26483413e-02,
            1.68310158e-02,  2.04539684e-02,  2.84317220e-02,  2.98190508e-02,
            3.37015266e-02,  4.51620623e-02,  4.25674517e-02,  3.36013376e-02,
            4.73498532e-02,  4.04007525e-02,  4.81237048e-02,  3.35699515e-02,
            2.57900847e-02,  2.80379757e-02,  3.50328754e-02,  2.96707352e-02,
            2.69705418e-02,  2.86018923e-02,  1.76845730e-02,  1.18744819e-02,
           -8.83235000e-04,  8.34845344e-03,  1.17345012e-02,  1.03725172e-02,
            7.24195275e-03,  1.31064652e-02,  1.12388782e-02,  7.74985402e-03,
            1.82851278e-02,  2.68562462e-02,  2.15217356e-02,  6.97990404e-03,
            6.85578466e-03,  1.13532177e-02,  1.30865119e-02,  9.06364984e-03,
            2.08317299e-02,  2.98865327e-02,  4.33678708e-02,  3.11096444e-02,
            3.42170071e-02,  3.21376456e-02,  3.42620032e-02,  3.38348012e-02,
            4.22411547e-02,  5.04527311e-02,  4.41285399e-02,  4.62261420e-02,
            4.11438639e-02,  4.47564766e-02,  5.28949380e-02,  4.93744127e-02,
            4.04354207e-02,  2.88983453e-02,  4.64015965e-02,  5.56090862e-02,
            5.45983516e-02,  7.19736843e-02,  6.68335851e-02,  5.22945280e-02,
            6.12366488e-02,  5.84508641e-02,  6.88681585e-02,  5.54448854e-02,
            6.72564782e-02,  6.53085280e-02,  7.78549430e-02,  8.46493737e-02,
            8.11458616e-02,  7.40564496e-02,  7.73175929e-02,  7.20382027e-02,
            7.10980621e-02,  7.13305229e-02,  7.52666845e-02,  6.63410201e-02,
            6.78075230e-02,  7.11312467e-02,  6.52152869e-02,  7.01146881e-02,
            7.63353317e-02,  6.77271835e-02,  7.41925904e-02,  7.50347504e-02,
            8.41281206e-02,  8.61466796e-02,  7.73537065e-02,  8.07981977e-02,
            7.51296234e-02,  7.37849347e-02,  7.60582716e-02,  7.14545827e-02,
            6.88650675e-02,  6.73645702e-02,  5.97434993e-02,  6.01751039e-02,
            6.07027933e-02,  6.44505268e-02,  6.28541495e-02,  5.40411246e-02,
            5.36930245e-02,  7.05610324e-02,  7.60953713e-02,  7.62304953e-02,
            8.13432268e-02,  7.50595252e-02,  7.70840183e-02,  7.04062389e-02,
            5.64049749e-02,  6.75229044e-02,  6.09967464e-02,  6.48617716e-02,
            5.36696739e-02,  5.72301354e-02,  4.69707726e-02,  4.21040151e-02,
            4.17167419e-02,  3.44382144e-02,  3.04764162e-02,  3.69606751e-02,
            3.39027320e-02,  2.82404219e-02,  1.82455813e-02,  9.28894768e-03,
            1.21073074e-02,  9.82100874e-03,  1.51086382e-02,  9.94379318e-03,
            1.06199813e-02,  7.60728667e-03,  3.71314643e-03,  2.62156865e-02,
            1.46887384e-02,  2.70420617e-02,  1.05016475e-02,  1.66784377e-02,
            1.40591929e-02,  2.03246410e-02,  4.06852395e-02,  4.93353989e-02,
            3.77650609e-02,  3.15671766e-02,  5.02066112e-02,  4.67242838e-02,
            4.93687664e-02,  5.44461354e-02,  3.65208792e-02,  1.51318225e-02,
            2.12028427e-02,  2.89795401e-02,  1.71489883e-02,  2.37837859e-02,
            1.10018492e-02,  1.59713902e-02,  1.00457253e-02,  9.67568745e-03,
           -1.18560966e-04, -1.06737667e-02, -1.59404054e-02,  3.15926460e-03,
            1.59497040e-03, -2.79353824e-03,  5.54926143e-05, -7.35489191e-03,
           -1.04997033e-02,  1.27723464e-03, -7.19835669e-03, -2.19239138e-02,
           -3.06789166e-02, -2.73925635e-02, -3.22343432e-02, -3.00478825e-02,
           -3.74419716e-02, -3.62298440e-02, -4.46027624e-02, -5.44630157e-02,
           -5.44719878e-02, -5.02127042e-02, -5.96673934e-02, -5.57116791e-02,
           -6.37090842e-02, -6.24302516e-02, -7.66661663e-02, -6.89895010e-02,
           -8.26283684e-02, -7.87124197e-02, -7.86799829e-02, -9.13851122e-02,
           -9.42359713e-02, -8.55573027e-02, -7.17365245e-02, -7.25156757e-02,
           -6.16642541e-02, -6.35648756e-02, -5.56572525e-02, -6.36568937e-02,
           -6.09385260e-02, -6.89563630e-02, -8.54223867e-02, -8.17544474e-02,
           -9.48546046e-02, -9.63604846e-02, -1.02391682e-01, -1.04585226e-01,
           -1.14171844e-01, -1.05761587e-01, -9.09524485e-02, -8.18063151e-02,
           -8.75204753e-02, -8.48345537e-02, -7.72613721e-02, -8.05885477e-02,
           -9.14615555e-02, -8.16089168e-02, -8.81878141e-02, -9.85733959e-02,
           -1.08772355e-01, -1.18691132e-01, -1.18807256e-01, -1.23086876e-01,
           -1.21245953e-01, -1.31173523e-01, -1.32380302e-01, -1.37130803e-01,
           -1.64243824e-01, -1.83266191e-01, -1.81545590e-01, -1.83157660e-01]),
     'measures': array([[-0.887714, -0.657278, -0.679323, ..., -0.866486, -0.659672,
            -0.827101],
           [-2.007034, -1.84813 , -2.186407, ..., -1.950448, -2.339671,
            -2.091066],
           [-1.063826, -0.56232 , -0.377023, ..., -0.87821 , -0.938614,
            -1.054721],
           ...,
           [ 0.951999,  0.762553,  0.84961 , ...,  1.100867,  1.056773,
             1.474935],
           [-0.244952, -0.080451, -0.034096, ...,  0.502963,  0.034639,
             0.041691],
           [ 1.182405,  1.443824,  1.330389, ...,  1.299144,  0.853087,
             1.011446]]),
     'sigma_sqr': 1.2311918885541226}
#### ZygtZSLsRSVNFtNMhk8_rEBZ8k7_dUVukjFmcQ==.subsim:
    {'acceptance': 0.350513022,
     'correlations': array([ 8.14097445e-01,  7.95839430e-01,  7.86740174e-01,  7.70595614e-01,
            7.56834480e-01,  7.37191450e-01,  7.19966464e-01,  7.18963248e-01,
            7.08970078e-01,  6.88011021e-01,  6.71098232e-01,  6.53795318e-01,
            6.39095334e-01,  6.21002709e-01,  6.17207271e-01,  6.00320110e-01,
            5.85052808e-01,  5.79448206e-01,  5.71322750e-01,  5.52139937e-01,
            5.44077355e-01,  5.36016553e-01,  5.13669227e-01,  5.08270694e-01,
            4.95052058e-01,  4.89700390e-01,  4.73934298e-01,  4.69134902e-01,
            4.47872975e-01,  4.38567388e-01,  4.26798396e-01,  4.14336731e-01,
            4.02288835e-01,  4.00263275e-01,  3.84293613e-01,  3.79779394e-01,
            3.70959086e-01,  3.76657497e-01,  3.73867717e-01,  3.71277157e-01,
            3.63519374e-01,  3.66967278e-01,  3.68767747e-01,  3.74617061e-01,
            3.70381315e-01,  3.66754594e-01,  3.55075785e-01,  3.47719976e-01,
            3.45508529e-01,  3.39403793e-01,  3.34457193e-01,  3.26620927e-01,
            3.26323520e-01,  3.25291526e-01,  3.21473712e-01,  3.06748232e-01,
            2.94623782e-01,  2.94928402e-01,  2.85824922e-01,  2.74950974e-01,
            2.66685247e-01,  2.58470116e-01,  2.48130486e-01,  2.44610747e-01,
            2.44171663e-01,  2.47054466e-01,  2.35525657e-01,  2.17639937e-01,
            2.14060404e-01,  2.09822905e-01,  2.04929178e-01,  1.96828301e-01,
            1.90568496e-01,  1.85051118e-01,  1.74686086e-01,  1.69075185e-01,
            1.72338257e-01,  1.71734747e-01,  1.76522080e-01,  1.69874715e-01,
            1.68041858e-01,  1.60689115e-01,  1.42119044e-01,  1.33847514e-01,
            1.27739391e-01,  1.29703676e-01,  1.25029896e-01,  1.19228666e-01,
            1.18358977e-01,  1.27140894e-01,  1.21030513e-01,  1.13951641e-01,
            1.20168468e-01,  1.29923318e-01,  1.28631195e-01,  1.25139687e-01,
            1.37086971e-01,  1.26186892e-01,  1.22388078e-01,  1.18778346e-01,
            1.09404871e-01,  1.03208723e-01,  1.03992016e-01,  1.03259319e-01,
            1.08508696e-01,  1.05648267e-01,  1.09778720e-01,  1.13785743e-01,
            1.17962342e-01,  1.14405388e-01,  1.16141990e-01,  1.14241205e-01,
            1.12305443e-01,  1.04713601e-01,  9.48800005e-02,  1.03385876e-01,
            1.01623963e-01,  1.08969056e-01,  1.05527116e-01,  1.04033978e-01,
            1.13277413e-01,  1.19471734e-01,  1.15462341e-01,  1.02714744e-01,
            9.63036237e-02,  9.33366506e-02,  9.07956686e-02,  9.04721990e-02,
            9.12679899e-02,  9.74387163e-02,  9.30281660e-02,  8.87433577e-02,
            9.11914977e-02,  8.13503064e-02,  8.37433785e-02,  8.18576442e-02,
            7.65646422e-02,  7.15971118e-02,  6.39827108e-02,  5.86238279e-02,
            6.18780080e-02,  6.34935965e-02,  5.86807791e-02,  5.91168899e-02,
            6.38791856e-02,  6.01749854e-02,  5.54244962e-02,  5.48990627e-02,
            6.25280380e-02,  6.24779403e-02,  5.74056060e-02,  4.98812765e-02,
            5.58029188e-02,  6.38766049e-02,  6.49362916e-02,  5.81720193e-02,
            5.96938034e-02,  5.71121512e-02,  6.65781130e-02,  6.46985816e-02,
            6.27065458e-02,  7.22392639e-02,  6.83433588e-02,  7.18612239e-02,
            8.04565133e-02,  7.89854504e-02,  7.52256771e-02,  7.49247650e-02,
            7.50927206e-02,  7.12228074e-02,  7.99995414e-02,  8.07044127e-02,
            7.69948139e-02,  6.71015253e-02,  7.02968915e-02,  6.30713132e-02,
            6.58995542e-02,  7.20845517e-02,  6.07804239e-02,  5.38313004e-02,
            5.03085989e-02,  6.16826923e-02,  6.17639480e-02,  5.27472339e-02,
            4.29197040e-02,  6.15817451e-02,  5.58475552e-02,  5.19588705e-02,
            5.90985143e-02,  5.64875461e-02,  6.56501741e-02,  6.71654571e-02,
            6.62091630e-02,  6.64640760e-02,  6.80822906e-02,  7.55110300e-02,
            7.06565302e-02,  6.32825179e-02,  6.05093407e-02,  5.73314339e-02,
            5.66283512e-02,  4.90604244e-02,  5.61550090e-02,  5.38504769e-02,
            5.06570039e-02,  5.78086826e-02,  5.76178761e-02,  5.31057515e-02,
            5.51424303e-02,  5.24851301e-02,  5.37910136e-02,  5.89917208e-02,
            5.86627290e-02,  5.44909068e-02,  5.32276526e-02,  4.99557606e-02,
            4.89453090e-02,  4.69936477e-02,  5.34609777e-02,  5.73043319e-02,
            5.34620755e-02,  5.51726879e-02,  5.26531913e-02,  4.59215422e-02,
            4.68817907e-02,  4.96501524e-02,  4.81491757e-02,  5.49824651e-02,
            6.02620950e-02,  6.14243624e-02,  5.62924901e-02,  5.99223953e-02,
            5.88024290e-02,  5.10965668e-02,  4.71608502e-02,  4.70679178e-02,
            4.50083991e-02,  3.52066736e-02,  3.45027323e-02,  3.76126482e-02,
            3.98106728e-02,  4.17907600e-02,  4.14108059e-02,  3.87614585e-02,
            3.90231900e-02,  3.59380235e-02,  4.21409303e-02,  4.30946817e-02,
            3.72829840e-02,  4.73446242e-02,  4.26550308e-02,  4.68945756e-02,
            4.42371508e-02,  4.57609022e-02,  4.44756660e-02,  4.13368311e-02,
            3.99672425e-02,  3.26930474e-02,  3.23024760e-02,  2.54963335e-02,
            2.50189314e-02,  2.02379017e-02,  2.58660394e-02,  3.05048096e-02,
            3.31676249e-02,  2.59619955e-02,  2.95656916e-02,  3.08033362e-02,
            3.16382043e-02,  3.34676238e-02,  3.51244431e-02,  2.56442287e-02,
            2.71432608e-02,  2.05498863e-02,  3.62141722e-02,  3.83317856e-02,
            4.56045029e-02,  4.12434569e-02,  3.97926988e-02,  3.92465792e-02,
            5.29927279e-02,  5.00398934e-02,  4.64206956e-02,  5.45713255e-02,
            5.45935445e-02,  5.71451498e-02,  5.34970863e-02,  6.23211780e-02,
            6.80013285e-02,  6.75189025e-02,  6.26220233e-02,  4.93082586e-02,
            5.15738648e-02,  4.60539212e-02,  4.81450548e-02,  5.13929035e-02,
            5.65960736e-02,  6.05927721e-02,  6.15187295e-02,  5.50729733e-02,
            5.33693285e-02,  4.90521284e-02,  4.46502460e-02,  3.72057626e-02,
            2.85665371e-02,  3.27167615e-02,  2.51918457e-02,  2.95491395e-02,
            2.87896763e-02,  2.19138592e-02,  2.38669888e-02,  2.64666286e-02,
            2.98261349e-02,  3.29320185e-02,  2.91146187e-02,  2.77604733e-02,
            3.30350739e-02,  3.35324608e-02,  2.12442365e-02,  1.90583170e-02,
            1.84418409e-02,  1.86373407e-02,  1.01362918e-02,  1.01819564e-02,
            1.32405626e-02,  1.26617937e-02,  1.71359721e-02,  1.02592612e-02,
            1.12055406e-02,  1.33424028e-02,  1.46747609e-02,  1.31298096e-02,
            2.16158368e-02,  2.29876750e-02,  1.48448095e-02,  1.49637090e-02,
            1.42602887e-02,  2.00699934e-02,  2.00303523e-02,  2.15046227e-02,
            2.11255519e-02,  2.26518867e-02,  3.17030694e-02,  3.05180866e-02,
            3.62586624e-02,  3.62203971e-02,  3.07594387e-02,  2.66816292e-02,
            2.69936177e-02,  2.17804612e-02,  1.66456886e-02,  4.21106605e-03,
            1.36903144e-02,  1.64473495e-02,  2.04085716e-02,  2.41247718e-02,
            1.75699319e-02,  1.40175705e-02,  1.62900442e-02,  1.56803580e-02,
            8.73407745e-03,  8.89023976e-04,  9.11682597e-04, -3.78095176e-04,
           -8.38293135e-03, -1.02076337e-02, -1.01641602e-02, -1.61898233e-02,
           -1.39586512e-02, -2.25867509e-02, -2.15890796e-02, -2.41540510e-02,
           -2.85038518e-02, -3.58531869e-02, -3.66330542e-02, -3.54087545e-02,
           -3.97417947e-02, -3.60075682e-02, -3.25693039e-02, -2.98726974e-02,
           -2.66584337e-02, -3.03353309e-02, -3.08860124e-02, -2.84036236e-02,
           -3.44546560e-02, -3.54682971e-02, -3.43460499e-02, -4.47123933e-02,
           -5.42745079e-02, -6.45852491e-02, -5.53228982e-02, -5.64939042e-02,
           -6.46190630e-02, -7.09132558e-02, -6.50941450e-02, -6.27417314e-02,
           -5.72725487e-02, -5.74787669e-02, -5.87379230e-02, -5.39582531e-02,
           -5.09610417e-02, -4.55755077e-02, -3.93118826e-02, -4.74532998e-02,
           -4.27886970e-02, -3.80978370e-02, -3.83093868e-02, -2.86602744e-02,
           -2.12128632e-02, -6.37578273e-03, -9.65356856e-03, -6.51101221e-03,
           -1.64325823e-03,  6.06168853e-03,  1.23488051e-02,  1.27069728e-02,
            6.54260051e-03,  1.03739930e-02,  2.64528918e-02,  3.21396925e-02,
            3.55102686e-02,  4.10650836e-02,  4.16740516e-02,  5.08357392e-02,
            5.03696867e-02,  5.64582904e-02,  6.53057110e-02,  6.20579856e-02,
            6.53211152e-02,  5.61374048e-02,  4.74689198e-02,  4.32712451e-02,
            4.58578842e-02,  5.39276515e-02,  5.68029077e-02,  4.46031252e-02,
            3.55977537e-02,  3.85361880e-02,  3.87238218e-02,  3.58076701e-02,
            3.70908207e-02,  3.95897496e-02,  4.93032427e-02,  5.15939432e-02,
            5.35817843e-02,  5.84167633e-02,  5.69925722e-02,  6.06353242e-02,
            6.32067221e-02,  6.41128106e-02,  5.91719271e-02,  5.94822018e-02,
            5.86804840e-02,  6.64976702e-02,  5.77872556e-02,  4.61321136e-02,
            4.66631919e-02,  4.29401817e-02,  4.65776968e-02,  4.56982679e-02,
            4.28165162e-02,  3.71533903e-02,  4.31985922e-02,  4.22724163e-02,
            4.51215082e-02,  5.37364725e-02,  4.57598056e-02,  4.36999118e-02,
            4.28701496e-02,  4.23173182e-02,  3.94637879e-02,  5.10288809e-02,
            4.92980289e-02,  4.99216024e-02,  5.44057132e-02,  5.05756049e-02,
            5.29092488e-02,  5.58887660e-02,  5.06992449e-02,  4.64303103e-02,
            5.33557359e-02,  5.13106806e-02,  4.75434730e-02,  4.35408534e-02,
            4.47158748e-02,  4.75918785e-02,  5.26717428e-02,  4.77642185e-02,
            4.97005585e-02,  4.03934174e-02,  3.95675008e-02,  3.34378236e-02,
            3.24378794e-02,  3.34082107e-02,  3.67162673e-02,  3.26225879e-02,
            3.67881270e-02,  3.51074449e-02,  3.24536096e-02,  3.50380885e-02]),
     'measures': array([[-0.089233, -0.360641, -0.339705, ...,  0.269883,  0.345565,
             0.329527],
           [ 0.196876,  0.138682,  0.310116, ..., -0.338361, -0.282151,
            -0.006422],
           [ 0.718212,  0.791117,  0.633963, ...,  1.127197,  0.733311,
             0.872542],
           ...,
           [-1.034737, -1.099109, -1.132291, ..., -1.050892, -1.231856,
            -1.126414],
           [ 0.580769,  0.783166,  0.882397, ...,  0.435125,  0.672602,
             0.540023],
           [ 0.78726 ,  0.930083,  0.972123, ...,  0.830087,  0.819682,
             0.888145]]),
     'sigma_sqr': 0.7587939371628688}
#### -50wh9o_h99xTbkJP4M4o5svWSs_cArc4y8o9g==.subsim:
    {'acceptance': 0.15353278199999998,
     'measures': array([[-0.415997, -0.377079, -0.374848, ..., -0.323446, -0.243707,
            -0.269935],
           [-0.305317, -0.271468, -0.283568, ..., -0.238135, -0.269383,
            -0.311422],
           [-0.147097, -0.07487 , -0.049379, ..., -0.134623, -0.034992,
            -0.075729],
           ...,
           [-0.607679, -0.513123, -0.480343, ..., -0.563664, -0.674404,
            -0.625446],
           [-0.509303, -0.421903, -0.556375, ..., -0.424511, -0.422368,
            -0.429252],
           [-0.181763, -0.272889, -0.418785, ..., -0.27001 , -0.182537,
            -0.113212]])}
#### .invocation_string:
    ('{seed} {d} {lambda} {eta} {N} {local_steps} {measuration_to_take} '
     '{measure_every} {step_size}')
#### 7bR6IA0kDxZCXG1izRXnFOb-pVbHfZ3u_O7m_w==.subsim:
    {'DeltaE': (0.08761886949447058, 0.001698569214946689),
     'acceptance': 0.27991298400000003,
     'correlations': array([0.40376584, 0.392637  , 0.38607765, 0.37781436, 0.37406802,
           0.36861574, 0.36198325, 0.35588607, 0.35002936, 0.3427028 ,
           0.3354452 , 0.33323392, 0.32964311, 0.32057502, 0.31467973,
           0.31188735, 0.30866694, 0.30568492, 0.29955595, 0.29601721,
           0.29140131, 0.28939494, 0.28629042, 0.27847615, 0.27305403,
           0.26796861, 0.26562544, 0.25972757, 0.25719445, 0.25676033,
           0.2526838 , 0.25226495, 0.25025683, 0.24676063, 0.24245882,
           0.24138951, 0.24139607, 0.24073373, 0.23708636, 0.23633138,
           0.23379066, 0.2329039 , 0.22947359, 0.23035516, 0.22489155,
           0.22570741, 0.22116498, 0.2166871 , 0.21634825, 0.21534325,
           0.21385377, 0.21263233, 0.21350719, 0.21332889, 0.21253643,
           0.21156496, 0.21178884, 0.21198576, 0.20771082, 0.20862352,
           0.20717112, 0.20716762, 0.20647458, 0.20449021, 0.20467661,
           0.20423995, 0.20291696, 0.20187363, 0.19941949, 0.19718885,
           0.19798555, 0.19573662, 0.1940633 , 0.19237824, 0.19212557,
           0.18378485, 0.183357  , 0.18395626, 0.17921126, 0.17537508,
           0.17450535, 0.17195888, 0.17144416, 0.17052963, 0.1689229 ,
           0.16733664, 0.16692279, 0.16408461, 0.16625093, 0.16336108,
           0.16462362, 0.16482888, 0.16336465, 0.16344861, 0.1604573 ,
           0.15818245, 0.15528344, 0.15659644, 0.15599964, 0.15461133,
           0.14706929, 0.15051537, 0.15260521, 0.15182461, 0.15216399,
           0.15142167, 0.1478613 , 0.14797305, 0.14671107, 0.14719643,
           0.14802462, 0.15146129, 0.15176611, 0.14677168, 0.15006915,
           0.15149697, 0.14880059, 0.15110179, 0.15095587, 0.14929298,
           0.14580991, 0.14670203, 0.14498518, 0.14170921, 0.14025783,
           0.14104623, 0.14080979, 0.14031547, 0.14023961, 0.1407039 ,
           0.14627245, 0.14733721, 0.14481605, 0.15051097, 0.14768959,
           0.14642411, 0.14260083, 0.14257487, 0.14283506, 0.14634077,
           0.14645965, 0.1471781 , 0.14743364, 0.14950974, 0.14908302,
           0.15216961, 0.15332283, 0.15392569, 0.1540053 , 0.15206509]),
     'measures': array([[ 2.147210e-01, -4.934400e-02, -5.960300e-02, ..., -2.421430e-01,
            -3.189800e-02, -6.990000e-04],
           [-3.025950e-01, -4.698980e-01, -4.882610e-01, ..., -1.618210e-01,
             8.988000e-03, -3.160080e-01],
           [-3.226610e-01, -2.737170e-01, -3.407160e-01, ..., -3.654800e-02,
            -2.781940e-01, -3.839550e-01],
           ...,
           [-1.003100e-01, -7.989300e-02, -1.660610e-01, ..., -2.834610e-01,
            -2.535840e-01, -1.955750e-01],
           [-9.171500e-01, -8.402200e-01, -6.918680e-01, ..., -1.039435e+00,
            -7.387040e-01, -7.463220e-01],
           [-1.338185e+00, -1.308330e+00, -1.344051e+00, ..., -1.368131e+00,
            -1.301576e+00, -1.376652e+00]]),
     'meff': (5.210323642032988, 0.10100672822336124),
     'sigma_sqr': 0.2511360207362902}
#### .index:
    {'-50wh9o_h99xTbkJP4M4o5svWSs_cArc4y8o9g==.subsim': {'setup': {...},
                                                         'simulator_name': 'i233F04XhpGVzRA4DpUioRxyGoxYKsILoBWJig==.simulator',
                                                         'status': 'done'},
     '-ltzAh3tG74yZFezyLNOHEXv9VrWDX2T1jLVNw==.subsim': {'setup': {...},
                                                         'simulator_name': 'i233F04XhpGVzRA4DpUioRxyGoxYKsILoBWJig==.simulator',
                                                         'status': 'done'},
     '1VvXWf5EqjZWDOROsRdDs1VgxMSqqQxac09VnQ==.subsim': {'setup': {...},
                                                         'simulator_name': 'i233F04XhpGVzRA4DpUioRxyGoxYKsILoBWJig==.simulator',
                                                         'status': 'done'},
     '22cULKqv4EaiHkejZ6HGZRFqZjxrXiYDzmoTag==.subsim': {'setup': {...},
                                                         'simulator_name': 'i233F04XhpGVzRA4DpUioRxyGoxYKsILoBWJig==.simulator',
                                                         'status': 'done'},
     '5y5OEYcdqJlIED2tALLyk0YoIsbI4iNo9ifaOg==.subsim': {'setup': {...},
                                                         'simulator_name': 'i233F04XhpGVzRA4DpUioRxyGoxYKsILoBWJig==.simulator',
                                                         'status': 'done'},
     '8hlZSnCtW3gt6X6uW1F2e4BvH5rdtpChmDu2cQ==.subsim': {'setup': {...},
                                                         'simulator_name': 'i233F04XhpGVzRA4DpUioRxyGoxYKsILoBWJig==.simulator',
                                                         'status': 'done'},
     'CFMyrSucNxzxRydEJ9K_1KLc9SDDFF44Ba7Cqg==.subsim': {'setup': {...},
                                                         'simulator_name': 'i233F04XhpGVzRA4DpUioRxyGoxYKsILoBWJig==.simulator',
                                                         'status': 'done'},
     'G9xHH7Jk58cGd76L-ui9vL97EhSre5LV_aFdlA==.subsim': {'setup': {...},
                                                         'simulator_name': 'i233F04XhpGVzRA4DpUioRxyGoxYKsILoBWJig==.simulator',
                                                         'status': 'done'},
     'KZIKr-eRoAxMYI9vVmXrcGR3DVs2q465c0Zs7g==.subsim': {'setup': {...},
                                                         'simulator_name': 'i233F04XhpGVzRA4DpUioRxyGoxYKsILoBWJig==.simulator',
                                                         'status': 'done'},
     'QBIscaV5XuE93mbzqbxKmgCE_3HfisIB1fM56A==.subsim': {'setup': {...},
                                                         'simulator_name': 'i233F04XhpGVzRA4DpUioRxyGoxYKsILoBWJig==.simulator',
                                                         'status': 'done'},
     'Qh7Eh5VHY8GsW0r5nkRCqyUNCQEkcsoGp6wk2A==.subsim': {'setup': {...},
                                                         'simulator_name': 'i233F04XhpGVzRA4DpUioRxyGoxYKsILoBWJig==.simulator',
                                                         'status': 'done'},
     'V334-FiAa-OCnaHmXYnYnAI4XCdus01cdmXODg==.subsim': {'setup': {...},
                                                         'simulator_name': 'i233F04XhpGVzRA4DpUioRxyGoxYKsILoBWJig==.simulator',
                                                         'status': 'done'},
     'Wmah2tqvsNdapX7RpQmG8dBM1gh7uurVxWzgyg==.subsim': {'setup': {...},
                                                         'simulator_name': 'i233F04XhpGVzRA4DpUioRxyGoxYKsILoBWJig==.simulator',
                                                         'status': 'done'},
     'Y0zMx8aQspmOQ1nRE20a4ZwIkUnKO-MowrQ2AQ==.subsim': {'setup': {...},
                                                         'simulator_name': 'i233F04XhpGVzRA4DpUioRxyGoxYKsILoBWJig==.simulator',
                                                         'status': 'done'},
     'YY_VYlCjiHRhwyGob0k5BN_D9kpvxxRuh0BuSw==.subsim': {'setup': {...},
                                                         'simulator_name': 'i233F04XhpGVzRA4DpUioRxyGoxYKsILoBWJig==.simulator',
                                                         'status': 'done'},
     'jDYQUct7_lbmHxrrJrRgRIx639DBSWC0tHjZwA==.subsim': {'setup': {...},
                                                         'simulator_name': 'i233F04XhpGVzRA4DpUioRxyGoxYKsILoBWJig==.simulator',
                                                         'status': 'done'},
     'jPMGJ7VoFyp41-D8z7562JkqFwzAxr35KnG5FQ==.subsim': {'setup': {...},
                                                         'simulator_name': 'i233F04XhpGVzRA4DpUioRxyGoxYKsILoBWJig==.simulator',
                                                         'status': 'done'},
     'mXg5Fb3IKR-adzZwMsP4xFrG0e4yMhR0tjcdqg==.subsim': {'setup': {...},
                                                         'simulator_name': 'i233F04XhpGVzRA4DpUioRxyGoxYKsILoBWJig==.simulator',
                                                         'status': 'done'},
     'nMDudP5twtBqL6zWsH8KkW7K03_oYGs_yAMxsA==.subsim': {'setup': {...},
                                                         'simulator_name': 'i233F04XhpGVzRA4DpUioRxyGoxYKsILoBWJig==.simulator',
                                                         'status': 'done'},
     'wuIxFYiwx9rWLPjp4AG1rHfGmi1ya2kaJxfJLQ==.subsim': {'setup': {...},
                                                         'simulator_name': 'i233F04XhpGVzRA4DpUioRxyGoxYKsILoBWJig==.simulator',
                                                         'status': 'done'}}
#### PIqoLBEm7sJp7LDc_3kwKuq71yJJw_qJh1iLQA==.subsim:
    {'acceptance': 0.27584946,
     'measures': array([[-0.208857, -0.159795, -0.10184 , ...,  0.618694,  0.294306,
             0.210916],
           [-0.181467, -0.063131, -0.258465, ..., -0.278711, -0.035145,
             0.012257],
           [-0.198296, -0.095019, -0.151697, ..., -0.025376, -0.197202,
            -0.294493],
           ...,
           [ 0.149117,  0.110212,  0.027448, ...,  0.277153,  0.061146,
             0.023805],
           [ 0.869987,  0.675786,  0.610476, ...,  0.831338,  0.901686,
             0.893484],
           [-0.312097, -0.210436, -0.159926, ...,  0.032609, -0.094117,
            -0.334771]])}
#### yNtf5wL6gOOxXHbGjMFFwIcVcojpmCWHakhtXw==.subsim:
    {'DeltaE': (0.1777721345513467, 0.001061362679825076),
     'acceptance': 0.350517744,
     'correlations': array([0.84849612, 0.83449189, 0.81378289, 0.80076028, 0.78915309,
           0.76991151, 0.75566158, 0.74913414, 0.73419159, 0.72485227,
           0.71655771, 0.70287078, 0.69666576, 0.67852552, 0.67094337,
           0.6563601 , 0.64435441, 0.63045796, 0.61608533, 0.60085794,
           0.58744868, 0.58036111, 0.57532583, 0.56392704, 0.55549697,
           0.55089211, 0.53591142, 0.52452852, 0.52064066, 0.50479702,
           0.50198939, 0.49066643, 0.49010352, 0.48236659, 0.46655641,
           0.45576756, 0.44803383, 0.43806499, 0.43019884, 0.42093535,
           0.4087235 , 0.4000132 , 0.39550845, 0.3850997 , 0.37529873,
           0.36435915, 0.36532195, 0.36450569, 0.35478564, 0.35222273,
           0.34510005, 0.33713536, 0.33228998, 0.32086748, 0.30805474,
           0.30413032, 0.29857175, 0.28675823, 0.28350614, 0.27154567,
           0.26464288, 0.26216252, 0.26159988, 0.26601355, 0.25915503,
           0.26214022, 0.25884327, 0.25761828, 0.25293194, 0.25465796,
           0.25239713, 0.24878893, 0.24029093, 0.23305433, 0.22859373,
           0.22159262, 0.21191645, 0.20150925, 0.20138537, 0.20317834,
           0.19509108, 0.1927266 , 0.18994504, 0.19188221, 0.19175699,
           0.18904556, 0.18382515, 0.18530327, 0.18251435, 0.17869403,
           0.17813275, 0.17939708, 0.17621141, 0.17210916, 0.16203192,
           0.15616446, 0.1576657 , 0.15340924, 0.14770736, 0.14932184,
           0.14796278, 0.14564412, 0.13825122, 0.13784996, 0.1327779 ,
           0.1323234 , 0.12624425, 0.1220566 , 0.11848325, 0.1152051 ,
           0.11202319, 0.10800873, 0.10590541, 0.10885461, 0.10060866,
           0.09480279, 0.08805574, 0.08568345, 0.08022224, 0.0779982 ,
           0.07644623, 0.07578762, 0.07586458, 0.07393098, 0.0721529 ,
           0.07640327, 0.07273118, 0.07258177, 0.07391615, 0.07682799,
           0.07585341, 0.07646801, 0.07150695, 0.06762098, 0.06145638,
           0.05893481, 0.06153237, 0.05943512, 0.06160289, 0.059536  ,
           0.05709201, 0.05881302, 0.05832791, 0.0610553 , 0.05198155,
           0.04958608, 0.053381  , 0.05578978, 0.05399362, 0.05201182]),
     'measures': array([[ 0.705681,  0.636919,  0.66207 , ...,  0.47554 ,  0.558379,
             0.687408],
           [-0.813477, -0.509616, -0.242496, ..., -0.139797, -0.613631,
            -0.710853],
           [-0.801673, -1.093638, -0.988512, ..., -1.035131, -1.016574,
            -0.702957],
           ...,
           [-1.297976, -1.096995, -0.998824, ..., -1.154694, -1.209699,
            -1.258642],
           [-0.30015 , -0.515609, -0.550151, ..., -0.107601, -0.070255,
            -0.178954],
           [-0.28964 , -0.539942, -1.07783 , ..., -0.326863, -0.455597,
            -0.250931]]),
     'meff': (1.2657053536710807, 0.007556709770243951),
     'sigma_sqr': 0.6287842696911184}
#### ukmuKajajPwIvw8gVF46jQL2xvhNpWzcTdMgIg==.subsim:
    {'acceptance': 0.29437627,
     'correlations': array([ 4.80596150e-01,  4.73012236e-01,  4.65950378e-01,  4.61441976e-01,
            4.51248300e-01,  4.47600074e-01,  4.41220559e-01,  4.34629119e-01,
            4.31279518e-01,  4.17379925e-01,  4.10567501e-01,  4.01542748e-01,
            3.88013192e-01,  3.82075324e-01,  3.82065881e-01,  3.75772473e-01,
            3.68141440e-01,  3.60410662e-01,  3.55969018e-01,  3.50881735e-01,
            3.47142054e-01,  3.36651733e-01,  3.36380434e-01,  3.31966827e-01,
            3.26175153e-01,  3.22908021e-01,  3.22952575e-01,  3.15657498e-01,
            3.06314940e-01,  3.02239338e-01,  2.93120780e-01,  2.95211030e-01,
            2.92096724e-01,  2.87236170e-01,  2.76297216e-01,  2.69194884e-01,
            2.56873989e-01,  2.53820463e-01,  2.55319771e-01,  2.57461758e-01,
            2.47408870e-01,  2.47652462e-01,  2.39332967e-01,  2.38895139e-01,
            2.31482136e-01,  2.28253360e-01,  2.21687074e-01,  2.18722779e-01,
            2.19016269e-01,  2.17082015e-01,  2.17815494e-01,  2.12714848e-01,
            2.11308622e-01,  2.06340062e-01,  2.01590338e-01,  2.00249823e-01,
            1.93956973e-01,  1.88301193e-01,  1.85911916e-01,  1.81352365e-01,
            1.85595989e-01,  1.80589948e-01,  1.75002941e-01,  1.73617199e-01,
            1.77146064e-01,  1.68437711e-01,  1.67377663e-01,  1.68692953e-01,
            1.64386071e-01,  1.63317530e-01,  1.57162645e-01,  1.59269006e-01,
            1.56086280e-01,  1.53900460e-01,  1.50230331e-01,  1.52118723e-01,
            1.48519512e-01,  1.49029484e-01,  1.42736526e-01,  1.42483186e-01,
            1.36363163e-01,  1.30454147e-01,  1.29689970e-01,  1.30150855e-01,
            1.33711636e-01,  1.26137411e-01,  1.22297197e-01,  1.27125275e-01,
            1.23326348e-01,  1.23224996e-01,  1.22279877e-01,  1.17952614e-01,
            1.19564004e-01,  1.26901050e-01,  1.20710340e-01,  1.21200514e-01,
            1.23440982e-01,  1.17069614e-01,  1.15059393e-01,  1.15843122e-01,
            1.14197507e-01,  1.07206186e-01,  1.13764327e-01,  1.10062983e-01,
            1.08329741e-01,  1.02511284e-01,  1.00734107e-01,  9.74839780e-02,
            9.70350522e-02,  9.33864386e-02,  8.86192767e-02,  8.76670262e-02,
            8.72288065e-02,  8.98946956e-02,  8.62122516e-02,  8.65988250e-02,
            8.18863539e-02,  8.64600254e-02,  7.91350206e-02,  7.00472456e-02,
            7.69139558e-02,  7.90183387e-02,  7.64355622e-02,  7.55657483e-02,
            7.30933149e-02,  6.95051313e-02,  6.83887658e-02,  7.35157387e-02,
            6.77894092e-02,  6.58021004e-02,  6.85952693e-02,  6.93852280e-02,
            6.87146142e-02,  6.34559847e-02,  5.60884814e-02,  5.69115807e-02,
            5.16546071e-02,  5.24635272e-02,  5.19028543e-02,  5.17912803e-02,
            5.20596587e-02,  4.72090945e-02,  5.10177162e-02,  4.95524470e-02,
            4.54133619e-02,  4.28237655e-02,  4.27157326e-02,  4.78394976e-02,
            4.86271892e-02,  4.63141558e-02,  4.36381549e-02,  3.43630583e-02,
            3.69154444e-02,  3.47352627e-02,  3.56469780e-02,  3.49562283e-02,
            3.38866182e-02,  4.19417746e-02,  4.10327029e-02,  4.14916879e-02,
            4.03082280e-02,  4.08375293e-02,  4.00293459e-02,  4.66073424e-02,
            4.24856343e-02,  4.33117516e-02,  4.31537859e-02,  4.23305487e-02,
            4.16830285e-02,  4.04806996e-02,  3.97241292e-02,  3.52205661e-02,
            3.02788891e-02,  2.58155998e-02,  2.48463691e-02,  2.37998953e-02,
            2.52705461e-02,  2.63944851e-02,  2.74160237e-02,  2.80648078e-02,
            3.13531776e-02,  2.68202881e-02,  2.75222766e-02,  3.23169137e-02,
            2.89912930e-02,  2.80947681e-02,  2.01600898e-02,  1.84059904e-02,
            1.59417978e-02,  2.12325536e-02,  1.78922596e-02,  1.68446581e-02,
            1.01151628e-02,  1.27636224e-02,  1.43390266e-02,  1.29080359e-02,
            1.51372595e-02,  1.49032013e-02,  1.13491803e-02,  4.60609754e-03,
            3.45429821e-03, -4.70059097e-03, -7.01038064e-03, -8.31691534e-03,
           -2.95835985e-03,  4.08526717e-03,  5.34051009e-03,  5.41674353e-03,
            6.08478382e-03,  3.31909248e-03,  2.62686988e-03,  4.08523243e-03,
            5.04723814e-03,  5.45745629e-03,  4.71115243e-03,  4.28629474e-03,
           -4.65005401e-04,  7.13117802e-04,  2.21990765e-03,  1.51072429e-03,
            7.08750762e-04, -5.13828949e-03, -6.60878265e-03, -3.32966722e-03,
           -2.21157760e-03, -2.18979796e-03, -5.17000466e-03, -4.70229950e-03,
           -4.10744205e-03, -2.18476338e-03,  1.79512162e-03,  4.46746634e-03,
            5.58480944e-03,  3.10122889e-03,  3.44677576e-03,  9.41128117e-04,
            2.33053864e-03,  5.11266100e-03,  8.34207142e-03,  3.43491428e-03,
            7.86445509e-03,  8.79302847e-03,  7.16080136e-03,  8.62082098e-03,
            1.14219564e-02,  6.91798914e-03, -1.16975285e-03, -9.69099810e-03,
           -1.16671829e-02, -1.20536701e-02, -1.51451680e-02, -8.13550171e-03,
           -1.69853690e-02, -1.12596840e-02, -8.24829093e-03, -9.46728646e-03,
           -5.58530043e-03, -4.02588003e-03,  7.18266540e-03,  7.45578269e-03,
            9.28398560e-03,  7.62647467e-03,  6.53015835e-03,  9.29541154e-03,
            7.49068087e-03,  8.25861975e-03,  1.32076786e-02,  6.40946203e-03,
            5.07766760e-03,  5.83109699e-03,  7.26813026e-03,  6.41494793e-03,
            8.65134199e-03,  2.54445905e-03,  6.30465752e-03,  7.31153435e-03,
            7.01238158e-03,  9.69662139e-03,  7.42033732e-03,  5.52094624e-03,
            3.80003838e-03,  1.13473670e-02,  7.48574203e-03,  5.54996586e-03,
            5.57651270e-04, -3.55774054e-03, -3.40412907e-03, -2.86592452e-03,
           -1.53868313e-03, -1.02950024e-02, -3.36851529e-03, -7.53536325e-03,
           -2.60865332e-03, -3.14412635e-04, -2.55193808e-03,  3.21988446e-03,
            4.10984590e-03,  1.87053610e-03,  3.23730478e-03,  6.90333263e-04,
            7.53600837e-05, -2.51433479e-03,  5.64074693e-03,  5.68925032e-03,
            7.90888382e-03,  5.63443082e-03,  8.68787630e-03,  1.20402000e-02,
            1.05959512e-02,  9.01531428e-03,  4.76438294e-03,  3.69985413e-03,
            3.73347608e-03,  1.14238031e-02,  1.27445973e-02,  6.99355483e-03,
            9.12686081e-03,  8.97600731e-03,  1.71805379e-03, -1.70603483e-03,
           -3.88237791e-04,  1.45076729e-03,  4.56306800e-03,  9.63852281e-03,
            6.57911636e-03,  1.65369011e-02,  1.10902560e-02,  4.04888380e-03,
            3.59482036e-03,  1.15572411e-03, -3.18980639e-03, -6.22606683e-04,
            2.28566269e-03, -4.11821626e-03, -4.49609292e-03, -3.88082933e-03,
           -1.53296821e-03, -1.26423736e-03, -7.11633754e-03, -4.73542362e-03,
           -4.66986408e-03, -5.37885854e-04, -3.17284197e-03, -5.12664625e-03,
           -4.22620762e-03, -3.95333229e-03,  2.01367833e-03,  5.77982214e-03,
            3.52883014e-03,  8.23735983e-03,  6.91917851e-03,  1.38513466e-03,
            6.89210000e-03,  1.31685109e-02,  1.44047105e-02,  1.41305788e-02,
            1.15438352e-02,  1.15085512e-02,  1.63730241e-02,  1.57604171e-02,
            1.64749201e-02,  1.97686126e-02,  2.43687200e-02,  2.93098534e-02,
            2.81372088e-02,  2.75197516e-02,  2.30793808e-02,  2.50352384e-02,
            2.23907455e-02,  2.10726903e-02,  1.89012489e-02,  2.04697703e-02,
            1.89186121e-02,  1.59140538e-02,  1.92379699e-02,  2.22326871e-02,
            2.35594606e-02,  2.61591381e-02,  2.75013549e-02,  2.79156027e-02,
            2.48128117e-02,  2.40552186e-02,  2.38648324e-02,  2.03714159e-02,
            2.28310523e-02,  2.37201615e-02,  2.32362229e-02,  2.62494505e-02,
            1.85461813e-02,  1.82108014e-02,  1.85338670e-02,  1.08676232e-02,
            1.14333263e-02,  1.25901022e-02,  1.15076986e-02,  1.20318176e-02,
            1.26372912e-02,  1.55875463e-02,  1.13033951e-02,  1.23466606e-02,
            6.78984846e-03,  1.26954444e-02,  9.07308226e-03,  7.63255246e-03,
            4.73092195e-03,  9.44080965e-03,  7.17494731e-03,  5.61361913e-03,
            1.82158618e-03,  5.30535080e-03,  5.57891339e-03,  7.67706690e-03,
            1.43295510e-02,  9.16913499e-03,  1.18795827e-02,  8.10573645e-03,
            3.74271161e-03,  6.89797563e-03,  1.12144592e-02,  3.77963808e-03,
            2.42144669e-03, -1.02492227e-03, -2.40948954e-04, -4.20827317e-04,
            5.23481487e-03,  1.65988002e-03,  2.13709175e-04,  1.71899722e-03,
            1.04989060e-02,  1.03761295e-02,  9.10259665e-03,  6.70433568e-03,
            8.29768050e-03,  6.30990293e-03,  7.56327742e-03,  9.21929091e-03,
            1.57618442e-02,  1.50768464e-02,  1.43378010e-02,  1.02344112e-02,
            4.14004835e-03,  5.49050750e-03,  2.90704589e-03,  3.87278355e-03,
            6.94579316e-03,  1.35289021e-02,  1.10171051e-02,  1.09757868e-02,
            9.64531181e-03,  1.31567914e-02,  1.59826271e-02,  2.12245570e-02,
            2.00892032e-02,  2.12628170e-02,  2.51869511e-02,  2.67345894e-02,
            2.47013550e-02,  2.24624193e-02,  2.62490629e-02,  2.06303575e-02,
            2.44741422e-02,  2.63949765e-02,  3.33636220e-02,  3.31542267e-02,
            3.08984806e-02,  2.98805722e-02,  2.47990554e-02,  2.68118974e-02,
            2.83101404e-02,  2.71840693e-02,  2.72002404e-02,  3.29380363e-02,
            3.62454982e-02,  3.16430272e-02,  2.63239219e-02,  2.41097426e-02,
            2.48969817e-02,  2.63654015e-02,  2.44320168e-02,  1.69801999e-02,
            1.40320349e-02,  7.81124427e-03,  3.20769916e-03,  2.62882595e-03,
            4.94696308e-03, -1.33108309e-03, -1.44828684e-03, -2.42640452e-04,
           -2.66237794e-03, -3.27779824e-03,  4.49139917e-03,  3.83046534e-03,
            1.25571982e-03,  5.28278494e-03,  1.11508391e-02,  1.26650976e-02,
            1.56238176e-02,  1.41676831e-02,  1.47860089e-02,  1.08227825e-02]),
     'measures': array([[-0.222442, -0.051456, -0.108686, ..., -0.056195,  0.065263,
            -0.028655],
           [-0.207033, -0.153378, -0.210511, ..., -0.002146, -0.118769,
            -0.17295 ],
           [-0.548055, -0.420491, -0.199013, ..., -0.742523, -0.573499,
            -0.60002 ],
           ...,
           [-1.377606, -1.42833 , -1.15672 , ..., -1.133548, -1.369074,
            -1.346325],
           [-0.91737 , -1.169143, -1.152578, ..., -0.565861, -0.917746,
            -0.737662],
           [-1.273926, -1.02337 , -1.057932, ..., -1.336211, -1.376011,
            -1.383717]]),
     'sigma_sqr': 0.44428272217783304}
#### 9pMjRgA8y-87gbFykJ0rwjkWkk7szGd2tNFrBQ==.subsim:
    {'acceptance': 0.32081786,
     'measures': array([[-0.340138, -0.384115, -0.317044, ..., -0.14355 , -0.209031,
            -0.158939],
           [ 0.286128,  0.348373,  0.411964, ..., -0.137762, -0.002084,
             0.32097 ],
           [-0.565614, -0.523935, -0.878549, ..., -0.136097, -0.285462,
            -0.387761],
           ...,
           [ 0.242801,  0.138976,  0.034897, ...,  0.527506,  0.459542,
             0.365569],
           [ 1.378694,  1.498069,  1.500992, ...,  0.910719,  1.304667,
             1.483195],
           [ 1.524523,  1.578224,  1.74756 , ...,  1.922262,  1.870168,
             1.640428]])}
#### 1iZULiamAzuRK2avjcn8IpYC797h7d1YceCnSg==.subsim:
    {'acceptance': 0.339254664,
     'correlations': array([ 7.63846680e-01,  7.41869049e-01,  7.27049981e-01,  6.99769214e-01,
            6.85313816e-01,  6.68208200e-01,  6.64611520e-01,  6.63309448e-01,
            6.58334665e-01,  6.47309721e-01,  6.30510178e-01,  6.27261622e-01,
            6.18659730e-01,  6.08550937e-01,  6.02947577e-01,  5.92764177e-01,
            5.81250142e-01,  5.66424380e-01,  5.59366529e-01,  5.47357543e-01,
            5.27397959e-01,  5.13792705e-01,  5.03419663e-01,  4.94094353e-01,
            4.84763844e-01,  4.78370688e-01,  4.68069856e-01,  4.49664125e-01,
            4.51077261e-01,  4.43174038e-01,  4.32072074e-01,  4.24782638e-01,
            4.13567096e-01,  4.07219145e-01,  4.04179708e-01,  3.91508087e-01,
            3.87840033e-01,  3.82041359e-01,  3.82052645e-01,  3.73203770e-01,
            3.65331741e-01,  3.50508955e-01,  3.54353875e-01,  3.46581536e-01,
            3.32658380e-01,  3.23796016e-01,  3.15379942e-01,  3.10229276e-01,
            2.99186372e-01,  2.87166067e-01,  2.83065949e-01,  2.80624844e-01,
            2.83538596e-01,  2.79905536e-01,  2.73337522e-01,  2.65372365e-01,
            2.59467356e-01,  2.62855352e-01,  2.55477763e-01,  2.58243430e-01,
            2.51628111e-01,  2.52769899e-01,  2.41321304e-01,  2.45540034e-01,
            2.28364870e-01,  2.31174218e-01,  2.24874943e-01,  2.24519410e-01,
            2.13354762e-01,  2.12095791e-01,  2.07967556e-01,  2.04373491e-01,
            1.91008372e-01,  1.88582023e-01,  1.92806703e-01,  1.82058155e-01,
            1.76173640e-01,  1.68272601e-01,  1.65233294e-01,  1.72549910e-01,
            1.67985295e-01,  1.58288276e-01,  1.53066913e-01,  1.50081020e-01,
            1.50377891e-01,  1.55469148e-01,  1.52114610e-01,  1.49462781e-01,
            1.51316926e-01,  1.41490578e-01,  1.36498196e-01,  1.36613021e-01,
            1.32884411e-01,  1.39170511e-01,  1.29175195e-01,  1.24164683e-01,
            1.24374059e-01,  1.23686298e-01,  1.11129445e-01,  1.11416413e-01,
            1.14321251e-01,  1.09634425e-01,  1.11125677e-01,  1.11477769e-01,
            1.11833116e-01,  1.15483523e-01,  1.14239229e-01,  1.05752449e-01,
            1.00019530e-01,  1.02875620e-01,  1.08677329e-01,  1.12668726e-01,
            1.12297475e-01,  1.11444106e-01,  1.05787745e-01,  1.10323275e-01,
            1.05357794e-01,  9.68712880e-02,  1.03059629e-01,  1.10847688e-01,
            1.10783305e-01,  1.05143194e-01,  9.66054049e-02,  9.72781164e-02,
            1.00691228e-01,  1.06130199e-01,  1.03656654e-01,  9.75679417e-02,
            8.39995071e-02,  8.51869580e-02,  8.84150320e-02,  8.47346666e-02,
            8.59756147e-02,  9.38741115e-02,  8.87088412e-02,  8.91309329e-02,
            8.37979793e-02,  9.43081431e-02,  9.12085329e-02,  9.89033166e-02,
            8.92226293e-02,  8.73006417e-02,  8.71406216e-02,  8.20276231e-02,
            6.70712577e-02,  6.26714995e-02,  6.03564457e-02,  6.22323382e-02,
            5.54756925e-02,  5.99059858e-02,  5.45567511e-02,  5.00112232e-02,
            4.77516726e-02,  4.66473262e-02,  5.23362976e-02,  5.01977803e-02,
            5.21583803e-02,  4.83033309e-02,  5.00777952e-02,  4.73046153e-02,
            4.77675111e-02,  4.77035360e-02,  4.74452385e-02,  4.98489418e-02,
            4.93025903e-02,  4.14806328e-02,  3.82824004e-02,  4.00374172e-02,
            3.33389774e-02,  4.32839995e-02,  4.59910560e-02,  5.50668222e-02,
            6.02232831e-02,  6.17558611e-02,  7.10155863e-02,  7.14990769e-02,
            6.53957369e-02,  6.62573722e-02,  7.11712492e-02,  6.88185219e-02,
            6.70608792e-02,  5.64931256e-02,  5.99831236e-02,  5.73526480e-02,
            6.03279078e-02,  6.12243710e-02,  5.29427851e-02,  4.94276209e-02,
            4.10833804e-02,  4.63674504e-02,  4.32878861e-02,  4.44751298e-02,
            3.37703617e-02,  4.50498414e-02,  4.02819471e-02,  4.73808560e-02,
            4.88791580e-02,  4.67925568e-02,  4.96037034e-02,  4.66303237e-02,
            4.09145273e-02,  3.98587721e-02,  4.75124758e-02,  4.63066924e-02,
            4.26592638e-02,  3.91334421e-02,  4.30270470e-02,  3.13101363e-02,
            2.90242468e-02,  3.19412908e-02,  2.78737515e-02,  2.92836610e-02,
            2.42468147e-02,  2.38290143e-02,  2.98873613e-02,  3.21367280e-02,
            3.20302764e-02,  2.84896124e-02,  3.31817506e-02,  2.84481420e-02,
            2.77461048e-02,  2.14184446e-02,  2.54085582e-02,  2.58124087e-02,
            2.24425762e-02,  1.65017675e-02,  1.19347923e-02,  1.44101114e-02,
            1.34517255e-02,  6.25792133e-03,  5.54053264e-03, -5.90168851e-03,
           -4.46903473e-03, -2.83755743e-03, -4.43958747e-04,  1.00757342e-04,
           -1.51810376e-04, -7.18783809e-03, -7.26833829e-03, -6.90721186e-03,
           -1.44333957e-03, -5.58992127e-03, -2.94746738e-03, -1.06416861e-02,
           -1.39417100e-02, -4.04119344e-03, -2.66619230e-03, -7.27961447e-03,
           -2.35236201e-03, -2.20774293e-03, -4.93634474e-03, -2.09770742e-03,
           -3.00997943e-03,  1.22748226e-03, -4.07570282e-04,  7.10073396e-03,
            2.56313850e-03, -3.00342373e-03, -4.71375745e-03, -3.72976733e-03,
            1.17288540e-04,  3.03005798e-03,  5.69861834e-03,  6.76889766e-03,
            1.96728658e-03, -1.31346164e-03,  1.17560368e-03,  4.14341017e-03,
           -1.43670197e-03, -6.69490970e-03, -1.74066656e-03, -7.02877880e-03,
           -4.78562971e-03,  4.63025381e-03,  1.14454217e-02,  1.68720493e-03,
           -5.00369914e-04,  1.74189164e-03, -4.17223027e-03, -1.11723944e-03,
           -2.72556231e-03, -1.06459126e-02, -1.14122130e-02, -1.28029478e-02,
           -1.56446287e-02, -1.70918561e-02, -1.99433356e-02, -2.26539849e-02,
           -2.58744629e-02, -2.52249351e-02, -2.16993648e-02, -1.75261404e-02,
           -1.26751808e-02, -1.79527278e-02, -1.57187452e-02, -1.06226756e-02,
           -7.26195897e-03, -3.24651938e-03, -1.56361943e-02, -9.83176725e-03,
           -1.54582441e-02, -2.01138663e-02, -2.48975147e-02, -2.08412087e-02,
           -2.78789422e-02, -2.14285948e-02, -2.50519758e-02, -1.14413708e-02,
           -1.44033607e-02, -1.99577271e-02, -2.23220380e-02, -4.79792690e-03,
            1.74990877e-03, -3.56851792e-03, -5.69306376e-03, -9.90892230e-03,
           -1.33094287e-02, -1.39088027e-02, -1.56692459e-02, -1.53221582e-02,
           -1.79743655e-02, -1.90205834e-02, -1.04405408e-02, -1.53956386e-02,
           -9.82631712e-03, -1.11562853e-02, -1.61606611e-02, -1.04284898e-02,
           -1.61255725e-02, -2.15015840e-02, -7.14653420e-03, -6.10357114e-03,
           -6.47679862e-04, -4.71491918e-03, -5.16747820e-03, -7.94536842e-03,
           -1.42680139e-02, -1.34205377e-02, -2.10806660e-02, -2.22254224e-02,
           -1.95428737e-02, -1.99556224e-02, -1.99425557e-02, -1.74708474e-02,
           -1.34011239e-02, -1.07388989e-02, -8.49627746e-03, -1.07325968e-02,
           -1.16444330e-02, -6.89689873e-03, -2.56508481e-03, -6.63028524e-03,
           -1.07019445e-02, -1.34221096e-02, -1.59612840e-02, -1.31591199e-02,
           -5.05244473e-03, -2.03578039e-03, -1.11484550e-03, -8.54646470e-05,
            6.59365574e-05,  6.23805830e-03,  7.26135020e-03,  1.86723613e-03,
           -2.58875909e-03, -1.02238724e-02, -1.18763886e-02, -1.51108218e-02,
           -7.94222859e-03, -1.57863120e-02, -1.03365125e-02, -2.72671906e-03,
           -3.30315642e-03, -8.22863307e-03, -7.82195081e-03, -1.11065097e-02,
           -1.88572434e-02, -1.29178911e-02, -1.08922108e-02, -1.34095491e-02,
           -1.07959936e-02, -6.26042585e-03, -3.00489732e-03, -8.21335902e-04,
           -5.29099796e-03, -3.87950339e-03, -6.57643356e-03, -7.37937570e-03,
           -6.25626708e-03, -1.30283972e-03, -1.09705639e-02, -1.09462725e-02,
           -7.57462842e-03, -7.39562047e-03, -1.24980873e-02, -1.05345803e-02,
           -1.35226108e-02, -4.10066020e-03, -6.29418273e-03, -7.15707387e-03,
           -1.05933160e-02, -9.52006342e-03, -1.04926841e-02, -1.06278934e-02,
           -2.62282312e-03,  6.53491864e-04,  9.65540245e-06, -2.12898622e-03,
           -1.54552659e-02, -9.40742067e-03, -1.06208197e-03,  3.95959158e-03,
            3.27372817e-03,  5.01067407e-03, -7.81439676e-03, -1.08537294e-02,
           -6.30006387e-03, -2.34933736e-03,  2.09906600e-04, -4.20974930e-03,
            5.94251865e-03,  9.74632003e-03,  6.61818532e-03,  9.88572007e-04,
            5.81089150e-03,  3.58384830e-03,  1.35321345e-03, -3.28749886e-04,
            4.69032412e-03,  6.44412612e-03,  4.36173643e-03, -1.68492425e-04,
           -1.03484476e-03,  1.05989765e-02,  1.31302437e-02,  1.38130122e-02,
            9.67227006e-03,  9.26075124e-03,  5.27771366e-03,  4.70343926e-03,
           -1.53835357e-04,  8.08550310e-03,  5.56779930e-03,  6.04260701e-03,
            1.52123233e-03, -8.88033243e-03, -1.06688967e-02, -1.01240130e-02,
           -1.16450562e-02, -1.00680490e-02, -1.36166032e-02, -1.34563685e-02,
           -1.09494842e-02, -6.86891831e-03, -9.71432730e-03, -9.48660356e-03,
           -1.84059540e-02, -1.82163138e-02, -2.85245871e-02, -3.63365535e-02,
           -3.93399079e-02, -3.69300469e-02, -3.76766353e-02, -4.23187941e-02,
           -4.12672939e-02, -4.21701892e-02, -4.71697775e-02, -4.26151620e-02,
           -4.37216455e-02, -4.27289725e-02, -5.41217962e-02, -5.29485985e-02,
           -4.75781903e-02, -5.04515327e-02, -5.27286566e-02, -5.40262693e-02,
           -5.49968806e-02, -5.34251690e-02, -4.87161866e-02, -5.32166847e-02,
           -4.99111578e-02, -4.35597133e-02, -5.37079007e-02, -5.81758769e-02,
           -5.52867984e-02, -4.46294242e-02, -3.62129874e-02, -3.88387088e-02,
           -3.70661937e-02, -3.63413342e-02, -3.51782157e-02, -3.52990874e-02,
           -2.83116819e-02, -1.84166397e-02, -2.10532374e-02, -1.84884404e-02,
           -1.11560599e-02, -1.47632795e-02, -8.61997579e-03, -1.08233644e-02]),
     'measures': array([[ 0.236537,  0.404546,  0.55923 , ..., -0.044273, -0.115127,
            -0.152643],
           [ 0.108356,  0.173025, -0.134687, ...,  0.202142,  0.080167,
             0.049751],
           [ 0.014807,  0.030888, -0.372462, ...,  0.052067,  0.092297,
             0.144649],
           ...,
           [-0.108196, -0.375444, -0.450202, ..., -0.108541, -0.149196,
            -0.322157],
           [-0.363725, -0.380598, -0.514219, ..., -0.125713, -0.342601,
            -0.162855],
           [ 0.557994,  0.188432,  0.368434, ...,  0.402961,  0.605574,
             0.674684]]),
     'sigma_sqr': 0.7111287140594359}
#### 0vFR0J6zY_bFTdFUxRVpKtZVSHgawxpftkvQqA==.subsim:
    {'acceptance': 0.28437469,
     'correlations': array([ 4.55397266e-01,  4.50173076e-01,  4.39308806e-01,  4.28116597e-01,
            4.18026789e-01,  4.10872399e-01,  4.00764030e-01,  3.98853648e-01,
            3.91102277e-01,  3.79104857e-01,  3.70832743e-01,  3.57022766e-01,
            3.53898678e-01,  3.53770316e-01,  3.41761280e-01,  3.38242202e-01,
            3.35352567e-01,  3.27188131e-01,  3.19503075e-01,  3.17319826e-01,
            3.17332731e-01,  3.13328424e-01,  3.06214427e-01,  2.99665471e-01,
            2.89061156e-01,  2.83802655e-01,  2.76855411e-01,  2.72917720e-01,
            2.73209980e-01,  2.67466432e-01,  2.67791138e-01,  2.62177828e-01,
            2.59018073e-01,  2.58227586e-01,  2.54594419e-01,  2.54742097e-01,
            2.50478064e-01,  2.49863188e-01,  2.47968855e-01,  2.42622899e-01,
            2.33558989e-01,  2.25848277e-01,  2.27281576e-01,  2.24282856e-01,
            2.20183103e-01,  2.14215632e-01,  2.11206854e-01,  2.07552530e-01,
            2.02703639e-01,  1.97162322e-01,  1.97337921e-01,  1.95469981e-01,
            1.96803761e-01,  1.91103225e-01,  1.88175898e-01,  1.87566400e-01,
            1.79184091e-01,  1.76397846e-01,  1.67828774e-01,  1.66953046e-01,
            1.62584682e-01,  1.62349912e-01,  1.55308073e-01,  1.51985058e-01,
            1.46322990e-01,  1.42323355e-01,  1.42131551e-01,  1.36164673e-01,
            1.32677834e-01,  1.32049159e-01,  1.25935651e-01,  1.24097631e-01,
            1.27579125e-01,  1.27354949e-01,  1.26743420e-01,  1.21626848e-01,
            1.22654815e-01,  1.22674146e-01,  1.16132221e-01,  1.14386834e-01,
            1.12052825e-01,  1.08437746e-01,  1.08338973e-01,  1.04141739e-01,
            1.01297070e-01,  1.04245981e-01,  1.03972896e-01,  1.03772041e-01,
            1.04064555e-01,  1.05951430e-01,  1.02729657e-01,  1.02116248e-01,
            1.01217930e-01,  1.01515831e-01,  9.93154345e-02,  9.98537713e-02,
            1.02809762e-01,  1.02876056e-01,  1.05967486e-01,  1.05220212e-01,
            1.01871386e-01,  9.42446450e-02,  9.48562261e-02,  9.70846741e-02,
            9.20645306e-02,  8.88569782e-02,  8.82337457e-02,  8.85705414e-02,
            8.75096412e-02,  8.63606654e-02,  8.46025970e-02,  8.35697627e-02,
            7.87586214e-02,  7.82776961e-02,  7.93850084e-02,  7.25995380e-02,
            7.23236591e-02,  7.58937295e-02,  7.39655164e-02,  7.00178181e-02,
            6.35023350e-02,  6.62477739e-02,  6.79263564e-02,  6.76240843e-02,
            6.51472925e-02,  6.59052131e-02,  6.26616974e-02,  6.20031917e-02,
            6.39797641e-02,  6.35420715e-02,  6.40854418e-02,  6.41079824e-02,
            5.94702337e-02,  5.82884698e-02,  5.35858833e-02,  5.93642190e-02,
            5.97193544e-02,  5.95156253e-02,  5.44516371e-02,  5.45659133e-02,
            5.31118966e-02,  5.12830113e-02,  5.05563225e-02,  4.84992531e-02,
            5.31483150e-02,  5.30704946e-02,  4.77983885e-02,  4.59108205e-02,
            4.88913158e-02,  4.67737604e-02,  4.61741646e-02,  3.72142049e-02,
            4.37125821e-02,  4.25664775e-02,  4.36428799e-02,  4.63847049e-02,
            4.48344262e-02,  4.13437684e-02,  3.61294945e-02,  3.39688312e-02,
            3.16739853e-02,  2.53527816e-02,  2.72872534e-02,  2.75104048e-02,
            2.06355735e-02,  1.94857447e-02,  2.08938220e-02,  2.28330698e-02,
            2.37402456e-02,  2.27298553e-02,  2.32903258e-02,  2.15432266e-02,
            2.01173919e-02,  2.01620716e-02,  2.28436983e-02,  2.62663284e-02,
            2.56180462e-02,  2.40263090e-02,  2.73846829e-02,  3.27601705e-02,
            3.56964260e-02,  3.49235991e-02,  2.81833293e-02,  2.40189353e-02,
            2.42461071e-02,  1.44946354e-02,  9.14927867e-03,  1.45763488e-02,
            1.63430172e-02,  1.58698977e-02,  1.48137951e-02,  7.89664211e-03,
            5.28462077e-03,  9.15116271e-03,  3.32026396e-03,  2.93562934e-03,
            2.94903136e-03,  3.04653419e-03,  3.34848794e-03, -6.87599204e-04,
            2.31233043e-03,  1.69246788e-03,  8.60897560e-04, -3.33962015e-03,
           -2.08420907e-04,  2.38590635e-03,  1.65956825e-03,  1.12187086e-03,
           -3.30227041e-04,  2.35025252e-03, -1.07591178e-03, -2.06556489e-03,
           -7.09175251e-04,  4.52147051e-04, -5.67537995e-03,  1.41741436e-03,
            2.71069301e-03,  1.90558285e-03,  1.75254428e-03,  4.18177006e-03,
           -1.35533432e-03,  1.70266198e-03,  5.15204629e-03,  2.54666699e-03,
           -3.24871549e-05,  6.31795045e-03,  8.80306000e-04,  4.79608967e-03,
            5.24304491e-03,  5.96347914e-03,  2.89272224e-03,  3.19533806e-03,
            4.92016471e-03,  4.54791720e-03,  9.35029292e-03,  1.22651278e-02,
            9.16763137e-03,  7.13143977e-04, -4.80619824e-04,  1.10255129e-03,
            6.90862663e-03,  6.49056295e-03,  7.54037261e-03,  7.40143806e-03,
            5.93719334e-03,  4.83518981e-03,  3.57572488e-03,  7.07243845e-03,
           -4.98181262e-05,  1.47544619e-03,  2.95666635e-04,  1.18142663e-04,
            5.89815234e-03,  1.46751332e-03,  7.01351997e-03,  4.55489044e-03,
            3.57953088e-03, -5.81858533e-04,  1.99735165e-03,  2.54769811e-04,
           -4.67208793e-03, -3.63475056e-03, -4.99385359e-03, -7.97188953e-03,
           -3.22717362e-03, -1.65916537e-03, -8.82252010e-04, -5.71061933e-03,
           -3.70017631e-03,  3.14708138e-03, -1.28085334e-04, -2.91502631e-03,
           -6.74942502e-03, -2.99724174e-03, -6.63082288e-03, -1.47353041e-02,
           -1.64482836e-02, -1.43265776e-02, -1.26245264e-02, -1.21212871e-02,
           -9.64430136e-03, -8.38326221e-03, -9.00133257e-03, -8.88192471e-03,
           -1.07817257e-02, -1.62788987e-02, -2.05749247e-02, -2.77165177e-02,
           -3.00719600e-02, -2.97939573e-02, -2.48847527e-02, -2.05984210e-02,
           -2.62903812e-02, -2.43443970e-02, -2.09434703e-02, -1.54918661e-02,
           -2.01248914e-02, -1.81070571e-02, -2.11958403e-02, -2.29062920e-02,
           -2.26223942e-02, -1.88590659e-02, -1.93623422e-02, -1.81921624e-02,
           -2.08406452e-02, -2.18448833e-02, -2.56225393e-02, -2.53404928e-02,
           -1.98271808e-02, -1.98229270e-02, -2.01276223e-02, -1.98630309e-02,
           -2.23618630e-02, -2.77148128e-02, -3.22128059e-02, -2.56684448e-02,
           -2.43672229e-02, -2.65348472e-02, -2.52034063e-02, -2.51783515e-02,
           -2.53610374e-02, -1.73883068e-02, -1.53073448e-02, -1.57759022e-02,
           -1.61018053e-02, -1.63921092e-02, -2.02115100e-02, -2.56441060e-02,
           -3.24911800e-02, -3.25086361e-02, -3.18277308e-02, -2.91745078e-02,
           -3.08581512e-02, -2.91136724e-02, -3.24762534e-02, -3.46295533e-02,
           -3.72748676e-02, -4.39210283e-02, -4.38111609e-02, -4.40952224e-02,
           -3.46487569e-02, -4.01613870e-02, -4.16247388e-02, -4.23840138e-02,
           -4.23562694e-02, -4.50139491e-02, -4.41230210e-02, -4.37064691e-02,
           -4.34120163e-02, -4.37828249e-02, -3.98754974e-02, -4.66540015e-02,
           -4.32986843e-02, -4.05821853e-02, -4.34666422e-02, -4.16150083e-02,
           -4.59558717e-02, -5.07302813e-02, -4.96608779e-02, -4.55564404e-02,
           -4.41682655e-02, -3.94216653e-02, -3.90075617e-02, -3.97929099e-02,
           -3.94933653e-02, -4.32006447e-02, -3.99309922e-02, -4.05267904e-02,
           -3.82726365e-02, -3.91218564e-02, -3.73712223e-02, -3.49503795e-02,
           -3.08057734e-02, -3.27144268e-02, -3.70372084e-02, -3.89412493e-02,
           -4.23777548e-02, -3.84595609e-02, -3.49728832e-02, -3.27876093e-02,
           -3.33562288e-02, -3.38602471e-02, -3.42830062e-02, -3.68910907e-02,
           -3.74403111e-02, -3.83331815e-02, -3.77776744e-02, -3.66151214e-02,
           -3.61574504e-02, -3.54042576e-02, -3.84100703e-02, -3.98529131e-02,
           -3.78399608e-02, -3.95123446e-02, -3.66451910e-02, -3.39692068e-02,
           -2.78871484e-02, -3.07214476e-02, -3.20880399e-02, -2.95057426e-02,
           -2.99732688e-02, -2.78547040e-02, -3.23159587e-02, -2.74926230e-02,
           -2.47627438e-02, -2.70495933e-02, -3.08595990e-02, -3.19945566e-02,
           -3.91050953e-02, -3.80559762e-02, -3.96127302e-02, -4.32042139e-02,
           -3.85421780e-02, -3.54390416e-02, -3.30190786e-02, -2.74612496e-02,
           -2.75661161e-02, -2.73366161e-02, -2.86198922e-02, -3.17420610e-02,
           -3.47737959e-02, -3.45262631e-02, -3.30300816e-02, -3.43544778e-02,
           -3.28762805e-02, -3.54810540e-02, -3.36246677e-02, -3.60611122e-02,
           -3.99236530e-02, -3.52838841e-02, -3.72790258e-02, -3.76680345e-02,
           -4.00044674e-02, -3.73781065e-02, -3.20211718e-02, -3.16342297e-02,
           -3.05776794e-02, -2.95899949e-02, -2.66855667e-02, -2.73295599e-02,
           -2.68781063e-02, -2.45653957e-02, -2.54261010e-02, -1.38170469e-02,
           -1.38611654e-02, -1.38497453e-02, -1.80119606e-02, -1.63744474e-02,
           -1.77993410e-02, -1.49911010e-02, -9.28700492e-03, -6.33812381e-03,
           -8.03856473e-03, -9.80516414e-03, -1.05051784e-02, -1.58315018e-02,
           -2.10905630e-02, -1.86963149e-02, -2.37309974e-02, -1.75279772e-02,
           -2.42598057e-02, -2.01590778e-02, -1.68092624e-02, -1.34162062e-02,
           -8.66157694e-03, -4.81063777e-03, -4.84881215e-03, -3.22355867e-03,
           -6.89370639e-03, -1.10501384e-02, -9.81079674e-03, -8.21334244e-03,
            1.68475903e-05, -2.14717291e-03, -2.38488702e-03, -6.11491339e-03,
           -6.60384502e-03, -5.88702039e-03, -1.08120872e-02, -1.18708166e-02,
           -1.34595497e-02, -1.49420402e-02, -1.62296078e-02, -1.77015684e-02,
           -1.56167798e-02, -1.23937825e-02, -1.24612427e-02, -1.10923791e-02,
           -1.07016642e-02, -1.34747853e-02, -1.25162349e-02, -9.60522834e-03,
           -7.97009271e-03, -8.40741542e-03, -7.44930130e-03, -1.15733063e-02,
           -1.35816050e-02, -1.27404656e-02, -9.54799924e-03, -5.75726539e-03]),
     'measures': array([[-0.259659, -0.185094, -0.13778 , ..., -0.084426, -0.187018,
            -0.156286],
           [ 0.144766,  0.054521, -0.202936, ..., -0.259893, -0.120252,
             0.04481 ],
           [ 0.871107,  0.66598 ,  0.723851, ...,  0.710933,  0.616441,
             0.846372],
           ...,
           [-1.011138, -0.923214, -1.060963, ..., -1.084699, -0.821803,
            -1.142022],
           [-1.825087, -1.853427, -1.667516, ..., -1.596906, -1.76395 ,
            -1.792741],
           [-0.059197,  0.157595,  0.336154, ..., -0.167591,  0.127708,
            -0.213384]]),
     'sigma_sqr': 0.40163386742904605}
#### y8vKRuwaIIf42x14F-GhD_ZVcf7VT7ivn93VtA==.subsim:
    {'DeltaE': (0.18405077055761626, 0.0010549199735522142),
     'acceptance': 0.379507709,
     'correlations': array([0.92727194, 0.905011  , 0.88680105, 0.8757215 , 0.87088876,
           0.85814491, 0.84176283, 0.8222462 , 0.81111193, 0.79167185,
           0.77089547, 0.75794414, 0.74795308, 0.73759903, 0.72685223,
           0.71251074, 0.69660122, 0.67826366, 0.66630762, 0.66198322,
           0.64656988, 0.63775305, 0.63361228, 0.61888502, 0.61036559,
           0.60061268, 0.59531515, 0.57994335, 0.56320761, 0.5496747 ,
           0.53390199, 0.52963767, 0.51990726, 0.50815297, 0.50267315,
           0.48643259, 0.47219058, 0.46316019, 0.45517436, 0.44200477,
           0.43696279, 0.43030611, 0.41779831, 0.40991343, 0.39807168,
           0.39425167, 0.39150339, 0.38099464, 0.38085435, 0.3842417 ,
           0.37684745, 0.37219954, 0.36149449, 0.35525363, 0.35227377,
           0.35156942, 0.34095087, 0.33016806, 0.32100311, 0.30805939,
           0.30531058, 0.29653584, 0.29180238, 0.29409258, 0.28909219,
           0.29412888, 0.29289363, 0.28786662, 0.29074446, 0.28331722,
           0.27069787, 0.26650497, 0.25854189, 0.25555922, 0.24900731,
           0.24175297, 0.24010496, 0.23178585, 0.22730126, 0.22229194,
           0.21710532, 0.20598711, 0.20528373, 0.20475345, 0.19988437,
           0.18912629, 0.17884134, 0.17357002, 0.17047803, 0.1692956 ,
           0.16795655, 0.16657323, 0.17445322, 0.16296875, 0.16387349,
           0.16519851, 0.16903257, 0.15963846, 0.15569084, 0.14694249,
           0.14162168, 0.14021819, 0.13899084, 0.14050677, 0.1348373 ,
           0.13388333, 0.13204903, 0.12158895, 0.11720252, 0.12046707,
           0.1172596 , 0.11712292, 0.11427009, 0.11107094, 0.11435711,
           0.11110027, 0.10568085, 0.1059439 , 0.10008672, 0.10816667,
           0.10528267, 0.10552762, 0.10134838, 0.10074872, 0.09406439,
           0.08974935, 0.09271031, 0.09817185, 0.10205276, 0.09092949,
           0.08543807, 0.08837647, 0.08833996, 0.09321835, 0.0993434 ,
           0.09752675, 0.09036669, 0.09013909, 0.09141821, 0.10182243,
           0.09304307, 0.09271596, 0.0937333 , 0.09132754, 0.08804529,
           0.09188741, 0.09048418, 0.10000707, 0.0979808 , 0.09309884]),
     'measures': array([[-0.311377, -0.190296, -0.230309, ..., -0.607925, -0.550802,
            -0.481531],
           [-0.207562, -0.361082,  0.028244, ..., -0.404946, -0.274784,
            -0.448379],
           [-0.369963, -0.514652, -0.252881, ..., -0.242659, -0.24318 ,
            -0.168018],
           ...,
           [ 0.966611,  0.720135,  0.793393, ...,  0.677885,  0.372869,
             0.53839 ],
           [ 0.612724,  0.564648,  0.618097, ...,  0.544178,  0.51024 ,
             0.428384],
           [ 0.292829,  0.545964,  0.713977, ...,  0.026526, -0.097991,
             0.274357]]),
     'meff': (1.180822748624644, 0.006768097188482076),
     'sigma_sqr': 0.665794598199508}
#### z0cjkP4lV98iLO-obJJRBCp1zJmifMct1eCDhA==.subsim:
    {'acceptance': 0.289145856,
     'correlations': array([ 5.30361535e-01,  5.16210468e-01,  5.07196312e-01,  5.05411760e-01,
            4.97744517e-01,  4.89891665e-01,  4.86515059e-01,  4.82235059e-01,
            4.69066790e-01,  4.67622354e-01,  4.65926096e-01,  4.66617085e-01,
            4.63338761e-01,  4.52688218e-01,  4.48515580e-01,  4.43305864e-01,
            4.43277656e-01,  4.38780545e-01,  4.29108793e-01,  4.24891354e-01,
            4.18799018e-01,  4.10840016e-01,  3.96763209e-01,  3.86881236e-01,
            3.83473628e-01,  3.78366508e-01,  3.75257653e-01,  3.70095048e-01,
            3.66507922e-01,  3.62235085e-01,  3.54090852e-01,  3.51481867e-01,
            3.49744789e-01,  3.46557851e-01,  3.49844373e-01,  3.50546732e-01,
            3.46873654e-01,  3.49881747e-01,  3.47121327e-01,  3.46347156e-01,
            3.42356740e-01,  3.36920779e-01,  3.30885991e-01,  3.28974224e-01,
            3.22955091e-01,  3.22443972e-01,  3.17422993e-01,  3.14034460e-01,
            3.09522146e-01,  3.07870453e-01,  3.07561680e-01,  3.04742221e-01,
            3.02044861e-01,  2.99582496e-01,  2.97707255e-01,  2.95110475e-01,
            2.92701888e-01,  2.92454939e-01,  2.92631860e-01,  2.81113412e-01,
            2.78267081e-01,  2.72855102e-01,  2.73387670e-01,  2.70159039e-01,
            2.69671150e-01,  2.61797994e-01,  2.60466743e-01,  2.58389560e-01,
            2.55850472e-01,  2.51822905e-01,  2.49751151e-01,  2.43632204e-01,
            2.35572672e-01,  2.39172549e-01,  2.33705543e-01,  2.26463713e-01,
            2.20661559e-01,  2.18379147e-01,  2.19140081e-01,  2.17503038e-01,
            2.17658825e-01,  2.15030651e-01,  2.17143115e-01,  2.08206796e-01,
            2.04376458e-01,  2.06917659e-01,  2.00606723e-01,  1.95912134e-01,
            1.88643651e-01,  1.85025179e-01,  1.87380448e-01,  1.89852671e-01,
            1.86701677e-01,  1.77242656e-01,  1.78666125e-01,  1.78142625e-01,
            1.75031117e-01,  1.70991741e-01,  1.68135448e-01,  1.66927979e-01,
            1.65268941e-01,  1.62635612e-01,  1.66952331e-01,  1.63296133e-01,
            1.55828814e-01,  1.59280763e-01,  1.52988163e-01,  1.53587071e-01,
            1.50127200e-01,  1.49892373e-01,  1.45812974e-01,  1.46500964e-01,
            1.44525363e-01,  1.44474688e-01,  1.35670821e-01,  1.32245144e-01,
            1.33303214e-01,  1.29339071e-01,  1.27630728e-01,  1.29272617e-01,
            1.24918860e-01,  1.21768597e-01,  1.25096556e-01,  1.24683631e-01,
            1.25454758e-01,  1.27009976e-01,  1.22122685e-01,  1.15752698e-01,
            1.10959046e-01,  1.03916245e-01,  1.07069942e-01,  1.07962853e-01,
            1.08681916e-01,  1.04908486e-01,  1.05073695e-01,  1.09985180e-01,
            1.08761778e-01,  1.08545609e-01,  1.02135522e-01,  9.91012669e-02,
            9.45481746e-02,  1.01486212e-01,  9.88253697e-02,  9.86310737e-02,
            8.97384832e-02,  8.54088155e-02,  8.42670419e-02,  8.60938631e-02,
            8.66753183e-02,  8.82255212e-02,  8.56547624e-02,  8.48050849e-02,
            8.79575357e-02,  8.88572830e-02,  7.91735755e-02,  7.77698888e-02,
            7.95656125e-02,  7.88304958e-02,  7.27317095e-02,  7.36940417e-02,
            7.15797238e-02,  7.49039303e-02,  7.50889535e-02,  7.09843641e-02,
            7.46866193e-02,  7.56115137e-02,  7.90752333e-02,  7.16000007e-02,
            7.88390808e-02,  7.08522003e-02,  6.76432639e-02,  6.27755210e-02,
            5.97074325e-02,  6.35774312e-02,  6.09554649e-02,  5.70138324e-02,
            5.85070592e-02,  5.38672810e-02,  5.59498436e-02,  5.32388816e-02,
            5.47784927e-02,  6.39587254e-02,  6.46469126e-02,  6.51586133e-02,
            6.51877258e-02,  6.27191652e-02,  5.83490998e-02,  6.05608130e-02,
            5.32647021e-02,  5.13986500e-02,  5.65706665e-02,  5.43424142e-02,
            5.68028193e-02,  4.97960647e-02,  4.64756309e-02,  4.71097075e-02,
            4.40711616e-02,  4.25282491e-02,  4.68347151e-02,  4.02230762e-02,
            4.00193372e-02,  3.98753376e-02,  3.42055788e-02,  3.21675328e-02,
            3.64013946e-02,  4.03456845e-02,  3.97587347e-02,  3.45893722e-02,
            3.39666230e-02,  3.49152104e-02,  3.32969078e-02,  3.51075547e-02,
            3.17630675e-02,  2.64388741e-02,  2.69046159e-02,  2.53129472e-02,
            2.33370834e-02,  2.39591598e-02,  1.53826603e-02,  1.43263467e-02,
            1.79347488e-02,  1.27275458e-02,  1.28169558e-02,  1.07215247e-02,
            1.41726834e-02,  1.50658873e-02,  1.33207910e-02,  1.64489873e-02,
            1.23821125e-02,  1.10505346e-02,  6.61423028e-03,  4.79331762e-03,
            5.20999633e-03, -1.47798627e-03, -1.10844502e-03, -4.23320781e-04,
            3.68889975e-03,  4.62254066e-03,  3.98083171e-03,  3.39672931e-03,
            4.71493687e-03,  6.20301799e-03,  1.08788598e-02,  1.19162656e-02,
            1.76238552e-02,  1.47716309e-02,  2.20870579e-02,  1.64653695e-02,
            1.62439080e-02,  2.31883502e-02,  2.39009611e-02,  2.41291147e-02,
            3.03345328e-02,  2.89247330e-02,  3.04241363e-02,  2.83646693e-02,
            3.09442843e-02,  2.56849289e-02,  2.49157866e-02,  2.29936711e-02,
            2.37068546e-02,  1.97722571e-02,  1.43123049e-02,  1.11646458e-02,
            1.97981837e-02,  1.58951098e-02,  1.76238574e-02,  2.08003620e-02,
            1.91569885e-02,  2.12696893e-02,  1.96032033e-02,  2.53181055e-02,
            2.08587313e-02,  2.02367340e-02,  2.25145802e-02,  2.24582182e-02,
            1.79533440e-02,  1.60567411e-02,  1.54662140e-02,  1.57547841e-02,
            1.21816774e-02,  1.92398283e-02,  1.79256688e-02,  1.84060568e-02,
            1.85263686e-02,  2.43099766e-02,  2.36808126e-02,  2.45538283e-02,
            3.16869427e-02,  2.62313976e-02,  2.48468054e-02,  2.33322979e-02,
            2.44045853e-02,  2.13914752e-02,  2.40391278e-02,  2.25292156e-02,
            2.21220774e-02,  2.17577642e-02,  2.06972747e-02,  2.31843533e-02,
            2.65609983e-02,  2.97221460e-02,  3.32503520e-02,  3.00306028e-02,
            2.16890061e-02,  1.34204478e-02,  1.38280787e-02,  1.51384635e-02,
            1.08718769e-02,  8.42922199e-03,  1.05079081e-02,  9.04602248e-03,
            6.16086387e-03,  1.49586642e-02,  1.97544873e-02,  1.99506351e-02,
            1.67769572e-02,  1.87270026e-02,  1.63596881e-02,  1.73134213e-02,
            1.96000388e-02,  1.85259183e-02,  1.65961212e-02,  1.62274649e-02,
            1.73643246e-02,  1.78182862e-02,  2.07181282e-02,  2.08723060e-02,
            1.88960237e-02,  1.70740103e-02,  1.45426153e-02,  1.56331774e-02,
            1.13836921e-02,  1.06902558e-02,  1.14982132e-02,  1.50764470e-02,
            1.22273221e-02,  1.26260954e-02,  1.21949489e-02,  1.50965440e-02,
            1.07700138e-02,  7.82608687e-03,  7.72029938e-03,  8.46142256e-03,
            1.00301639e-02,  9.65402993e-03,  2.46515778e-03,  3.71456610e-03,
            1.33226153e-03,  3.20606316e-03,  5.64342350e-03,  4.77887048e-03,
            4.19040770e-03,  6.12481476e-03,  3.22345011e-03, -9.38886019e-04,
            2.53567434e-03,  3.23827072e-03,  1.40752013e-03,  4.05589570e-03,
            6.38042401e-03,  3.55106387e-03,  7.35877976e-03,  6.45618625e-03,
            7.29029001e-03,  3.18820912e-03, -5.58614781e-03, -5.06342875e-03,
           -1.74188165e-03,  1.03586401e-03,  2.72540186e-03, -1.80109421e-03,
           -3.27187388e-03, -7.20110392e-03, -6.64063013e-03, -1.59998648e-02,
           -1.48656217e-02, -1.93772061e-02, -1.95551923e-02, -1.20583079e-02,
           -8.74515038e-03, -5.27883678e-03, -5.27072706e-03, -3.03512977e-03,
           -6.14000432e-03, -6.23823275e-03, -9.29050512e-03, -7.74381306e-03,
           -1.12863939e-02, -8.69345661e-03, -1.24413540e-02, -9.02242879e-03,
           -7.52419946e-03, -5.89131231e-03, -2.55489267e-03, -7.62155391e-04,
            4.95149941e-04,  2.62439533e-03,  2.00709572e-03, -1.39014219e-03,
           -7.58413412e-03, -5.79837910e-03, -6.85009410e-03, -8.59186086e-03,
           -1.04918266e-02, -3.90722832e-03, -4.04386235e-03, -2.42986119e-03,
           -6.89271359e-03, -1.09267838e-02, -1.24981714e-02, -8.21407852e-03,
           -5.58819374e-03, -6.05412557e-03, -1.25922564e-02, -1.18397602e-02,
           -3.20040682e-03, -5.96242810e-03, -8.74936836e-04, -3.99930366e-04,
           -7.67311460e-03, -6.30533507e-03, -8.54136993e-03, -7.85777786e-03,
           -6.48645341e-03, -5.09089072e-03, -3.09749921e-03, -2.38749930e-03,
           -1.35137573e-03, -4.11281601e-03, -4.52008899e-03, -7.57273696e-03,
           -2.11404183e-03, -5.89476269e-03, -4.24321323e-03, -8.56984810e-03,
           -6.23724220e-03, -4.24047742e-03, -2.84105879e-03,  8.29200494e-04,
            5.19346229e-03,  7.68459568e-04, -1.40862630e-03,  3.44996652e-03,
            4.56312052e-03,  1.14096521e-02,  1.02736268e-02,  1.26556487e-02,
            1.23876932e-02,  6.69982568e-03,  8.16398397e-03,  1.00340007e-02,
            1.02372582e-02,  1.15314987e-02,  7.51858737e-03,  7.81710051e-03,
            7.38754248e-03,  7.32309512e-03,  7.38778530e-03,  3.56701664e-03,
           -4.00724517e-03, -5.18432228e-03, -8.71708610e-04,  1.83345436e-03,
           -2.94623257e-03, -5.94633930e-05, -3.38834817e-03, -6.58154510e-03,
           -5.85167495e-03, -4.13080856e-03, -3.63749358e-03, -6.70684466e-03,
           -1.07498771e-02, -4.88914925e-03, -4.02170171e-03, -1.26856301e-02,
           -1.59747518e-02, -1.34811491e-02, -1.59028569e-02, -2.06366660e-02,
           -1.41020932e-02, -1.91553584e-02, -1.84358637e-02, -2.14227568e-02,
           -2.27915077e-02, -2.15382541e-02, -2.38076163e-02, -2.66380440e-02,
           -2.31198071e-02, -2.19006688e-02, -2.48572242e-02, -2.90165309e-02,
           -2.70450687e-02, -2.65504771e-02, -2.43018911e-02, -3.07273081e-02,
           -3.19552794e-02, -3.43024651e-02, -3.99463505e-02, -4.56387503e-02]),
     'measures': array([[-1.577606, -1.360675, -1.193536, ..., -1.32483 , -1.199154,
            -1.483959],
           [ 0.161716,  0.135353,  0.102304, ..., -0.347043, -0.056887,
            -0.125951],
           [-0.374972, -0.326473, -0.143799, ..., -0.299653, -0.415653,
            -0.598693],
           ...,
           [ 0.302075,  0.545831,  0.557278, ..., -0.046829, -0.063518,
             0.308588],
           [ 0.894713,  0.839884,  0.531001, ...,  0.563434,  0.758718,
             0.776227],
           [-0.398731, -0.211272,  0.088072, ...,  0.819194,  0.620953,
             0.016041]]),
     'sigma_sqr': 0.42625627902083896}
#### nMDudP5twtBqL6zWsH8KkW7K03_oYGs_yAMxsA==.subsim:
    {'acceptance': 0.18505792000000001,
     'measures': array([[-0.138994, -0.26628 , -0.295113, ..., -0.079115, -0.068605,
            -0.067982],
           [-0.215065, -0.380589, -0.556807, ..., -0.093449, -0.195489,
            -0.194999],
           [-0.158415, -0.182246, -0.087562, ...,  0.057582,  0.111905,
            -0.076374],
           ...,
           [-0.266015, -0.162982, -0.103888, ..., -0.158875, -0.246077,
            -0.18678 ],
           [-0.576153, -0.538302, -0.402486, ..., -0.696625, -0.732037,
            -0.761395],
           [-0.09422 , -0.313024, -0.42269 , ..., -0.154269, -0.201829,
            -0.112249]])}
#### 5y5OEYcdqJlIED2tALLyk0YoIsbI4iNo9ifaOg==.subsim:
    {'acceptance': 0.176009908,
     'measures': array([[-0.139822, -0.150169, -0.152895, ..., -0.138491, -0.112378,
            -0.082584],
           [-0.2675  , -0.313675, -0.147438, ..., -0.401776, -0.36583 ,
            -0.229427],
           [-0.216294, -0.375445, -0.329608, ..., -0.052953, -0.200541,
            -0.248128],
           ...,
           [-1.472582, -1.441194, -1.382607, ..., -1.423088, -1.552219,
            -1.543794],
           [-1.082587, -1.105853, -1.00959 , ..., -0.794449, -0.829113,
            -0.842139],
           [-1.334594, -1.364268, -1.267682, ..., -1.460815, -1.397976,
            -1.275771]])}
#### UzfvnH8JmG2j_Q3EfLyRD3kFHLMXaKOQrQwcVA==.subsim:
    {'acceptance': 0.27964308000000004,
     'measures': array([[-0.223173, -0.270852, -0.038931, ..., -0.328008, -0.405526,
            -0.571679],
           [-0.316983, -0.104094, -0.287333, ..., -0.053563,  0.000958,
            -0.256963],
           [-0.844417, -0.502643, -0.651248, ..., -0.63247 , -0.817712,
            -0.83461 ],
           ...,
           [-0.796903, -0.485301, -0.425251, ..., -0.63827 , -0.753862,
            -0.939659],
           [ 0.019639, -0.04056 , -0.163573, ..., -0.424019, -0.304744,
             0.025817],
           [-0.012068, -0.089559, -0.063349, ..., -0.302976, -0.189831,
            -0.109624]])}
#### T9AbvJWkcdRi_yfn4Ay-NswkB58EwRPNZ98QrQ==.subsim:
    {'acceptance': 0.29989412000000004,
     'measures': array([[-0.424663, -0.396366, -0.205149, ..., -0.412517, -0.405234,
            -0.306844],
           [-0.131714, -0.255937, -0.226822, ...,  0.112663, -0.043116,
            -0.171909],
           [-0.621555, -0.380687, -0.297121, ..., -0.383173, -0.552786,
            -0.68307 ],
           ...,
           [ 0.540292,  0.738806,  0.752117, ...,  0.585179,  0.421103,
             0.363149],
           [ 0.887251,  0.709542,  0.721437, ...,  0.88916 ,  0.818118,
             0.811669],
           [ 0.859761,  0.853377,  0.827239, ...,  0.901654,  0.802849,
             0.927819]])}
#### 6ryzoedRAIMQdBhAuo4kkW19XSVfWm3hb2K8Kg==.subsim:
    {'acceptance': 0.7123448780000001,
     'correlations': array([ 6.88300924e+00,  6.87961318e+00,  6.72903567e+00,  6.58423195e+00,
            6.47088344e+00,  6.35155338e+00,  6.23072818e+00,  6.11373804e+00,
            5.99454890e+00,  5.85694551e+00,  5.64356496e+00,  5.56620634e+00,
            5.44096140e+00,  5.30424920e+00,  5.18327023e+00,  4.98594690e+00,
            4.88424577e+00,  4.65426577e+00,  4.52513028e+00,  4.42888797e+00,
            4.31969505e+00,  4.27758567e+00,  4.14503426e+00,  4.02948442e+00,
            3.98583108e+00,  4.00213969e+00,  3.90391300e+00,  3.89544442e+00,
            3.87569849e+00,  3.83836713e+00,  3.80433996e+00,  3.70187783e+00,
            3.59499141e+00,  3.57722097e+00,  3.53254516e+00,  3.46012534e+00,
            3.47952060e+00,  3.35079632e+00,  3.27456316e+00,  3.22577147e+00,
            3.18883893e+00,  3.13439215e+00,  3.03519991e+00,  2.93731599e+00,
            2.90736351e+00,  2.85811710e+00,  2.76824984e+00,  2.70400673e+00,
            2.61226355e+00,  2.60292436e+00,  2.54349379e+00,  2.46278924e+00,
            2.42030162e+00,  2.40317380e+00,  2.31431636e+00,  2.25155589e+00,
            2.13181658e+00,  2.10738581e+00,  2.07833970e+00,  2.05064926e+00,
            2.03455610e+00,  2.00327089e+00,  1.91779022e+00,  1.91827643e+00,
            1.85568510e+00,  1.83820471e+00,  1.86559846e+00,  1.84865480e+00,
            1.78501740e+00,  1.69138771e+00,  1.74772065e+00,  1.69303343e+00,
            1.58461210e+00,  1.57515221e+00,  1.55927095e+00,  1.41722765e+00,
            1.40953648e+00,  1.38480469e+00,  1.33387508e+00,  1.33731475e+00,
            1.32009284e+00,  1.33472143e+00,  1.36653661e+00,  1.37464894e+00,
            1.38523126e+00,  1.37110316e+00,  1.31884865e+00,  1.36495669e+00,
            1.33921276e+00,  1.25425592e+00,  1.28600884e+00,  1.23451530e+00,
            1.19971312e+00,  1.23790904e+00,  1.23516730e+00,  1.18111872e+00,
            1.13750346e+00,  1.09230580e+00,  1.04151675e+00,  9.98152559e-01,
            9.37117093e-01,  9.81318085e-01,  9.48734313e-01,  9.13085703e-01,
            9.61953460e-01,  1.03750471e+00,  9.62125737e-01,  9.14862162e-01,
            8.43193595e-01,  8.54730130e-01,  7.32859662e-01,  7.64904970e-01,
            7.53062480e-01,  7.59050575e-01,  7.23051485e-01,  7.33725452e-01,
            6.85888281e-01,  6.70215743e-01,  6.45725549e-01,  5.93522666e-01,
            5.16975812e-01,  5.63869657e-01,  5.60135471e-01,  5.81031312e-01,
            4.83699557e-01,  5.33871353e-01,  5.59026200e-01,  5.75331425e-01,
            5.65133362e-01,  5.29293425e-01,  5.27313499e-01,  5.05496234e-01,
            4.35586376e-01,  4.16871769e-01,  4.83425742e-01,  4.76151729e-01,
            5.15317372e-01,  5.62403319e-01,  5.30330861e-01,  4.51873974e-01,
            5.13940006e-01,  5.36367709e-01,  5.29846678e-01,  5.14748909e-01,
            5.60093819e-01,  5.76624378e-01,  6.11668027e-01,  6.09582778e-01,
            5.58196228e-01,  5.39311071e-01,  5.22464180e-01,  5.04177841e-01,
            4.73599151e-01,  4.28522136e-01,  4.33410753e-01,  3.89511155e-01,
            3.81142063e-01,  3.61926387e-01,  3.28421859e-01,  4.34229907e-01,
            4.87469365e-01,  3.89521913e-01,  4.08255626e-01,  3.32704875e-01,
            3.00236821e-01,  2.77269722e-01,  1.93305724e-01,  1.62075423e-01,
            1.44421002e-01,  8.06902329e-02,  4.83754815e-02,  1.98190732e-02,
            5.72446104e-02, -7.84127464e-03, -6.78776781e-02, -8.83887548e-02,
           -6.56112928e-02, -9.28286051e-02, -1.95418313e-02, -7.07344921e-02,
           -8.56035009e-02, -9.20422982e-02, -4.16022315e-02, -9.05183984e-02,
           -1.24857630e-01, -1.23774024e-01, -1.59777869e-01, -1.60500762e-01,
           -1.07732909e-01, -5.02848464e-02, -9.22436860e-02, -1.15092195e-02,
           -7.36838023e-02,  4.87823028e-03, -2.65242970e-02,  6.63258271e-03,
            9.87023848e-02,  1.65984357e-01,  9.70985380e-02,  1.81147652e-01,
            1.38966284e-01,  1.48103128e-01,  1.17064248e-01,  7.42589539e-02,
            2.75405866e-02, -8.49916749e-04, -4.82050321e-02, -6.88520139e-02,
           -1.24210740e-01, -1.63104612e-01, -1.63615615e-01, -2.06682798e-01,
           -2.08044954e-01, -1.75993386e-01, -1.20305429e-01, -1.02393723e-01,
           -1.00547468e-01, -1.69351389e-01, -1.55100402e-01, -1.25081573e-01,
           -1.18992186e-01, -7.93441194e-02, -6.78109533e-02, -1.02771901e-01,
           -4.16651705e-02, -1.05508026e-01, -1.54644393e-01, -1.33835645e-01,
           -2.23653782e-01, -2.21373588e-01, -2.40549265e-01, -2.84856497e-01,
           -2.51527031e-01, -2.30839287e-01, -1.90340607e-01, -2.07841393e-01,
           -2.46166803e-01, -1.97884720e-01, -2.79219618e-01, -2.92101491e-01,
           -1.86238198e-01, -1.90372921e-01, -2.13642865e-01, -2.12119197e-01,
           -1.40291062e-01, -9.10664321e-02, -6.24127554e-02, -5.87049801e-02,
           -5.94040714e-02,  6.39052473e-03, -2.89353094e-02,  5.91958070e-02,
            3.34805996e-02,  3.07968128e-02,  1.41014004e-01,  1.66483423e-01,
            1.24459422e-01,  1.56937194e-01,  1.28988966e-01,  9.62023315e-02,
            8.16962875e-02,  2.07194077e-02,  6.86740793e-03, -4.11917444e-02,
            1.35622409e-03,  1.42745536e-02,  5.38615281e-02,  2.29920083e-02,
           -1.15583754e-01, -8.15535694e-02, -1.66205973e-01, -1.94020047e-01,
           -2.13925996e-01, -1.84130468e-01, -1.45387844e-01, -1.08601308e-01,
           -9.71471375e-02, -5.71015943e-02, -8.43986174e-02, -2.95593479e-02,
           -1.36687200e-01, -1.45101681e-01, -1.80140857e-01, -1.79060058e-01,
           -2.13843390e-01, -1.62914688e-01, -1.56484259e-01, -1.24793281e-01,
           -1.38740668e-01, -2.03743516e-01, -1.48469328e-01, -1.24941362e-01,
           -1.25378741e-01, -1.02506004e-01, -7.21410656e-02, -3.75939133e-02,
           -2.17725592e-02,  1.15328384e-02,  1.01191900e-02,  2.91996174e-02,
           -2.50956203e-02,  1.63343924e-02,  1.21009590e-01,  1.74724717e-01,
            2.00656110e-01,  1.80382631e-01,  1.60020005e-01,  2.02714040e-01,
            2.89467398e-01,  2.67791834e-01,  2.32074310e-01,  1.55501543e-01,
            1.44203050e-01,  1.06429932e-01,  4.14890042e-02,  4.51470944e-03,
            4.98723616e-03,  9.26912639e-02,  4.81642505e-02,  2.21499145e-02,
            2.88106945e-02,  4.65598831e-02,  7.37705777e-02,  9.81329629e-02,
            1.00209397e-01,  1.15954902e-01,  1.09039210e-01,  1.18633317e-01,
            1.10229946e-01,  3.20223489e-02, -2.14799567e-02, -6.77512510e-02,
           -7.06464834e-02, -8.69099458e-02, -3.17767755e-02,  2.18912987e-02,
            6.73456299e-03,  1.84027022e-02,  1.33730096e-04, -6.25865266e-02,
           -6.88985896e-02, -1.31012263e-01, -1.76629455e-01, -1.38582009e-01,
           -3.36535136e-02, -3.30772326e-02, -1.24644499e-02,  2.55308663e-02,
            1.80472162e-02,  1.41694601e-01,  2.34522422e-01,  1.99364982e-01,
            2.25359157e-01,  2.22873788e-01,  2.05044340e-01,  2.12451958e-01,
            2.16089871e-01,  2.22630949e-01,  2.56438100e-01,  2.39652136e-01,
            2.23052702e-01,  1.17931821e-01,  1.75724483e-01,  1.92200528e-01,
            2.04486926e-01,  1.86034530e-01,  1.82420028e-01,  5.45855579e-02,
           -3.57004658e-02, -6.91772226e-02, -6.98108031e-02, -3.98341329e-02,
           -4.93128576e-02, -7.33080847e-03,  1.56149567e-02, -1.19817469e-02,
            4.38130762e-02, -7.03395064e-02, -5.68723022e-02,  5.37354828e-02,
            7.64460477e-02,  2.66568005e-02,  7.62650159e-02,  1.45496005e-01,
            1.21285615e-01,  1.75634981e-01,  1.80685049e-01,  1.72211812e-01,
            1.48020380e-01,  1.70590096e-01,  1.91375249e-01,  1.54085663e-01,
            2.12976376e-01,  1.73814384e-01,  1.40266932e-01,  2.30058442e-01,
            2.57798757e-01,  2.37067938e-01,  2.60707672e-01,  2.41043081e-01,
            3.12357244e-01,  3.05135831e-01,  2.56333354e-01,  2.26058761e-01,
            1.63505241e-01,  2.01716727e-01,  2.53188627e-01,  1.91808495e-01,
            2.65683269e-01,  2.36117615e-01,  2.82663543e-01,  3.17685565e-01,
            2.66616984e-01,  2.79497629e-01,  2.08821374e-01,  2.45347220e-01,
            2.22610188e-01,  2.67116914e-01,  2.35531101e-01,  1.97133390e-01,
            2.20503923e-01,  2.08023646e-01,  3.12434686e-01,  3.22605062e-01,
            3.24190199e-01,  3.67460226e-01,  3.64691337e-01,  4.28776669e-01,
            3.60973996e-01,  3.13983589e-01,  2.74229077e-01,  2.53914947e-01,
            2.88689360e-01,  3.25340826e-01,  2.81048480e-01,  2.84811973e-01,
            2.03954407e-01,  1.87821892e-01,  2.08079726e-01,  2.02830190e-01,
            1.84835922e-01,  1.96533546e-01,  2.41855201e-01,  2.95506850e-01,
            3.57950719e-01,  3.07015745e-01,  3.03103635e-01,  3.26353188e-01,
            2.62466114e-01,  2.35767227e-01,  2.12346355e-01,  1.84496741e-01,
            1.95994383e-01,  1.08812520e-01,  6.88078316e-02,  9.48204581e-02,
            9.63786909e-02,  5.21826839e-02,  1.34592691e-01,  7.31164324e-02,
            5.93760738e-02,  1.04956472e-01,  2.08222290e-01,  1.90012266e-01,
            1.59221982e-01,  1.60132247e-01,  1.30395660e-01,  1.39589356e-01,
            1.60255018e-01,  1.44239845e-01,  1.37884494e-01,  1.18748331e-01,
            1.46746041e-01,  2.27234307e-01,  2.55687800e-01,  2.59192649e-01,
            2.97021637e-01,  2.70562703e-01,  3.31234997e-01,  3.65863138e-01,
            3.01564666e-01,  3.31688391e-01,  2.55692755e-01,  2.89372121e-01,
            2.50144845e-01,  3.31695902e-01,  3.68624010e-01,  4.27687044e-01,
            4.06149637e-01,  4.20134721e-01,  4.75596173e-01,  4.10448664e-01,
            3.52623529e-01,  3.23179297e-01,  3.28193697e-01,  2.85589433e-01,
            3.65930897e-01,  4.75650132e-01,  4.84150273e-01,  5.04124082e-01]),
     'measures': array([[-3.262775, -2.488364, -3.010628, ..., -2.997792, -2.755443,
            -3.147601],
           [-1.540722, -1.431906, -1.54609 , ..., -1.436665, -1.257124,
            -1.248795],
           [ 2.195882,  3.036549,  3.664841, ...,  2.178718,  2.349801,
             2.257497],
           ...,
           [-2.101394, -2.695639, -3.363309, ..., -3.374102, -2.442132,
            -2.3437  ],
           [-2.578285, -2.430829, -1.748769, ..., -1.437938, -2.520118,
            -2.999098],
           [-1.702859, -2.204101, -1.778158, ..., -2.73677 , -2.684251,
            -2.063647]]),
     'sigma_sqr': 6.888594245385721}
#### MW8YIVXCgFKWYx745sPvLlR7T0YThz8YNmjIGw==.subsim:
    {'DeltaE': (0.1925554728726926, 0.001718403825996487),
     'acceptance': 0.399073015,
     'correlations': array([1.19625245, 1.17917881, 1.1606269 , 1.13705532, 1.11752375,
           1.09676378, 1.07116938, 1.0386327 , 1.02777403, 1.0202382 ,
           1.00378728, 0.98927224, 0.96811332, 0.95327624, 0.93381601,
           0.91609182, 0.90428183, 0.89716906, 0.88130465, 0.86089877,
           0.84964774, 0.8355388 , 0.82254101, 0.80908361, 0.8002543 ,
           0.79012616, 0.78026846, 0.76433141, 0.75825428, 0.73870916,
           0.72476851, 0.71337618, 0.69900112, 0.68960188, 0.67865487,
           0.6679924 , 0.65648389, 0.64583903, 0.63352158, 0.62221686,
           0.60862885, 0.59337387, 0.57748228, 0.57926956, 0.56869125,
           0.55364452, 0.54657566, 0.52816431, 0.5228285 , 0.51565041,
           0.50591571, 0.49299094, 0.48534585, 0.47541991, 0.47226331,
           0.47299089, 0.46255482, 0.45932283, 0.45315524, 0.43829852,
           0.43326043, 0.42510301, 0.41691818, 0.40681849, 0.390017  ,
           0.38416433, 0.37808392, 0.36568111, 0.3645981 , 0.34893105,
           0.34018087, 0.33454711, 0.3269814 , 0.32065527, 0.30589981,
           0.30294016, 0.29566851, 0.29725039, 0.29845022, 0.28682253,
           0.2761354 , 0.26308653, 0.25477373, 0.24977216, 0.2403788 ,
           0.23434905, 0.22482931, 0.21738316, 0.210445  , 0.20450987,
           0.20855229, 0.20907934, 0.20494184, 0.20626876, 0.19472886,
           0.1899435 , 0.18373892, 0.1852929 , 0.17866767, 0.1728678 ,
           0.18165253, 0.1793827 , 0.17937427, 0.18231551, 0.17436648,
           0.1669435 , 0.1604106 , 0.16464858, 0.14983383, 0.14796663,
           0.15044306, 0.1483663 , 0.13469779, 0.13770539, 0.1334174 ,
           0.13720034, 0.13819056, 0.14276901, 0.14270893, 0.13580997,
           0.1339042 , 0.12863627, 0.12493939, 0.12004304, 0.11204358,
           0.11122787, 0.10292485, 0.09957956, 0.08754473, 0.08217107,
           0.06940985, 0.07252824, 0.07405806, 0.06892436, 0.06867914,
           0.06497889, 0.06274051, 0.06543786, 0.06706405, 0.065775  ,
           0.06538825, 0.06917212, 0.06396378, 0.06221397, 0.05713334,
           0.05932099, 0.05987934, 0.06349426, 0.06993641, 0.07866761]),
     'measures': array([[-0.695707, -0.991479, -1.143485, ..., -0.86206 , -0.86997 ,
            -0.797076],
           [-0.135146, -0.247453, -0.530755, ...,  0.112659, -0.010179,
            -0.139828],
           [-0.327047, -0.09396 , -0.270767, ..., -0.206332, -0.327537,
            -0.359379],
           ...,
           [-1.048805, -0.616171, -0.604681, ..., -1.074598, -1.01775 ,
            -1.096361],
           [ 0.624843,  0.568317,  0.822697, ...,  0.330527,  0.375806,
             0.339537],
           [ 0.239201,  0.177128,  0.042435, ...,  0.678448,  0.416135,
             0.450505]]),
     'meff': (1.0788181831642627, 0.00962759077084094),
     'sigma_sqr': 0.8382183444914335}
#### xORpZkGyuNhY_GcRzU5irBX8IXPzsAoW_uqZsA==.subsim:
    {'DeltaE': (0.1324393145707101, 0.001669969841093112),
     'acceptance': 0.32085869300000003,
     'correlations': array([0.65353658, 0.63867549, 0.62818767, 0.61644151, 0.60752327,
           0.59387389, 0.5832879 , 0.57252133, 0.56573372, 0.5536883 ,
           0.54095861, 0.53677345, 0.52767371, 0.51702415, 0.50202271,
           0.49603731, 0.485884  , 0.47776208, 0.4721646 , 0.46399677,
           0.4555141 , 0.4460335 , 0.44102563, 0.43370446, 0.42708562,
           0.41990279, 0.41483083, 0.40767277, 0.39639939, 0.39009089,
           0.38030471, 0.37102716, 0.36568487, 0.35684591, 0.35437216,
           0.34748923, 0.33884677, 0.33220571, 0.32809194, 0.322524  ,
           0.32239139, 0.32176957, 0.31890043, 0.31632675, 0.30890828,
           0.30786939, 0.3059115 , 0.29931474, 0.29716369, 0.29063879,
           0.28375682, 0.27948394, 0.28237865, 0.28027952, 0.27662606,
           0.27341796, 0.27003524, 0.25969215, 0.25742414, 0.26192838,
           0.26078069, 0.25997783, 0.25443027, 0.24581481, 0.25175015,
           0.25128427, 0.24884727, 0.24085904, 0.2412675 , 0.23795739,
           0.23676645, 0.23433533, 0.23161591, 0.22951614, 0.22842175,
           0.22924892, 0.22509341, 0.22153963, 0.21769738, 0.21453042,
           0.2145659 , 0.20845963, 0.20541479, 0.19941372, 0.19772208,
           0.19593276, 0.19008005, 0.18803048, 0.1901753 , 0.18751689,
           0.18272018, 0.17775236, 0.17451475, 0.17595678, 0.17585278,
           0.17598344, 0.17462126, 0.1692444 , 0.17547801, 0.17176236,
           0.17006103, 0.17135176, 0.16684472, 0.16911407, 0.16949934,
           0.17151434, 0.1697476 , 0.17339619, 0.17267276, 0.1694568 ,
           0.16722734, 0.16793288, 0.17451366, 0.17270886, 0.17202588,
           0.17356548, 0.17039021, 0.17214582, 0.16700363, 0.1659847 ,
           0.16421044, 0.16178301, 0.1660056 , 0.16827076, 0.17227414,
           0.16946445, 0.17720174, 0.17606926, 0.17251216, 0.16915942,
           0.16248414, 0.16259014, 0.16420143, 0.16377972, 0.16801817,
           0.16877547, 0.17170687, 0.1771377 , 0.17534213, 0.17990461,
           0.18322491, 0.18841445, 0.18597589, 0.18503287, 0.1875668 ,
           0.19085736, 0.19108417, 0.19313879, 0.1900566 , 0.18957683]),
     'measures': array([[ 9.407160e-01,  4.829980e-01,  3.586400e-01, ...,  6.218170e-01,
             5.353240e-01,  5.867100e-01],
           [ 5.341900e-02,  1.165360e-01, -5.692500e-02, ...,  4.932730e-01,
             3.427400e-02,  1.662700e-02],
           [-1.906710e-01, -1.417420e-01,  2.050000e-04, ..., -4.098250e-01,
            -5.654210e-01, -6.397330e-01],
           ...,
           [-7.678500e-02, -8.188600e-02, -7.638800e-02, ...,  5.828600e-02,
             7.471800e-02,  4.286600e-02],
           [-5.732780e-01, -4.713160e-01, -4.837440e-01, ..., -4.300690e-01,
            -4.701030e-01, -4.796280e-01],
           [-1.188028e+00, -1.296958e+00, -1.170621e+00, ..., -1.185537e+00,
            -1.362030e+00, -1.388509e+00]]),
     'meff': (2.2804793419215765, 0.028755296239558914),
     'sigma_sqr': 0.43069818832834517}
#### LJAhc1nT-smL4CLowHLFEkID0FbaD3jR4v9VTw==.subsim:
    {'acceptance': 0.279917858,
     'correlations': array([ 0.4530869 ,  0.43419482,  0.42579278,  0.41763074,  0.40379189,
            0.39024207,  0.3839889 ,  0.37814377,  0.36951643,  0.36971504,
            0.36965966,  0.36498266,  0.36513654,  0.36021149,  0.35728518,
            0.35345422,  0.34713401,  0.34457754,  0.34017207,  0.33239677,
            0.33189412,  0.32522299,  0.32694018,  0.31976293,  0.31275631,
            0.31348745,  0.30593263,  0.30223258,  0.29770857,  0.29328285,
            0.28976852,  0.27992104,  0.27386943,  0.26539271,  0.26525547,
            0.26511322,  0.26018467,  0.25613778,  0.2525735 ,  0.25272453,
            0.24587561,  0.24272102,  0.24204062,  0.23304687,  0.23335279,
            0.22440043,  0.21917079,  0.2107365 ,  0.21427811,  0.2101728 ,
            0.20884351,  0.20557856,  0.20483789,  0.20146642,  0.19675946,
            0.18964146,  0.18848199,  0.1855074 ,  0.18794458,  0.18658053,
            0.18289214,  0.18271555,  0.17886536,  0.1754907 ,  0.17530537,
            0.17554246,  0.176551  ,  0.17852742,  0.17937897,  0.17610967,
            0.17603894,  0.17469454,  0.17398755,  0.16990676,  0.16875049,
            0.16561802,  0.16103238,  0.16190893,  0.15431826,  0.15621685,
            0.1540649 ,  0.15446058,  0.14886674,  0.14934526,  0.14531794,
            0.14086767,  0.13302118,  0.130909  ,  0.13231699,  0.12589878,
            0.12520766,  0.12364272,  0.12678149,  0.12541687,  0.12563644,
            0.12367225,  0.11961458,  0.11753575,  0.11457406,  0.11521763,
            0.10951413,  0.10778917,  0.10618368,  0.10500098,  0.10522734,
            0.1014674 ,  0.10075733,  0.10149647,  0.0990802 ,  0.0922146 ,
            0.09129061,  0.08891309,  0.08688356,  0.08975066,  0.09243074,
            0.08898795,  0.08677819,  0.08405302,  0.08292849,  0.09056707,
            0.0854801 ,  0.08136894,  0.08001014,  0.07989648,  0.07595382,
            0.08428664,  0.08378898,  0.07687688,  0.07860657,  0.08007728,
            0.07765366,  0.07319563,  0.06971537,  0.0702564 ,  0.06463254,
            0.06282543,  0.05974185,  0.06030332,  0.05730191,  0.05212235,
            0.04856245,  0.04310636,  0.04751329,  0.05302244,  0.05495548,
            0.05745527,  0.05578924,  0.05070386,  0.0479233 ,  0.0476315 ,
            0.04309749,  0.04964759,  0.04897139,  0.04983175,  0.04356589,
            0.04789234,  0.04608209,  0.0460169 ,  0.04029922,  0.04010597,
            0.0407652 ,  0.03490091,  0.02925648,  0.03180509,  0.02881585,
            0.02582061,  0.02827276,  0.02696484,  0.02842711,  0.02558191,
            0.02428086,  0.02306208,  0.02050128,  0.01806722,  0.01278693,
            0.0176788 ,  0.0191152 ,  0.01787593,  0.01834687,  0.01841293,
            0.01376319,  0.00652493,  0.01255874,  0.01330454,  0.01032936,
            0.01313848,  0.00733156,  0.00658589,  0.00998827,  0.00835415,
           -0.00110856, -0.0021876 , -0.00568358, -0.00188489,  0.00253799,
           -0.00126885, -0.00078138, -0.00285779, -0.00519585, -0.01138555,
           -0.00848157, -0.01450625, -0.01898871, -0.01613543, -0.0146546 ,
           -0.0195459 , -0.01750759, -0.01926611, -0.02541278, -0.0203882 ,
           -0.02587041, -0.02658505, -0.02522565, -0.02424355, -0.02252687,
           -0.02490626, -0.02260802, -0.02403949, -0.02408988, -0.02381971,
           -0.02148802, -0.021402  , -0.02201095, -0.02435495, -0.02721515,
           -0.02490534, -0.02774955, -0.02720702, -0.02641881, -0.02316105,
           -0.02274458, -0.02842236, -0.03233271, -0.0394427 , -0.04099024,
           -0.03785051, -0.03909017, -0.04149313, -0.04697105, -0.0463004 ,
           -0.04720536, -0.04585204, -0.04276497, -0.04004796, -0.03789014,
           -0.03186516, -0.03329261, -0.02998385, -0.02595826, -0.02783651,
           -0.03386348, -0.03391589, -0.03552656, -0.03830259, -0.03613567,
           -0.03745589, -0.0387135 , -0.04100259, -0.04077108, -0.0375336 ,
           -0.03915414, -0.04031228, -0.03635456, -0.03290539, -0.03181193,
           -0.028389  , -0.03385074, -0.03279201, -0.03074351, -0.02614201,
           -0.02649585, -0.02841488, -0.0238449 , -0.01774448, -0.02007678,
           -0.01792792, -0.01666406, -0.01800303, -0.01705686, -0.01501684,
           -0.01824582, -0.01465171, -0.01636821, -0.02001313, -0.0175117 ,
           -0.01367984, -0.01126773, -0.01016219, -0.01149251, -0.01334598,
           -0.01415237, -0.01021113, -0.00814578, -0.00993926, -0.00636601,
           -0.00688765, -0.01159418, -0.01280447, -0.01251801, -0.01722609,
           -0.01449339, -0.01323374, -0.0133919 , -0.01486156, -0.01911234,
           -0.01812874, -0.01856015, -0.01817129, -0.01774305, -0.02162996,
           -0.01916081, -0.01887151, -0.02007126, -0.01430661, -0.01920838,
           -0.01909881, -0.01394853, -0.02502767, -0.02728302, -0.02526814,
           -0.02744027, -0.02626973, -0.02722012, -0.02414736, -0.02023339,
           -0.02116521, -0.01940755, -0.01845206, -0.01943693, -0.02232121,
           -0.02100902, -0.02224004, -0.02065265, -0.0203368 , -0.02412222,
           -0.02116285, -0.01611854, -0.01729875, -0.02457968, -0.02787306,
           -0.03190188, -0.02962559, -0.0273987 , -0.02835397, -0.02603986,
           -0.02302227, -0.02335789, -0.02332884, -0.02632845, -0.02587006,
           -0.02277362, -0.0208165 , -0.0205839 , -0.01799254, -0.02189857,
           -0.02546698, -0.02730245, -0.02557813, -0.02366735, -0.02556523,
           -0.02883467, -0.03380668, -0.03554684, -0.03939196, -0.04048866,
           -0.04094932, -0.0399243 , -0.03952841, -0.04029986, -0.04056851,
           -0.04475685, -0.04428023, -0.04393277, -0.0333322 , -0.03704142,
           -0.03370533, -0.03325954, -0.0335813 , -0.0403869 , -0.04095667,
           -0.04118823, -0.04128702, -0.0420173 , -0.03800243, -0.03903772,
           -0.0326705 , -0.03562477, -0.03710989, -0.0355948 , -0.03985166,
           -0.04512778, -0.04358603, -0.04199028, -0.04819937, -0.04831249,
           -0.04865133, -0.04755973, -0.04103855, -0.04100278, -0.04591983,
           -0.04876802, -0.04828522, -0.0486637 , -0.05235044, -0.05060763,
           -0.05264977, -0.05268868, -0.0493951 , -0.04440411, -0.04625014,
           -0.04539042, -0.04307954, -0.04699184, -0.04405541, -0.04303051,
           -0.04601083, -0.04506487, -0.04842572, -0.0419697 , -0.04021652,
           -0.03750832, -0.03892865, -0.04074865, -0.03791585, -0.03669625,
           -0.03244959, -0.03474128, -0.03992147, -0.03847631, -0.04196935,
           -0.03248648, -0.03094302, -0.03682502, -0.03685819, -0.03663127,
           -0.03791648, -0.04026207, -0.04485059, -0.04432388, -0.04290117,
           -0.04500012, -0.04602942, -0.04256268, -0.04094338, -0.04399674,
           -0.04932565, -0.05084146, -0.0520031 , -0.0524442 , -0.05907329,
           -0.05619962, -0.05540663, -0.05358858, -0.05447677, -0.05134295,
           -0.05311579, -0.05575617, -0.05819213, -0.06022676, -0.06076008,
           -0.05799857, -0.06018609, -0.06196956, -0.06792773, -0.07060192,
           -0.06982302, -0.07229388, -0.07333189, -0.07819915, -0.07900102,
           -0.07487289, -0.07380409, -0.07392418, -0.07705432, -0.07319863,
           -0.07545795, -0.07518401, -0.07326748, -0.07364026, -0.0773141 ,
           -0.07680729, -0.07291077, -0.06965122, -0.06409447, -0.06689604,
           -0.07099728, -0.07360513, -0.07248495, -0.07192919, -0.07053476,
           -0.06881653, -0.06630804, -0.06429984, -0.06002971, -0.05749204,
           -0.06046898, -0.05787931, -0.05754204, -0.05194928, -0.05099204]),
     'measures': array([[-0.376624, -0.551124, -0.405928, ..., -0.363701, -0.5272  ,
            -0.53476 ],
           [-0.222845, -0.36142 , -0.592018, ..., -0.260163, -0.376473,
            -0.37058 ],
           [-0.195356, -0.011388, -0.110627, ..., -0.086203, -0.255174,
            -0.07902 ],
           ...,
           [-0.120577, -0.061593, -0.055908, ..., -0.278589, -0.33894 ,
            -0.345644],
           [-0.804944, -0.902505, -0.999188, ..., -0.363056, -0.504445,
            -0.838333],
           [-1.172479, -1.297884, -1.166911, ..., -0.711524, -0.604764,
            -0.965764]]),
     'sigma_sqr': 0.3630598169612829}
#### Ih6RzA2dYwUQprv2TThoym5WFlfQI03Kx9uFtg==.subsim:
    {'acceptance': 0.28920212,
     'measures': array([[-1.190678, -1.033022, -0.566971, ..., -0.57469 , -0.534487,
            -0.999293],
           [-0.047099, -0.160994, -0.262252, ..., -0.420003, -0.328237,
            -0.176694],
           [-0.242714, -0.336746, -0.344393, ..., -0.504234, -0.33534 ,
            -0.215116],
           ...,
           [-0.417607, -0.466376, -0.263133, ..., -0.605538, -0.714584,
            -0.513713],
           [-0.118661, -0.295248, -0.363071, ..., -0.051491, -0.065181,
            -0.124808],
           [-0.552136, -0.320476, -0.51437 , ..., -0.417707, -0.436122,
            -0.474148]])}
#### 1VvXWf5EqjZWDOROsRdDs1VgxMSqqQxac09VnQ==.subsim:
    {'acceptance': 0.19696295900000002,
     'measures': array([[ 0.830865,  0.909163,  0.691235, ...,  0.874855,  1.045043,
             0.919248],
           [ 0.490925,  0.362562,  0.440161, ...,  0.53564 ,  0.591837,
             0.555926],
           [-0.236808, -0.145978, -0.297   , ..., -0.438276, -0.356974,
            -0.224226],
           ...,
           [ 0.254154,  0.361977,  0.21897 , ...,  0.479628,  0.278231,
             0.273965],
           [ 0.553491,  0.622326,  0.649971, ...,  0.45775 ,  0.618394,
             0.505757],
           [ 0.453933,  0.392904,  0.262733, ...,  0.722891,  0.652389,
             0.575145]])}
#### 1IAr-nE54lcFxt3LKW8r0WtrPdR71_wO1K2vaQ==.subsim:
    {'acceptance': 0.39902157999999993,
     'measures': array([[-0.237264, -0.228206, -0.095617, ..., -0.004969, -0.418314,
            -0.019365],
           [ 0.009367,  0.32429 ,  0.546356, ..., -0.378636, -0.361008,
            -0.326554],
           [-1.321377, -1.309873, -1.43549 , ..., -1.435395, -1.162379,
            -1.388419],
           ...,
           [ 0.68087 ,  0.492991,  0.600656, ...,  0.41139 ,  0.249034,
             0.460734],
           [-0.368976, -0.625195, -0.413817, ..., -0.078411, -0.256527,
            -0.459307],
           [ 0.158948,  0.133783, -0.023174, ..., -0.23027 , -0.011201,
            -0.055184]])}
#### aB_UD1nyFH6uoshhYwnzJTtEg9oFdM70qZTF_g==.subsim:
    {'DeltaE': (0.07949163941656315, 0.0006866282213381858),
     'acceptance': 0.271952192,
     'correlations': array([0.72108745, 0.71625924, 0.7064341 , 0.69955059, 0.6934843 ,
           0.68811977, 0.68259521, 0.67425948, 0.66564375, 0.66189253,
           0.65372703, 0.64747172, 0.64255475, 0.63713431, 0.63178246,
           0.6352058 , 0.62605966, 0.62059181, 0.61381693, 0.60754986,
           0.59559292, 0.58852528, 0.58710019, 0.57891591, 0.57192199,
           0.57108338, 0.56861892, 0.56010066, 0.55650172, 0.55178012,
           0.54805032, 0.54562096, 0.53960536, 0.53677955, 0.53108198,
           0.52513527, 0.52047684, 0.5156772 , 0.51541957, 0.50562663,
           0.50430537, 0.50558401, 0.49698454, 0.49229753, 0.49132166,
           0.48898893, 0.48257618, 0.47951414, 0.47447005, 0.46743959,
           0.46106951, 0.45185707, 0.44611201, 0.44321751, 0.4371585 ,
           0.43369479, 0.43086089, 0.42942431, 0.42849765, 0.42398127,
           0.42450152, 0.42003018, 0.41590505, 0.41239067, 0.41106926,
           0.40685163, 0.40728864, 0.39641575, 0.39691286, 0.39858847,
           0.39831517, 0.3967522 , 0.39410238, 0.39046917, 0.38798065,
           0.38299783, 0.38295363, 0.38404141, 0.37853106, 0.37575539,
           0.37324279, 0.37378511, 0.36643711, 0.36163507, 0.36047223,
           0.3596249 , 0.3561751 , 0.35682145, 0.36039656, 0.3551151 ,
           0.35286836, 0.3475269 , 0.34824282, 0.34221193, 0.34101235,
           0.34145227, 0.33310967, 0.33268262, 0.32871736, 0.32733103,
           0.32641867, 0.31845943, 0.31666355, 0.30754322, 0.30750945,
           0.30501078, 0.30540426, 0.30730839, 0.31014283, 0.31087454,
           0.30939611, 0.31059361, 0.30509559, 0.30520811, 0.30474277,
           0.30028886, 0.29975593, 0.29410783, 0.2910336 , 0.28363696,
           0.28806358, 0.28449373, 0.28378491, 0.28329342, 0.28117525,
           0.27898695, 0.27673917, 0.28200006, 0.28082331, 0.27851643,
           0.28310619, 0.28775964, 0.28433873, 0.28116468, 0.28404701,
           0.28238009, 0.27875018, 0.28121149, 0.27901174, 0.27632434,
           0.27785459, 0.2800002 , 0.28286857, 0.28098912, 0.28079948,
           0.27784737, 0.27760985, 0.27766229, 0.28075988, 0.27739256]),
     'measures': array([[-0.082514, -0.319223, -0.201963, ..., -0.059565, -0.082116,
            -0.1421  ],
           [ 0.253884,  0.416152,  0.347803, ...,  0.366758,  0.173061,
             0.068946],
           [-0.226445, -0.129391, -0.1404  , ..., -0.092204, -0.068069,
            -0.142839],
           ...,
           [-0.131847,  0.035608, -0.044987, ..., -0.203273, -0.241817,
            -0.348674],
           [-0.232806, -0.014194,  0.147916, ..., -0.527032, -0.366811,
            -0.201545],
           [-0.518066, -0.463726, -0.294469, ..., -0.441591, -0.244555,
            -0.409718]]),
     'meff': (6.330194927899876, 0.054678586527707816),
     'sigma_sqr': 0.22617418672273304}
#### 22cULKqv4EaiHkejZ6HGZRFqZjxrXiYDzmoTag==.subsim:
    {'acceptance': 0.157879259,
     'measures': array([[ 0.108178,  0.02311 ,  0.016787, ...,  0.069606,  0.147747,
             0.185221],
           [-0.311207, -0.252147, -0.286152, ..., -0.164795, -0.179806,
            -0.251196],
           [-0.314146, -0.194466, -0.074602, ..., -0.545335, -0.365151,
            -0.360488],
           ...,
           [-0.396273, -0.504042, -0.394289, ..., -0.486178, -0.561558,
            -0.458843],
           [-0.32937 , -0.374478, -0.168407, ..., -0.475789, -0.507117,
            -0.505647],
           [-0.400251, -0.49295 , -0.54858 , ..., -0.572522, -0.4949  ,
            -0.492579]])}
#### 2oUjzvSu1ROMzHgeAA-Dy9wmirzguQyFOEW6BQ==.subsim:
    {'acceptance': 0.42450228,
     'measures': array([[-0.270379, -0.129694, -0.047044, ..., -0.002849, -0.241032,
            -0.410959],
           [ 0.202598, -0.100211,  0.012427, ...,  0.730879,  0.530587,
             0.511438],
           [-0.14771 , -0.370656, -0.219599, ..., -0.420131,  0.173863,
            -0.008458],
           ...,
           [ 1.636877,  1.750746,  1.774908, ...,  2.426594,  2.260687,
             1.825755],
           [ 1.378368,  1.507751,  1.510583, ...,  1.60733 ,  1.603673,
             1.5598  ],
           [ 1.38303 ,  1.13636 ,  1.638137, ...,  1.780254,  1.600572,
             1.36422 ]])}
#### GYvO9eXJIyUotCdI-V33ziPBG7D-HSdyrbR7nQ==.subsim:
    {'acceptance': 0.33928246000000006,
     'measures': array([[-0.698883, -0.648483, -0.562355, ..., -0.500931, -0.439712,
            -0.438003],
           [-1.766852, -1.520547, -1.369757, ..., -1.90024 , -1.781799,
            -1.852919],
           [-2.27801 , -2.04109 , -2.149347, ..., -1.448913, -1.737352,
            -1.821391],
           ...,
           [-0.104182, -0.044909, -0.15662 , ..., -0.051467, -0.013019,
            -0.0752  ],
           [ 0.218834,  0.106809,  0.033864, ...,  0.535905,  0.67939 ,
             0.414691],
           [-0.245908, -0.333346, -0.279746, ..., -0.144867, -0.191022,
            -0.161354]])}
#### jo1E_JcRbbXmEhKTWw7ma3_xl0n60vV0WupFCg==.subsim:
    {'acceptance': 0.5193780800000001,
     'measures': array([[-0.013144, -0.108482, -0.334771, ..., -0.004665, -0.038345,
             0.247808],
           [-0.267167, -0.41968 , -0.596636, ...,  0.201292,  0.278856,
            -0.008125],
           [ 0.606731,  0.777711,  0.686679, ...,  0.492763,  0.470589,
             0.995076],
           ...,
           [-0.201896, -0.070763,  0.309604, ...,  0.622854,  1.011499,
             0.469233],
           [ 2.133114,  2.197954,  2.290295, ...,  1.993763,  2.144367,
             2.000471],
           [ 3.192577,  2.912604,  2.557467, ...,  3.363616,  3.25249 ,
             2.700644]])}
#### CFMyrSucNxzxRydEJ9K_1KLc9SDDFF44Ba7Cqg==.subsim:
    {'acceptance': 0.160285867,
     'measures': array([[-0.085703, -0.071891, -0.12553 , ..., -0.159895, -0.16731 ,
            -0.067546],
           [-0.324679, -0.261747, -0.248221, ..., -0.310183, -0.305256,
            -0.262875],
           [-0.17787 , -0.164226, -0.18217 , ..., -0.140984, -0.153667,
            -0.141272],
           ...,
           [-1.346206, -1.270177, -1.334148, ..., -1.374296, -1.28828 ,
            -1.342647],
           [-1.32998 , -1.452681, -1.506921, ..., -1.28914 , -1.266695,
            -1.361158],
           [-0.889996, -1.049556, -0.98415 , ..., -1.006345, -0.920317,
            -0.810467]])}
#### KzT1pgvCBt6KXpBWvwGYlRpYqyfH_3DAeS2yRg==.subsim:
    {'DeltaE': (0.21041347497650068, 0.0015144999261357194),
     'acceptance': 0.339264064,
     'correlations': array([0.82681558, 0.81438037, 0.7976709 , 0.78505817, 0.76892635,
           0.76068421, 0.75158295, 0.74039279, 0.72736218, 0.71021086,
           0.6920801 , 0.67991615, 0.6680645 , 0.65789596, 0.64555862,
           0.6328939 , 0.6201611 , 0.60436252, 0.59029018, 0.57981226,
           0.56753721, 0.55910414, 0.55564752, 0.54663979, 0.5305765 ,
           0.51784965, 0.51013156, 0.49782985, 0.49117099, 0.48534369,
           0.47327275, 0.46554911, 0.45975219, 0.44899297, 0.43885394,
           0.43368832, 0.42256671, 0.4161632 , 0.40563529, 0.39484902,
           0.3910916 , 0.38470387, 0.37380304, 0.36576429, 0.36212205,
           0.35529387, 0.34506941, 0.33951425, 0.33010343, 0.31877225,
           0.31864023, 0.31222561, 0.30697824, 0.30132346, 0.29449485,
           0.29470194, 0.28869736, 0.27551579, 0.27055889, 0.26207031,
           0.26074855, 0.25550085, 0.2471136 , 0.24095508, 0.23051067,
           0.2226077 , 0.21963256, 0.21806107, 0.21729464, 0.21402368,
           0.2087542 , 0.20916012, 0.20583248, 0.19733871, 0.19672728,
           0.19407786, 0.18974881, 0.18161501, 0.1838396 , 0.17655581,
           0.17085834, 0.16624815, 0.15435303, 0.14495609, 0.13936285,
           0.13483208, 0.13557068, 0.12945854, 0.13221013, 0.130237  ,
           0.12857323, 0.12477457, 0.12323594, 0.12012963, 0.11342568,
           0.11208448, 0.11442143, 0.10827207, 0.10299953, 0.0928442 ,
           0.09309371, 0.09877707, 0.09390915, 0.09864646, 0.09721273,
           0.10152588, 0.09938537, 0.09014041, 0.08878327, 0.09164025,
           0.08555826, 0.07531495, 0.07209403, 0.07529516, 0.07433737,
           0.07515031, 0.07068546, 0.06570133, 0.06192259, 0.06344238,
           0.05729415, 0.05869436, 0.05659673, 0.05102431, 0.05710891,
           0.05255767, 0.0462844 , 0.04515972, 0.04616389, 0.0509218 ,
           0.05321761, 0.05240076, 0.05474443, 0.05010638, 0.05158891,
           0.04380315, 0.04826863, 0.04967946, 0.04829085, 0.04826587,
           0.04720538, 0.04241019, 0.04037156, 0.03926124, 0.04303936,
           0.05454637, 0.05022631, 0.05038425, 0.0552074 , 0.05295724]),
     'measures': array([[-9.880000e-04, -5.188800e-02, -2.812320e-01, ..., -2.472320e-01,
            -3.985930e-01, -1.048930e-01],
           [-1.003505e+00, -8.638850e-01, -8.729680e-01, ..., -1.151457e+00,
            -8.824650e-01, -1.091194e+00],
           [-1.444269e+00, -1.435821e+00, -1.387127e+00, ..., -7.512340e-01,
            -9.289900e-01, -1.015543e+00],
           ...,
           [-1.099282e+00, -1.130649e+00, -9.439190e-01, ..., -7.602870e-01,
            -9.036040e-01, -1.243617e+00],
           [-7.823730e-01, -5.599480e-01, -4.788710e-01, ..., -3.656110e-01,
            -4.112310e-01, -6.810930e-01],
           [ 2.366900e-01,  2.589920e-01, -1.174740e-01, ..., -2.293070e-01,
            -7.058000e-02,  2.246150e-01]]),
     'meff': (0.903468247312591, 0.006502922847378008),
     'sigma_sqr': 0.5231319181949297}
#### Y7iKiUrLdTKg5Lepj6qw2RTwmxgvJPN2cV-_XA==.subsim:
    {'acceptance': 0.4602256599999999,
     'correlations': array([ 1.84325901e+00,  1.81244965e+00,  1.77475843e+00,  1.74661191e+00,
            1.70728993e+00,  1.68268102e+00,  1.63897002e+00,  1.61401908e+00,
            1.60184794e+00,  1.58267870e+00,  1.53954155e+00,  1.52668004e+00,
            1.49063770e+00,  1.47148626e+00,  1.45222453e+00,  1.43091079e+00,
            1.40310536e+00,  1.36325093e+00,  1.32982401e+00,  1.30310395e+00,
            1.27815295e+00,  1.25979288e+00,  1.24174048e+00,  1.22676972e+00,
            1.20496179e+00,  1.18578291e+00,  1.17084532e+00,  1.14568207e+00,
            1.13224317e+00,  1.11872946e+00,  1.09533508e+00,  1.07836506e+00,
            1.07280131e+00,  1.04991912e+00,  1.02018140e+00,  1.01209255e+00,
            9.78379825e-01,  9.56741591e-01,  9.43612285e-01,  9.24113143e-01,
            9.11249487e-01,  8.86821292e-01,  8.73851262e-01,  8.54027215e-01,
            8.45142638e-01,  8.42932559e-01,  8.15534860e-01,  8.00429724e-01,
            7.76322890e-01,  7.55846469e-01,  7.40641752e-01,  7.34163025e-01,
            7.23463152e-01,  7.00286758e-01,  6.80199713e-01,  6.60561407e-01,
            6.49716093e-01,  6.23411055e-01,  6.05441369e-01,  6.04988701e-01,
            5.94807012e-01,  5.71742396e-01,  5.63289433e-01,  5.52173770e-01,
            5.44947799e-01,  5.35514936e-01,  5.12930115e-01,  4.89752095e-01,
            4.83048920e-01,  4.71765168e-01,  4.82570949e-01,  4.58911404e-01,
            4.34390599e-01,  4.27506485e-01,  4.24627557e-01,  4.00659301e-01,
            3.85061805e-01,  3.78280201e-01,  3.58241172e-01,  3.52809010e-01,
            3.52257315e-01,  3.59352372e-01,  3.45795497e-01,  3.45816832e-01,
            3.36765425e-01,  3.44383231e-01,  3.55756010e-01,  3.52517174e-01,
            3.25269605e-01,  3.25219685e-01,  3.31791130e-01,  3.31900102e-01,
            3.21568048e-01,  3.07641075e-01,  2.93523698e-01,  3.05700215e-01,
            3.06411418e-01,  2.98269988e-01,  2.87766019e-01,  2.75478647e-01,
            2.90230708e-01,  2.66922263e-01,  2.76102713e-01,  2.75877201e-01,
            2.69916347e-01,  2.65998866e-01,  2.43997365e-01,  2.20917980e-01,
            2.18337155e-01,  2.05702257e-01,  1.86250503e-01,  1.92074550e-01,
            1.94944363e-01,  1.82005962e-01,  1.79144432e-01,  1.70814391e-01,
            1.68719847e-01,  1.79443099e-01,  1.73493612e-01,  1.62403840e-01,
            1.49643003e-01,  1.38325884e-01,  1.43136264e-01,  1.55889613e-01,
            1.59721582e-01,  1.44572375e-01,  1.34019992e-01,  1.25647148e-01,
            1.33504562e-01,  1.45982879e-01,  1.24638973e-01,  1.32171447e-01,
            1.21366588e-01,  1.09705674e-01,  8.29021107e-02,  8.12508621e-02,
            7.45109708e-02,  8.30997530e-02,  9.92166633e-02,  1.01974715e-01,
            1.04619190e-01,  1.04792493e-01,  8.41565484e-02,  5.54459679e-02,
            5.74033504e-02,  5.84678555e-02,  5.34869487e-02,  3.94529503e-02,
            3.36170226e-02,  3.02744751e-02,  1.86504387e-02,  1.88949537e-02,
            1.25227000e-02,  1.97167828e-02,  2.87638729e-02,  2.89807717e-02,
            5.03713363e-03,  6.13922416e-03,  1.75658057e-02,  2.81008426e-02,
            4.28763485e-02,  4.13429154e-02,  2.64192214e-02, -1.02808642e-02,
           -1.35267510e-02, -1.02336359e-02,  9.28265611e-03,  8.65072511e-03,
           -1.90264946e-02, -5.60077800e-03, -3.94419422e-03, -4.72250319e-03,
            2.24010851e-03,  5.08088669e-03, -1.06044114e-03, -1.13835510e-02,
           -1.61344691e-03, -8.76870789e-03, -4.68544805e-03, -8.51819223e-03,
            3.78690495e-03,  1.01310798e-03, -1.10273843e-02, -2.66719261e-02,
           -2.66174722e-02, -1.91794930e-02, -1.77987048e-02, -1.22732570e-02,
           -2.87408932e-03,  5.00208848e-03,  1.41029011e-04, -1.23400923e-02,
            3.93544465e-03, -3.07673487e-03, -7.38269415e-03,  1.06024422e-02,
           -1.12889011e-02, -3.11589108e-02, -2.46308901e-02, -1.63007960e-02,
           -6.05996016e-03, -6.13368740e-03, -1.80248309e-03,  2.91912078e-04,
            8.19548882e-03,  1.48537311e-02,  1.23399592e-02,  1.72343799e-02,
            2.25794042e-02,  1.37546042e-02,  9.82511623e-03, -1.13472090e-02,
           -1.33947400e-02, -6.28570178e-03,  7.68759114e-03,  9.30367925e-03,
           -1.00050097e-02, -2.18627027e-02, -2.62311226e-02, -2.27958186e-02,
           -3.55405295e-02, -3.00946093e-02, -2.29389701e-02, -3.26328550e-02,
           -3.78508992e-02, -4.33216479e-02, -4.18863398e-02, -4.48717972e-02,
           -2.63195384e-02, -3.74209761e-02, -2.94746070e-02, -2.64273105e-02,
           -1.10816490e-02, -3.77474459e-03, -1.78362189e-02, -7.24966284e-03,
            8.13975004e-03,  1.38008254e-02,  1.13943041e-02,  8.48299965e-03,
            1.54332580e-02,  1.62474365e-02,  2.88412796e-02,  4.83607164e-02,
            5.07690306e-02,  6.58048139e-02,  3.98363158e-02,  2.98132792e-02,
            3.16944987e-02,  3.97755018e-02,  4.93088349e-02,  4.86961925e-02,
            6.26346331e-02,  7.80612033e-02,  8.05966203e-02,  9.05486001e-02,
            1.01148122e-01,  7.26389725e-02,  7.14309091e-02,  7.02327718e-02,
            7.73212619e-02,  8.51092027e-02,  7.86408557e-02,  8.39702024e-02,
            8.01293147e-02,  9.28468339e-02,  1.00712381e-01,  9.82948312e-02,
            9.99151692e-02,  9.43339620e-02,  1.08114764e-01,  1.08387100e-01,
            1.09124740e-01,  7.79195846e-02,  7.77888460e-02,  6.57877576e-02,
            7.19729828e-02,  9.04838609e-02,  8.89063519e-02,  9.58252090e-02,
            8.01311030e-02,  7.91397938e-02,  9.28645433e-02,  1.12924513e-01,
            1.14619160e-01,  1.16327381e-01,  1.15546812e-01,  1.21260517e-01,
            1.22307416e-01,  1.17940126e-01,  1.24044117e-01,  1.24244363e-01,
            1.31130515e-01,  1.14165896e-01,  1.02899665e-01,  1.04976905e-01,
            9.92048838e-02,  1.03538153e-01,  9.45150263e-02,  9.51722525e-02,
            1.03803468e-01,  1.06136922e-01,  9.42928434e-02,  9.48872112e-02,
            8.17672520e-02,  7.02302713e-02,  5.22834154e-02,  4.67867909e-02,
            3.53865028e-02,  3.83126643e-02,  4.53199920e-02,  2.84574255e-02,
            1.99878432e-02,  2.94575962e-02,  6.41660759e-03,  8.44496745e-03,
            2.01948942e-02,  1.00103335e-02,  2.85764363e-02,  2.97546761e-02,
            1.70355390e-02, -5.73156155e-03, -2.74909037e-02, -2.38585467e-02,
           -9.75948589e-03, -2.91191894e-02, -9.27427907e-03, -5.80000287e-03,
           -1.92764976e-02, -2.81281496e-02, -3.96100213e-02, -3.24675556e-02,
           -3.98266487e-02, -3.36331293e-02, -1.56909755e-02, -3.17121402e-02,
           -5.00106363e-02, -4.21418511e-02, -4.21428297e-02, -4.43799502e-02,
           -5.06580624e-02, -5.42358691e-02, -7.97162513e-02, -8.37237733e-02,
           -7.50650745e-02, -7.27177658e-02, -8.02348683e-02, -9.33708949e-02,
           -8.68848821e-02, -8.86452255e-02, -9.43118359e-02, -1.08415033e-01,
           -9.52187929e-02, -8.96240408e-02, -8.31939505e-02, -8.67839336e-02,
           -8.04196669e-02, -7.31213798e-02, -6.99853141e-02, -7.56050839e-02,
           -7.53532170e-02, -6.35801060e-02, -5.66960737e-02, -6.34658621e-02,
           -6.43629775e-02, -5.01428640e-02, -3.24321124e-02, -2.54032469e-02,
           -3.97341932e-02, -4.24999449e-02, -6.33838030e-02, -5.87489654e-02,
           -4.24400339e-02, -4.65533989e-02, -4.17190386e-02, -1.73082888e-02,
           -2.50022929e-02, -3.80836221e-02, -3.76185444e-02, -4.65665872e-02,
           -5.31350120e-02, -6.11555019e-02, -5.72122853e-02, -5.89228000e-02,
           -6.01420232e-02, -5.97539194e-02, -4.89070307e-02, -2.93181301e-02,
           -5.72765752e-04,  5.11537871e-03, -3.11584268e-03, -1.55832620e-03,
            6.22431754e-03,  1.29689548e-02,  1.43296481e-02,  1.61999772e-02,
            2.65708898e-03, -1.15619262e-03, -1.10443208e-02, -7.96889903e-03,
           -3.88571077e-03, -1.56374328e-02, -6.35590668e-03, -8.19483010e-03,
           -9.67655079e-03, -1.51347525e-02, -1.05392012e-02, -2.14652910e-02,
           -3.41236073e-02, -4.92494229e-02, -5.20777696e-02, -4.37068160e-02,
           -2.73550241e-02, -1.67617968e-02, -2.04270268e-02, -5.71051383e-03,
           -2.96272837e-02, -3.90474616e-02, -4.36549327e-02, -3.88338134e-02,
           -3.87663908e-02, -3.41969015e-02, -3.61090699e-02, -2.31035432e-02,
           -3.98424316e-02, -4.24531346e-02, -3.83628545e-02, -3.33593755e-02,
           -6.47833013e-03,  4.21562160e-03,  7.29531555e-03,  2.23645407e-03,
           -1.16604161e-03, -1.87869207e-02, -2.50796373e-02, -4.16296990e-02,
           -4.44259752e-02, -5.24375151e-02, -3.66505594e-02, -3.66786357e-02,
           -5.11366722e-02, -4.06365320e-02, -4.05253792e-02, -3.23992563e-02,
           -1.87218787e-02, -1.99718406e-02, -1.54381741e-02, -3.53679670e-04,
            2.82145091e-03,  1.55374046e-02,  2.85424152e-02,  1.86952936e-02,
            3.61312829e-02,  4.36869702e-02,  3.09420339e-02,  4.21716488e-02,
            4.70930653e-02,  6.38785548e-02,  4.31054600e-02,  3.92219206e-02,
            1.27850810e-02,  1.50244836e-02,  1.49547378e-02,  2.78992049e-02,
            3.02266786e-02,  3.66982313e-02,  4.13928421e-02,  4.18054222e-02,
            3.22069275e-02,  2.57743478e-02,  2.91799174e-02,  4.47749281e-02,
            6.95991419e-02,  6.08263990e-02,  6.59284570e-02,  5.89299746e-02,
            4.80295274e-02,  4.05130643e-02,  2.82676516e-02,  1.71477302e-02,
            4.93362069e-03,  1.60199053e-03,  3.08430258e-03,  3.77152995e-03,
           -1.18588549e-02, -1.55399796e-02, -6.64905347e-03, -6.19844182e-03,
           -1.61686037e-02, -1.62698646e-02, -2.18085956e-02, -2.54267160e-02,
           -1.33511152e-02, -1.30918909e-02, -1.36092289e-02, -9.73133328e-03,
            5.68210290e-03, -7.85878286e-03, -1.76434984e-02, -1.28029186e-02]),
     'measures': array([[ 0.515914,  0.210093,  0.254681, ...,  0.97309 ,  0.91571 ,
             0.852935],
           [ 1.228248,  1.19645 ,  1.743176, ...,  1.154892,  0.820924,
             0.7498  ],
           [ 0.091737,  0.514194,  0.109674, ...,  0.299621,  0.154324,
             0.455658],
           ...,
           [-0.186023, -0.685625, -0.762098, ..., -0.862839, -0.924798,
            -0.501765],
           [-1.404445, -1.539304, -1.485894, ..., -1.76734 , -1.496353,
            -1.679597],
           [ 0.817454,  0.789761,  0.667297, ...,  0.49104 ,  0.739221,
             0.700877]]),
     'sigma_sqr': 1.5328975971440673}
#### 9p1Rn2NN1flUNETGrUZoTPfyJ5UohJF4fDiEew==.subsim:
    {'DeltaE': (0.1370505719077218, 0.0012343567489787554),
     'acceptance': 0.519290358,
     'correlations': array([2.7372944 , 2.6958621 , 2.65683582, 2.62055011, 2.56568539,
           2.50971873, 2.4667879 , 2.43572798, 2.41641013, 2.36950692,
           2.33386269, 2.27395807, 2.23080733, 2.21738713, 2.16804751,
           2.14589434, 2.09606825, 2.07781219, 2.05128413, 2.02799143,
           1.98837247, 1.95780667, 1.91385007, 1.86609177, 1.85515145,
           1.81424241, 1.77333375, 1.74207839, 1.71840548, 1.70300177,
           1.6629143 , 1.64416905, 1.61687441, 1.5900326 , 1.55245312,
           1.53460215, 1.52863251, 1.52179598, 1.50740839, 1.49189794,
           1.46216075, 1.43337697, 1.41611638, 1.4025912 , 1.39137258,
           1.3599862 , 1.34521447, 1.32541719, 1.31512572, 1.30768183,
           1.28203454, 1.27283325, 1.26244856, 1.23760469, 1.19158893,
           1.19683726, 1.16373568, 1.16006668, 1.13796239, 1.11894978,
           1.07097791, 1.05927351, 1.06590995, 1.04611054, 1.04340282,
           1.02966592, 1.01871514, 1.00932961, 0.97734708, 0.95432628,
           0.93600997, 0.93315032, 0.93031487, 0.90530315, 0.89251492,
           0.88403186, 0.87174646, 0.87546002, 0.86857185, 0.87404671,
           0.87780934, 0.87989255, 0.85984904, 0.89034395, 0.85803533,
           0.84790719, 0.83857486, 0.82098611, 0.79924794, 0.79602519,
           0.78681177, 0.77205162, 0.77775487, 0.77112419, 0.75607044,
           0.73769418, 0.72983405, 0.71343019, 0.6956124 , 0.69244378,
           0.68845551, 0.67526737, 0.66123014, 0.65049328, 0.65103426,
           0.62054   , 0.60907406, 0.58628056, 0.5912754 , 0.59624818,
           0.59277055, 0.58992979, 0.56691354, 0.55199205, 0.53342418,
           0.53277564, 0.529027  , 0.50342808, 0.51638118, 0.50165908,
           0.49380132, 0.47453303, 0.47061515, 0.48349994, 0.46806853,
           0.4528664 , 0.459169  , 0.47178313, 0.47656445, 0.47091039,
           0.47013781, 0.4605925 , 0.47288281, 0.46291544, 0.45227683,
           0.46861258, 0.46931577, 0.45764635, 0.4528139 , 0.45403561,
           0.46814244, 0.44161702, 0.44760909, 0.43101375, 0.42721166,
           0.43700364, 0.43416969, 0.43305859, 0.43423113, 0.4340264 ]),
     'measures': array([[-0.272973, -0.178632, -0.267445, ...,  0.1148  ,  0.329776,
             0.35532 ],
           [-0.715778, -0.837137, -0.439909, ..., -0.714218, -0.302171,
            -0.623916],
           [ 0.166422,  0.43093 , -0.475409, ...,  0.521787,  0.367328,
             0.406621],
           ...,
           [-0.361632, -0.302105, -0.458806, ..., -0.675671, -0.335707,
            -0.213193],
           [-1.004253, -1.055728, -1.22281 , ..., -0.857818, -0.711228,
            -1.04216 ],
           [-1.847939, -1.374792, -1.128428, ..., -1.228105, -1.29007 ,
            -2.09325 ]]),
     'meff': (2.1296012202299, 0.019180420790904137),
     'sigma_sqr': 1.5444103041423618}
#### Q-ryBS1NROHVNd_dTZqGuiUTGQt-72mB3iwNaQ==.subsim:
    {'DeltaE': (0.1574048061986116, 0.002273505354381435),
     'acceptance': 0.363701869,
     'correlations': array([1.10156795, 1.07538683, 1.05511739, 1.0332587 , 1.01549464,
           0.99491732, 0.97639073, 0.96116833, 0.93977384, 0.91922804,
           0.90743772, 0.89276414, 0.871185  , 0.84969502, 0.838686  ,
           0.82478359, 0.815345  , 0.79682294, 0.78993362, 0.7730662 ,
           0.7588175 , 0.74027441, 0.7190722 , 0.71549967, 0.70740938,
           0.68534954, 0.66660653, 0.65503181, 0.64369099, 0.63018304,
           0.61545047, 0.6046677 , 0.58707319, 0.57542503, 0.5654156 ,
           0.56264751, 0.55061631, 0.54397581, 0.54176748, 0.52154511,
           0.51079012, 0.50656706, 0.4951747 , 0.48012723, 0.47417474,
           0.46247238, 0.45750455, 0.45185226, 0.44267541, 0.43398026,
           0.42362587, 0.42701077, 0.4212402 , 0.41447743, 0.40968433,
           0.39846218, 0.39532197, 0.38792197, 0.38302113, 0.37845607,
           0.36671642, 0.36494943, 0.35459478, 0.35224119, 0.34279712,
           0.3378936 , 0.3322005 , 0.33320557, 0.33350324, 0.33562491,
           0.33621613, 0.32508621, 0.31925024, 0.31514365, 0.30410169,
           0.3061433 , 0.29838649, 0.29121808, 0.28203548, 0.27432269,
           0.2674976 , 0.26661498, 0.2597257 , 0.2588619 , 0.26112671,
           0.26412406, 0.26279862, 0.26010533, 0.26122928, 0.26169151,
           0.25866564, 0.2566052 , 0.26257264, 0.25750029, 0.25767375,
           0.25535969, 0.25896537, 0.2674945 , 0.26559271, 0.26226992,
           0.26545406, 0.26616761, 0.26702607, 0.26342387, 0.26505648,
           0.26681116, 0.26997934, 0.26561451, 0.26092813, 0.26179956,
           0.2634592 , 0.25696638, 0.25810318, 0.2578361 , 0.25288778,
           0.25241598, 0.25034471, 0.24525734, 0.24172361, 0.23498909,
           0.23542576, 0.23815015, 0.23832219, 0.23606852, 0.22986042,
           0.22836714, 0.22542768, 0.23153866, 0.2304198 , 0.23098153,
           0.23670933, 0.22462046, 0.22586228, 0.23430019, 0.23342127,
           0.22357522, 0.22302597, 0.21912246, 0.22423797, 0.21750885,
           0.21963166, 0.22072125, 0.22527863, 0.21986909, 0.2267957 ,
           0.22546588, 0.22370868, 0.22808256, 0.23137614, 0.23376679]),
     'measures': array([[-0.808841, -0.645446, -0.071682, ..., -0.751154, -0.560733,
            -0.68382 ],
           [ 0.236559, -0.017646, -0.222064, ..., -0.254312, -0.018803,
             0.049567],
           [ 0.533658,  0.495081,  0.426471, ...,  0.112584,  0.29708 ,
             0.407751],
           ...,
           [-0.075134, -0.358863, -0.291477, ...,  0.152672,  0.03448 ,
            -0.142256],
           [-0.002928, -0.214739, -0.307948, ..., -0.623462, -0.374066,
            -0.176015],
           [-0.894117, -0.834023, -0.659336, ..., -0.276482, -0.448441,
            -0.485474]]),
     'meff': (1.6144478217815754, 0.023318574926857807),
     'sigma_sqr': 0.6483102869328174}
#### zToBoOZuhz8VsY11EwETdlMnLO-VrKsAfDXcAg==.subsim:
    {'DeltaE': (0.1498525048320794, 0.0008483929129996965),
     'acceptance': 0.29431626,
     'correlations': array([0.54060023, 0.53150861, 0.52346641, 0.51716479, 0.50962227,
           0.50060988, 0.49401901, 0.48659409, 0.47833722, 0.46978789,
           0.45965009, 0.46238576, 0.45609399, 0.4466304 , 0.44302852,
           0.43908139, 0.43207453, 0.4268751 , 0.4280797 , 0.42544203,
           0.41529658, 0.41079438, 0.40486278, 0.39958928, 0.39411084,
           0.38828171, 0.38227871, 0.38196061, 0.37438836, 0.36638625,
           0.36028348, 0.35374728, 0.34985801, 0.34524191, 0.33901633,
           0.33507241, 0.32905298, 0.32748555, 0.32303367, 0.32085613,
           0.31385358, 0.31242376, 0.30276774, 0.29474117, 0.29461103,
           0.28797925, 0.28394439, 0.27964403, 0.27354821, 0.26924103,
           0.2636917 , 0.25878345, 0.25226407, 0.24765308, 0.2451965 ,
           0.24172156, 0.23578962, 0.23641768, 0.22812429, 0.22538907,
           0.21764238, 0.21195462, 0.21082631, 0.20968313, 0.20687761,
           0.20551382, 0.20167949, 0.20004432, 0.19499652, 0.18843395,
           0.18777093, 0.18811056, 0.18357538, 0.18248152, 0.17430803,
           0.17376789, 0.1709819 , 0.16925229, 0.16954619, 0.17011283,
           0.1682324 , 0.16738358, 0.16381052, 0.16235866, 0.15893493,
           0.15488476, 0.15337494, 0.1542437 , 0.15401824, 0.15414372,
           0.149443  , 0.14390018, 0.14187397, 0.13867865, 0.13753035,
           0.13543068, 0.13161596, 0.12577613, 0.12152093, 0.12003761,
           0.11827455, 0.11425302, 0.11375097, 0.11280445, 0.10899374,
           0.10667301, 0.10435924, 0.09539471, 0.09347075, 0.08999696,
           0.08437705, 0.08391907, 0.07785616, 0.07950681, 0.07711709,
           0.0743538 , 0.07364002, 0.07380749, 0.06982989, 0.06868773,
           0.06789215, 0.06498011, 0.05637949, 0.05997121, 0.06074481,
           0.06115428, 0.0621124 , 0.05968905, 0.06444006, 0.06412011,
           0.05966261, 0.05623939, 0.05925961, 0.06120167, 0.06059561,
           0.06107836, 0.06016065, 0.06555883, 0.0683903 , 0.06862065,
           0.07123628, 0.07149209, 0.07394637, 0.0760335 , 0.07455981,
           0.07490563, 0.0776444 , 0.07472088, 0.0743186 , 0.07252095]),
     'measures': array([[-0.449366, -0.503012, -0.235994, ..., -0.489794, -0.712158,
            -0.530097],
           [ 0.264676,  0.380875,  0.239899, ...,  0.049343, -0.129546,
            -0.121425],
           [-0.349202, -0.475208, -0.317287, ..., -0.207411, -0.181783,
            -0.324079],
           ...,
           [ 0.97571 ,  0.699044,  0.649754, ...,  0.941327,  1.112887,
             0.934348],
           [ 0.587599,  0.444996,  0.544527, ...,  0.487025,  0.613672,
             0.861202],
           [-0.251662, -0.207007, -0.109156, ..., -0.039988, -0.342405,
            -0.483739]]),
     'meff': (1.7812791230041543, 0.01008474690312635),
     'sigma_sqr': 0.3185398154456739}
#### QnjOgzFMvwRDU_ROSIiqRtV7KXhQ6YZqmkoESw==.subsim:
    {'acceptance': 0.46029047999999995,
     'measures': array([[-0.731168, -0.111158, -0.215918, ..., -1.018439, -0.931327,
            -0.991909],
           [-0.880772, -0.849404, -0.864708, ..., -0.718322, -1.04202 ,
            -0.89141 ],
           [-0.887771, -0.942442, -0.918341, ..., -1.245479, -0.782683,
            -0.762544],
           ...,
           [-2.185359, -1.866578, -1.159471, ..., -2.109857, -1.817566,
            -2.142705],
           [-2.019086, -1.834264, -1.038255, ..., -1.380957, -1.292903,
            -1.76496 ],
           [-0.449081, -0.561395, -0.807305, ..., -0.196335, -0.361798,
            -0.538201]])}
#### Wmah2tqvsNdapX7RpQmG8dBM1gh7uurVxWzgyg==.subsim:
    {'acceptance': 0.15562409600000002,
     'measures': array([[ 0.003736, -0.113339, -0.186406, ..., -0.008871, -0.05309 ,
             0.105036],
           [-0.323887, -0.270353, -0.352889, ..., -0.486598, -0.396085,
            -0.408453],
           [-0.283126, -0.41765 , -0.227837, ..., -0.268429, -0.189571,
            -0.224081],
           ...,
           [-0.139361, -0.038233, -0.086831, ..., -0.195052, -0.297541,
            -0.136368],
           [-0.341635, -0.300424, -0.271805, ..., -0.254804, -0.249346,
            -0.32006 ],
           [-0.258747, -0.253316, -0.08971 , ..., -0.174195, -0.181633,
            -0.225039]])}
#### dVOtxvQM1d5Etm22rxYwdiBiOmJBLLaYrwMikw==.subsim:
    {'acceptance': 0.31310682,
     'measures': array([[-0.164589, -0.226805, -0.134676, ..., -0.249793, -0.091289,
            -0.187749],
           [-1.162589, -1.277952, -1.234108, ..., -1.451027, -1.206172,
            -1.39104 ],
           [-0.200516, -0.512441, -1.001009, ...,  0.171524,  0.064599,
            -0.024028],
           ...,
           [ 0.934014,  0.90196 ,  0.670833, ...,  0.698187,  0.600443,
             0.794752],
           [ 0.20557 ,  0.668342,  0.535343, ...,  0.530618,  0.229829,
             0.313429],
           [-0.182462, -0.109845, -0.248831, ..., -0.431036, -0.226686,
            -0.264426]])}
#### nfIqhbp4UASjhJDGZk-ymSCnhBOFFQ18tpwQIA==.subsim:
    {'acceptance': 0.320852116,
     'correlations': array([ 6.16163934e-01,  6.03421608e-01,  5.90191986e-01,  5.77173013e-01,
            5.66498487e-01,  5.61014419e-01,  5.52696945e-01,  5.41175865e-01,
            5.33192491e-01,  5.24749865e-01,  5.18935280e-01,  5.13562507e-01,
            5.09098484e-01,  4.95946628e-01,  4.82757332e-01,  4.81629508e-01,
            4.75837961e-01,  4.62065037e-01,  4.60410965e-01,  4.47245045e-01,
            4.42238089e-01,  4.40824702e-01,  4.30102099e-01,  4.23869136e-01,
            4.10502840e-01,  4.07526505e-01,  3.97714544e-01,  3.99958730e-01,
            3.94671298e-01,  3.84190628e-01,  3.79447082e-01,  3.72577027e-01,
            3.67652346e-01,  3.65238954e-01,  3.65436276e-01,  3.60124880e-01,
            3.53328780e-01,  3.44226763e-01,  3.37481727e-01,  3.27387422e-01,
            3.17306671e-01,  3.09781477e-01,  3.03740853e-01,  3.05183687e-01,
            3.13218473e-01,  3.15945236e-01,  3.07149806e-01,  3.07013472e-01,
            3.08918520e-01,  3.08031736e-01,  3.04365131e-01,  2.88533222e-01,
            2.76236738e-01,  2.78734164e-01,  2.73670662e-01,  2.62288999e-01,
            2.64788488e-01,  2.57565076e-01,  2.54708108e-01,  2.40141756e-01,
            2.41147862e-01,  2.38551386e-01,  2.38357220e-01,  2.36335195e-01,
            2.32572182e-01,  2.23189527e-01,  2.19963961e-01,  2.20583265e-01,
            2.14156203e-01,  2.14582539e-01,  2.11286391e-01,  2.10489200e-01,
            2.04599727e-01,  2.01769338e-01,  2.02362395e-01,  1.97056486e-01,
            1.92999368e-01,  1.93714240e-01,  1.99912723e-01,  1.94600223e-01,
            1.95027069e-01,  1.89480132e-01,  1.99509699e-01,  2.01721941e-01,
            1.97388161e-01,  1.93848028e-01,  1.84943366e-01,  1.77196469e-01,
            1.75865326e-01,  1.72898959e-01,  1.70068355e-01,  1.68930865e-01,
            1.65911794e-01,  1.68316329e-01,  1.74667706e-01,  1.83579058e-01,
            1.80819872e-01,  1.76092590e-01,  1.74925308e-01,  1.71667997e-01,
            1.66350886e-01,  1.68150008e-01,  1.64192846e-01,  1.64415774e-01,
            1.58122287e-01,  1.60624261e-01,  1.60801414e-01,  1.57849625e-01,
            1.51226844e-01,  1.50584360e-01,  1.54057406e-01,  1.51661229e-01,
            1.49004002e-01,  1.48010759e-01,  1.41473614e-01,  1.38946011e-01,
            1.38200996e-01,  1.35852777e-01,  1.33534068e-01,  1.24585221e-01,
            1.25367120e-01,  1.22046733e-01,  1.21397619e-01,  1.26525655e-01,
            1.16023488e-01,  1.18496697e-01,  1.19002923e-01,  1.20776639e-01,
            1.10306825e-01,  1.05907546e-01,  1.06326852e-01,  1.02938499e-01,
            1.00376151e-01,  1.00481944e-01,  9.90175454e-02,  9.84054197e-02,
            9.78251787e-02,  9.41339545e-02,  9.99948838e-02,  9.02723320e-02,
            9.55162961e-02,  9.41530649e-02,  9.54693178e-02,  9.37832954e-02,
            9.33888992e-02,  8.59877558e-02,  8.69128730e-02,  8.59341050e-02,
            9.11137465e-02,  9.05099269e-02,  8.05146465e-02,  8.40179636e-02,
            8.18267121e-02,  7.73435743e-02,  7.42802148e-02,  7.22101486e-02,
            6.95744385e-02,  7.33725103e-02,  7.73831430e-02,  7.68861266e-02,
            7.59290605e-02,  7.78686243e-02,  7.63633728e-02,  7.14161444e-02,
            7.13378895e-02,  7.26770247e-02,  6.73346674e-02,  7.85677818e-02,
            7.65539423e-02,  7.78110553e-02,  8.46137579e-02,  7.96474611e-02,
            7.47874948e-02,  6.52808024e-02,  6.34357877e-02,  6.54970813e-02,
            6.26683651e-02,  6.90950947e-02,  6.29453678e-02,  7.43356534e-02,
            7.38766410e-02,  7.26458855e-02,  7.41603569e-02,  8.26241378e-02,
            7.86084231e-02,  7.74088731e-02,  7.61924815e-02,  7.50272067e-02,
            7.47102182e-02,  6.73623914e-02,  5.81794444e-02,  5.64292672e-02,
            5.83849221e-02,  5.10033832e-02,  5.07767485e-02,  4.79698902e-02,
            4.28823039e-02,  4.20196225e-02,  3.33513438e-02,  3.30442039e-02,
            3.57822703e-02,  4.22803840e-02,  5.19315917e-02,  5.12883292e-02,
            4.83865377e-02,  4.75973289e-02,  3.98050877e-02,  3.98987736e-02,
            4.11841950e-02,  4.36428584e-02,  3.62002985e-02,  3.21975067e-02,
            2.90469101e-02,  3.01389746e-02,  3.33955178e-02,  3.96864344e-02,
            3.95005132e-02,  3.46887425e-02,  3.05199272e-02,  2.71519155e-02,
            2.60997031e-02,  2.49342730e-02,  2.56039908e-02,  2.58717096e-02,
            2.65995092e-02,  1.92156162e-02,  1.20303056e-02,  9.67705367e-03,
            4.71538381e-03,  5.56721136e-03,  7.34239023e-03,  5.99939531e-03,
            4.55263944e-03, -1.23500824e-04,  6.77037179e-03,  1.99033429e-03,
            5.50452673e-03,  4.66373888e-03,  8.23511507e-03,  1.05564402e-02,
            1.20140723e-02,  8.03935422e-03,  1.09703045e-02,  1.11307287e-02,
            1.37871092e-02,  1.54043365e-02,  1.65251780e-02,  2.18975251e-02,
            1.77006095e-02,  2.15106041e-02,  1.99618735e-02,  2.67423652e-02,
            1.99992737e-02,  1.58313081e-02,  1.36110193e-02,  1.85109823e-02,
            2.05318300e-02,  2.33886140e-02,  1.89605825e-02,  2.25469244e-02,
            2.72445480e-02,  3.33715924e-02,  3.72922456e-02,  3.46763992e-02,
            3.74800002e-02,  4.12134957e-02,  3.46502046e-02,  3.65184788e-02,
            3.34070336e-02,  2.81972551e-02,  2.62786348e-02,  2.45342108e-02,
            2.87159507e-02,  2.80470286e-02,  2.90005657e-02,  2.94852522e-02,
            2.75543856e-02,  3.09575856e-02,  2.99846741e-02,  2.88562870e-02,
            2.89763651e-02,  3.11906645e-02,  3.19873139e-02,  3.08680278e-02,
            3.19573309e-02,  2.54374541e-02,  2.79796870e-02,  3.23637675e-02,
            2.19016229e-02,  2.05664610e-02,  1.81168934e-02,  2.05061117e-02,
            2.20176429e-02,  1.71277248e-02,  1.13384042e-02,  1.63255888e-02,
            8.68217891e-03,  4.50040962e-03,  3.51796188e-03,  1.03005728e-02,
            6.06583311e-03,  4.16593671e-03,  1.32737556e-03, -1.00141151e-04,
            6.43942174e-03,  5.96263042e-03,  7.60646233e-03,  9.64657895e-03,
            3.06827759e-03,  2.05580954e-03,  5.66922412e-05,  6.31772652e-03,
            1.30402917e-02,  1.04662409e-02,  9.81889680e-03,  6.33661773e-03,
            1.41592439e-02,  1.17175353e-02,  1.49555195e-02,  8.82245595e-03,
            6.26921360e-03,  6.70047761e-03,  1.07531194e-02,  7.22033846e-03,
            1.26229201e-02,  9.90304678e-03,  1.10247501e-02,  7.11180966e-03,
            3.04090967e-03,  2.36903184e-03,  7.20676786e-04,  2.15854187e-03,
           -3.12672380e-05,  4.01023075e-03, -3.77030896e-03, -4.11894128e-03,
           -7.86152776e-03, -3.01083694e-03,  7.57393135e-04,  2.93703152e-03,
            1.32964941e-03, -4.28762869e-03, -2.97469282e-03,  4.43659759e-04,
            3.61755766e-03, -3.58911343e-03, -3.38269055e-03, -5.28463632e-03,
           -1.67712152e-03,  4.63101581e-05,  3.80878066e-04,  4.56772623e-03,
           -1.38059734e-03, -2.94591134e-03, -6.45550548e-03, -4.23021880e-03,
           -8.15941377e-04, -5.96291618e-03, -7.40169066e-03, -7.39413277e-03,
           -5.14630395e-04, -2.48957034e-03, -1.81613485e-03,  6.00434638e-03,
            1.16023323e-02,  1.77988252e-02,  2.00623852e-02,  2.11566770e-02,
            1.57407635e-02,  2.66093324e-02,  2.23746618e-02,  2.20577301e-02,
            1.66460798e-02,  2.02151241e-02,  1.56823434e-02,  2.02907199e-02,
            2.47610222e-02,  2.10915404e-02,  2.26205377e-02,  2.30065047e-02,
            2.38262110e-02,  2.14906336e-02,  2.83897454e-02,  3.21789107e-02,
            2.90976426e-02,  3.23903560e-02,  3.18516253e-02,  2.79326964e-02,
            2.95645554e-02,  2.79424465e-02,  3.52165174e-02,  3.39802749e-02,
            3.33146565e-02,  3.08410646e-02,  3.35540682e-02,  3.33119963e-02,
            3.17902305e-02,  2.99310720e-02,  3.54363680e-02,  3.66249483e-02,
            3.51631433e-02,  4.04637015e-02,  3.75336529e-02,  3.44494488e-02,
            3.22846999e-02,  2.82048637e-02,  2.38519157e-02,  2.66264263e-02,
            2.48111089e-02,  3.02456142e-02,  3.08382441e-02,  2.92075644e-02,
            2.93913133e-02,  2.33892237e-02,  2.11390978e-02,  2.52790838e-02,
            1.50199676e-02,  1.45344948e-02,  2.32623650e-02,  2.20513512e-02,
            2.19350983e-02,  2.60512776e-02,  3.31703922e-02,  2.98867474e-02,
            3.43505660e-02,  3.65428993e-02,  4.44988130e-02,  4.36687387e-02,
            4.44003840e-02,  3.74217113e-02,  3.51044500e-02,  3.41999499e-02,
            3.43438143e-02,  3.18016362e-02,  3.29188656e-02,  3.62548208e-02,
            3.85429199e-02,  3.60251295e-02,  2.74288908e-02,  2.13003368e-02,
            1.44526286e-02,  1.61370998e-02,  1.57581701e-02,  1.43495210e-02,
            1.84651505e-02,  1.59309267e-02,  1.32107534e-02,  1.71683233e-02,
            1.30799398e-02,  1.93871675e-02,  2.45419436e-02,  2.23128866e-02,
            2.09273179e-02,  2.67208458e-02,  2.26099883e-02,  2.49347208e-02,
            2.49196728e-02,  2.81215662e-02,  3.22433161e-02,  3.43771634e-02,
            3.67260404e-02,  2.67663518e-02,  3.67623918e-02,  2.82972520e-02,
            2.37598582e-02,  2.49226226e-02,  2.46590138e-02,  2.15300183e-02,
            1.74193423e-02,  2.30943736e-02,  2.98179338e-02,  2.79388323e-02,
            3.39040465e-02,  3.00665620e-02,  2.58264033e-02,  1.86432552e-02,
            2.51682545e-02,  2.57875138e-02,  4.13667582e-02,  4.56437879e-02,
            5.17933981e-02,  5.37860527e-02,  5.96101751e-02,  5.65443101e-02,
            5.39665464e-02,  5.96496088e-02,  5.92126644e-02,  5.23129567e-02,
            5.41613505e-02,  5.02258463e-02,  5.74248823e-02,  5.58617877e-02,
            5.41922027e-02,  5.17786723e-02,  5.69631441e-02,  5.44486152e-02,
            5.52967895e-02,  5.38031635e-02,  4.94453964e-02,  5.78616940e-02]),
     'measures': array([[-0.383932, -0.318436, -0.43454 , ..., -0.201726, -0.168509,
            -0.292773],
           [-0.441177, -0.278107, -0.162269, ..., -0.073096, -0.065381,
            -0.386872],
           [-0.789783, -0.896859, -0.861677, ..., -0.338407, -0.142469,
            -0.463665],
           ...,
           [-0.183211, -0.464982, -0.643822, ..., -0.242109, -0.237291,
            -0.217401],
           [-1.329767, -1.556152, -1.70864 , ..., -1.261795, -1.163888,
            -1.046725],
           [-2.338898, -2.402801, -2.439264, ..., -2.36383 , -2.651056,
            -2.582455]]),
     'sigma_sqr': 0.6059279065227784}
#### Qws0atPclBtOah_kYvUPIkovvnSprF9aE5O5tg==.subsim:
    {'DeltaE': (0.14327650918087204, 0.0018886876620288057),
     'acceptance': 0.28443236,
     'correlations': array([0.61303948, 0.60302867, 0.59090955, 0.58558367, 0.5766173 ,
           0.56802772, 0.56169133, 0.56159063, 0.55286212, 0.54475112,
           0.53523501, 0.53116059, 0.52514135, 0.51959305, 0.51315381,
           0.50795171, 0.50033431, 0.49951499, 0.49633714, 0.48828885,
           0.48422499, 0.47398047, 0.46941873, 0.4612625 , 0.45181665,
           0.44713025, 0.44306389, 0.43993755, 0.43562719, 0.4290507 ,
           0.41817601, 0.4140564 , 0.41128128, 0.40717827, 0.40042311,
           0.39250085, 0.39113718, 0.38295428, 0.38041123, 0.37463968,
           0.36597303, 0.36426997, 0.36049457, 0.35583789, 0.35172999,
           0.34462274, 0.34088922, 0.33839693, 0.33256979, 0.32870769,
           0.32445235, 0.32297562, 0.31697302, 0.30972395, 0.31182255,
           0.30694203, 0.30123561, 0.29777763, 0.29117578, 0.28629558,
           0.28393559, 0.27611029, 0.27489629, 0.27296753, 0.27270941,
           0.27252654, 0.26654991, 0.25926836, 0.26021362, 0.2524105 ,
           0.25083469, 0.24925329, 0.24572893, 0.24172764, 0.2376798 ,
           0.234749  , 0.23056098, 0.22707937, 0.21950319, 0.2160712 ,
           0.2188441 , 0.21238288, 0.20529409, 0.20192231, 0.19898444,
           0.19414005, 0.18802802, 0.18267571, 0.17351817, 0.16930299,
           0.16560945, 0.15967336, 0.16012469, 0.15428813, 0.1492105 ,
           0.1473437 , 0.14095532, 0.13778495, 0.13357281, 0.13086933,
           0.12951625, 0.12483269, 0.12603034, 0.12724118, 0.12460625,
           0.12337058, 0.12227972, 0.11819785, 0.11813577, 0.11819807,
           0.11801178, 0.11723983, 0.11341394, 0.11657968, 0.11655676,
           0.11530024, 0.11024137, 0.10640042, 0.10263048, 0.10109291,
           0.09592558, 0.09487289, 0.09696592, 0.09866116, 0.09312956,
           0.09106801, 0.09711468, 0.09923231, 0.09952233, 0.09632253,
           0.09105903, 0.09109416, 0.09072345, 0.09219169, 0.09254148,
           0.09243512, 0.09006432, 0.09026293, 0.08750415, 0.0906368 ,
           0.08750075, 0.08947855, 0.08689975, 0.0910441 , 0.08647752,
           0.08589543, 0.08683157, 0.08259851, 0.08700347, 0.0876678 ]),
     'measures': array([[-0.322828, -0.388049, -0.34535 , ..., -0.009519, -0.157689,
            -0.303405],
           [-0.140936,  0.019572, -0.11982 , ..., -0.111525, -0.159131,
            -0.209627],
           [-0.187123, -0.325212, -0.057396, ..., -0.116847, -0.212296,
            -0.336791],
           ...,
           [ 0.784056,  0.694041,  0.729055, ...,  0.657439,  0.715434,
             0.632103],
           [ 0.960449,  0.775153,  0.63104 , ...,  0.975821,  1.055496,
             1.04107 ],
           [ 0.720836,  0.725164,  0.866555, ...,  0.680691,  0.730154,
             0.56888 ]]),
     'meff': (1.9485430615918298, 0.025685922000754913),
     'sigma_sqr': 0.2861706467366646}
#### S6gStPg-XnsF_oPMyoFQB2b-APOaToOmqdwWrQ==.subsim:
    {'acceptance': 0.379509808,
     'correlations': array([ 1.01227503e+00,  9.92252390e-01,  9.71093310e-01,  9.43486424e-01,
            9.17920603e-01,  9.00452239e-01,  8.88524417e-01,  8.69701209e-01,
            8.53880157e-01,  8.37793225e-01,  7.98940969e-01,  7.82792030e-01,
            7.62806378e-01,  7.40170549e-01,  7.21438502e-01,  7.16161965e-01,
            7.20074527e-01,  7.06039103e-01,  6.91467544e-01,  6.76696288e-01,
            6.60863151e-01,  6.56066151e-01,  6.48204536e-01,  6.36166437e-01,
            6.32181968e-01,  6.14145895e-01,  5.96695994e-01,  5.85834076e-01,
            5.78623153e-01,  5.75316183e-01,  5.60429761e-01,  5.40237929e-01,
            5.30050505e-01,  5.28328897e-01,  5.06503567e-01,  5.00654297e-01,
            4.93905047e-01,  4.86448452e-01,  4.77963823e-01,  4.67476222e-01,
            4.64775887e-01,  4.68033372e-01,  4.52639190e-01,  4.46389441e-01,
            4.31115768e-01,  4.20387733e-01,  4.14616182e-01,  4.11115707e-01,
            4.03110370e-01,  3.91272186e-01,  3.80077195e-01,  3.66124589e-01,
            3.60970892e-01,  3.58718736e-01,  3.56993714e-01,  3.51932406e-01,
            3.43740954e-01,  3.38325117e-01,  3.41754529e-01,  3.33059466e-01,
            3.33860200e-01,  3.27908086e-01,  3.28093149e-01,  3.13442605e-01,
            3.02398726e-01,  2.99085682e-01,  2.97876701e-01,  2.82884523e-01,
            2.76984591e-01,  2.83804419e-01,  2.82369034e-01,  2.80464207e-01,
            2.74365630e-01,  2.75024592e-01,  2.67638109e-01,  2.60147735e-01,
            2.60682377e-01,  2.56731024e-01,  2.53897117e-01,  2.51064358e-01,
            2.43227448e-01,  2.44112213e-01,  2.45189822e-01,  2.42909402e-01,
            2.39337560e-01,  2.34351144e-01,  2.45382769e-01,  2.38084210e-01,
            2.30128487e-01,  2.22529588e-01,  2.13002929e-01,  2.11439702e-01,
            2.05423750e-01,  1.92833063e-01,  1.76910760e-01,  1.72099977e-01,
            1.56528870e-01,  1.55292948e-01,  1.54084950e-01,  1.54717961e-01,
            1.51668244e-01,  1.44522659e-01,  1.35744269e-01,  1.35621918e-01,
            1.27689220e-01,  1.18960349e-01,  1.07648108e-01,  1.05809510e-01,
            9.70804116e-02,  9.29321430e-02,  1.00679045e-01,  9.20169914e-02,
            6.89463726e-02,  7.34393273e-02,  8.42272689e-02,  8.95091524e-02,
            8.96005142e-02,  9.04332812e-02,  8.99469389e-02,  7.53910909e-02,
            6.73829100e-02,  5.53800263e-02,  5.87223149e-02,  6.32928837e-02,
            6.56593916e-02,  5.02635723e-02,  5.61956624e-02,  5.09691914e-02,
            5.93533638e-02,  6.72468596e-02,  7.31521720e-02,  7.73631492e-02,
            8.48767879e-02,  8.04581888e-02,  7.82757711e-02,  7.15374932e-02,
            7.82319048e-02,  7.92787830e-02,  6.93443697e-02,  8.31635415e-02,
            8.75932428e-02,  7.74637415e-02,  7.40596385e-02,  6.29365415e-02,
            6.50168858e-02,  6.97927378e-02,  7.89635435e-02,  8.10838452e-02,
            7.65905225e-02,  6.41453443e-02,  7.11247633e-02,  7.19587666e-02,
            7.47778323e-02,  6.48528715e-02,  5.78685265e-02,  5.08247606e-02,
            5.42459184e-02,  4.68363037e-02,  4.59357210e-02,  5.48448723e-02,
            4.76748440e-02,  4.74725656e-02,  5.37609177e-02,  4.18561935e-02,
            4.15134042e-02,  4.30091500e-02,  4.53478024e-02,  5.10003861e-02,
            4.15240690e-02,  4.64889796e-02,  4.68360821e-02,  4.09728948e-02,
            4.01137384e-02,  4.88069108e-02,  4.10648370e-02,  5.55183845e-02,
            5.47896198e-02,  4.57913859e-02,  4.49148712e-02,  3.97063956e-02,
            2.77633047e-02,  3.90563701e-02,  3.97043164e-02,  3.32658501e-02,
            2.85828843e-02,  2.59164223e-02,  2.50477077e-02,  1.76769688e-02,
            1.43303814e-02,  1.71049151e-02,  8.48745507e-03,  8.81002700e-03,
            5.04057793e-03,  5.05808401e-03,  2.09646343e-03, -4.33977448e-03,
           -9.69284285e-03, -4.46032453e-03, -6.17093873e-03, -1.52975794e-02,
           -1.51329557e-02, -2.15673392e-02, -2.77253660e-02, -2.85991008e-02,
           -1.95999412e-02, -1.67588797e-02, -1.92230835e-02, -2.21171542e-02,
           -2.52971054e-02, -2.78869269e-02, -2.75552839e-02, -1.95533697e-02,
           -3.08082181e-02, -3.98142978e-02, -1.68023852e-02, -1.39252574e-02,
           -8.00813929e-03, -1.11681921e-02, -5.83519437e-03, -1.16042841e-03,
            8.75203058e-03,  1.30684930e-02,  2.30547401e-03,  5.56325300e-03,
            8.08266421e-03,  6.82121895e-03,  1.48910209e-02,  8.59827883e-03,
            1.31215252e-02,  1.43031468e-02,  2.04374842e-02,  2.95845888e-02,
            3.12391986e-02,  3.87857123e-02,  4.80177501e-02,  4.51431074e-02,
            4.85439706e-02,  4.46350571e-02,  4.23223035e-02,  4.49455886e-02,
            4.69375248e-02,  4.54775685e-02,  3.92493053e-02,  4.80412921e-02,
            5.36717099e-02,  3.92139687e-02,  3.94566501e-02,  3.60170543e-02,
            3.55158280e-02,  2.99098753e-02,  3.38247927e-02,  4.19694903e-02,
            3.86319205e-02,  3.40206047e-02,  3.08581252e-02,  2.81328904e-02,
            2.80977985e-02,  2.04429671e-02,  1.64365301e-02,  1.98314438e-02,
            1.15155206e-02,  1.29574554e-02,  1.24541153e-02,  1.03396287e-02,
            1.34593857e-02,  8.84749239e-03,  1.18226880e-02,  7.43029560e-03,
            7.17255852e-04,  1.04397409e-02,  3.22366481e-03,  5.29941403e-03,
            1.49798573e-02,  1.20284535e-02,  2.12589291e-03,  8.55829214e-04,
           -5.50457155e-03,  5.89073194e-03, -1.27906535e-03, -7.92556281e-03,
           -2.42556840e-02, -1.66335467e-02, -1.84332304e-03, -2.25333783e-03,
           -1.46959982e-03, -3.65665848e-03, -5.05599894e-05, -1.29926466e-02,
           -1.03808947e-02, -6.86003791e-03,  6.62999668e-03,  2.46180506e-03,
            4.25840302e-03,  7.41937508e-03,  2.39725205e-02,  9.32766798e-03,
            1.05873576e-02,  2.10017243e-02,  2.00753131e-02,  2.53936647e-02,
            2.59423127e-02,  2.57203088e-02,  2.54893274e-02,  1.70312829e-02,
            2.99326122e-02,  3.90728213e-02,  2.21638461e-02,  2.19666096e-02,
            2.48768741e-02,  2.13853520e-02,  1.66905735e-02,  2.62977109e-02,
            2.88853070e-02,  2.63471435e-02,  3.15052883e-02,  2.55490364e-02,
            2.02885961e-02,  1.67098249e-02,  1.59090938e-02,  1.36815376e-02,
            1.17209913e-02,  7.40302780e-03,  6.08557918e-03,  7.92412121e-03,
            8.17110453e-03,  1.37221843e-02,  1.08840226e-02,  8.12949051e-03,
            7.80765608e-03,  1.16011729e-02,  1.67034403e-02,  1.10822183e-02,
            7.37139205e-03,  1.13778102e-02, -9.97855498e-05, -4.86060206e-03,
            6.22123293e-03,  1.65109129e-02,  1.83848492e-02,  2.39141511e-02,
            1.90098275e-02,  3.40573083e-02,  2.47873646e-02,  3.80635091e-02,
            5.07463690e-02,  5.62376104e-02,  5.39441444e-02,  6.30727757e-02,
            5.44802464e-02,  6.39062283e-02,  6.27571401e-02,  5.76929599e-02,
            6.11016891e-02,  6.02054172e-02,  6.31170637e-02,  6.31503687e-02,
            6.10840272e-02,  6.31759541e-02,  7.48712128e-02,  7.85408249e-02,
            8.78431811e-02,  8.76597062e-02,  9.09613030e-02,  9.52195805e-02,
            9.72560765e-02,  9.32730331e-02,  9.69593733e-02,  9.52270022e-02,
            7.46637121e-02,  6.52954866e-02,  7.03953735e-02,  7.71698077e-02,
            7.52275964e-02,  7.56090275e-02,  6.47288310e-02,  5.87041900e-02,
            6.01485673e-02,  6.11851522e-02,  6.47056518e-02,  6.14896999e-02,
            6.45478012e-02,  6.13041402e-02,  5.00611253e-02,  4.33812629e-02,
            4.07044466e-02,  4.38874231e-02,  4.09566754e-02,  4.91356066e-02,
            5.40113045e-02,  5.96810028e-02,  5.52941600e-02,  5.23428568e-02,
            4.35481027e-02,  4.14859602e-02,  3.93877885e-02,  3.70609372e-02,
            2.51278352e-02,  1.66543990e-02,  5.82085147e-03, -4.39185743e-03,
           -8.90431376e-03, -4.62970832e-05, -4.20393200e-03, -3.21101318e-03,
            5.99393047e-03,  5.21801389e-03,  1.00138719e-02,  1.02159516e-02,
            6.29885361e-03,  1.83611590e-03, -1.43568313e-03, -3.85338025e-03,
           -1.52476372e-02, -1.72829612e-03, -1.10350610e-02, -2.14384755e-02,
           -2.09642084e-02, -2.70955694e-02, -3.72143806e-02, -3.88095045e-02,
           -3.88392171e-02, -2.82196015e-02, -2.46422942e-02, -2.16803305e-02,
           -2.10443176e-02, -1.30395067e-02, -1.53112979e-02, -1.74126323e-02,
           -2.14396517e-02, -1.51160119e-02, -1.66814966e-02, -1.65959061e-02,
           -1.52260726e-02, -2.77913925e-02, -1.91703155e-02, -5.15003628e-03,
           -6.83560202e-04,  5.34411955e-03,  5.61248194e-03, -2.34622234e-03,
           -6.63068887e-03, -5.10055459e-03, -7.27886654e-03, -5.59575086e-03,
           -1.31223424e-03, -7.01630972e-03, -2.10150598e-02, -2.01501604e-02,
           -2.68078762e-02, -2.11068750e-02, -1.86092781e-02, -7.18062768e-03,
           -3.03431638e-03,  4.09580908e-03,  3.55525036e-03,  8.43313047e-03,
            2.35188972e-03, -1.19227811e-02, -1.59240194e-02, -3.47756829e-02,
           -4.31714378e-02, -4.17029823e-02, -4.36652449e-02, -4.26231197e-02,
           -4.23540846e-02, -4.03926102e-02, -3.43953344e-02, -3.25910821e-02,
           -3.87890314e-02, -3.06041938e-02, -3.96920453e-02, -3.67104920e-02,
           -4.09264781e-02, -4.10242337e-02, -4.02755489e-02, -4.36432361e-02,
           -4.26813689e-02, -4.15225268e-02, -3.07945270e-02, -3.60758703e-02,
           -1.77134625e-02, -1.84927316e-02, -1.52697099e-02, -1.65480844e-02,
           -1.66758507e-02, -1.87448825e-02, -2.13734471e-02, -2.81238933e-02,
           -1.60482563e-02, -6.81015366e-03, -2.72595506e-03,  7.36875817e-04,
            3.59942819e-03,  7.76034663e-03,  3.84345657e-03,  8.04639235e-03,
            1.40338843e-02,  2.30591499e-02,  1.85864856e-02,  2.10216245e-02]),
     'measures': array([[-5.066500e-02,  9.184900e-02,  3.896120e-01, ...,  6.263000e-01,
             9.645700e-02,  4.378900e-02],
           [ 4.630600e-02,  2.467000e-02,  1.010510e-01, ..., -7.572000e-02,
             2.498340e-01, -9.100000e-05],
           [-4.668950e-01, -2.976250e-01, -1.721450e-01, ..., -1.214390e-01,
            -2.409310e-01, -4.211720e-01],
           ...,
           [ 7.198070e-01,  4.524750e-01,  1.242860e-01, ...,  5.315950e-01,
             6.288370e-01,  5.091030e-01],
           [ 1.566659e+00,  1.611367e+00,  1.414703e+00, ...,  1.746249e+00,
             1.485873e+00,  1.476052e+00],
           [-7.122700e-02,  6.819400e-02, -3.981300e-02, ..., -4.670330e-01,
            -2.116730e-01, -8.603300e-02]]),
     'sigma_sqr': 0.9272797156725788}
#### lmxqmzEzRYmYQv1nSuYze_mBMJthH7meGjNDpA==.subsim:
    {'acceptance': 0.71230008,
     'measures': array([[-1.807491, -1.546442, -1.175476, ..., -2.065734, -2.310462,
            -1.635523],
           [-1.293208, -2.031425, -1.065232, ..., -1.469877, -1.613237,
            -1.403545],
           [-0.497983, -0.731523, -0.059092, ...,  0.641967,  0.788753,
             0.174972],
           ...,
           [ 0.330481,  0.278871,  1.984022, ...,  0.295205,  0.728393,
             0.908268],
           [ 0.905776,  0.313896,  0.215842, ...,  1.244005,  1.417913,
             1.457608],
           [-1.774176, -1.120605, -0.713899, ..., -1.588345, -2.22935 ,
            -2.416889]])}
#### jPMGJ7VoFyp41-D8z7562JkqFwzAxr35KnG5FQ==.subsim:
    {'acceptance': 0.267230719,
     'measures': array([[-0.357936, -0.324256, -0.361449, ..., -0.247492, -0.203471,
            -0.415087],
           [-0.041401, -0.150822, -0.188218, ...,  0.130936,  0.116609,
             0.046991],
           [ 0.347298,  0.32544 ,  0.308252, ...,  0.590905,  0.446185,
             0.48398 ],
           ...,
           [ 0.062298,  0.174067,  0.274955, ...,  0.534451,  0.446017,
             0.37458 ],
           [-0.029482,  0.030419,  0.113316, ...,  0.024164, -0.14408 ,
            -0.132122],
           [-0.169641, -0.156961,  0.018614, ..., -0.052975, -0.159755,
            -0.224277]])}
#### V334-FiAa-OCnaHmXYnYnAI4XCdus01cdmXODg==.subsim:
    {'acceptance': 0.523296686,
     'measures': array([[-0.921612, -0.939244, -0.90746 , ..., -1.469559, -1.302732,
            -1.374525],
           [-0.213305,  0.053617,  0.00783 , ..., -0.520092, -0.524869,
            -0.310086],
           [-0.947767, -1.057922, -0.69138 , ..., -1.946386, -1.457274,
            -1.452964],
           ...,
           [ 0.772576,  1.42199 ,  1.261585, ...,  0.672779,  0.26648 ,
             0.517023],
           [ 1.359646,  0.844839,  0.670894, ...,  0.980606,  1.268381,
             1.518814],
           [-0.184756, -0.428385, -0.805011, ..., -0.304419, -0.471405,
            -0.182413]])}
#### SUo3sMbiiUxz7C0F6KS-83SxVjst9lN3Zbn8cw==.subsim:
    {'DeltaE': (0.19980449764623176, 0.0010725581409037018),
     'acceptance': 0.424515498,
     'correlations': array([1.27772863, 1.24770042, 1.21907839, 1.19676875, 1.17209298,
           1.14762156, 1.12220283, 1.09613322, 1.07549944, 1.04491909,
           1.01704492, 0.99812722, 0.97897045, 0.96225093, 0.9519369 ,
           0.93107639, 0.91925324, 0.89829875, 0.86771823, 0.85036382,
           0.82862997, 0.81735251, 0.7966205 , 0.77148118, 0.75393012,
           0.73808088, 0.71748027, 0.70578291, 0.69615972, 0.68140599,
           0.66415957, 0.65441896, 0.63265451, 0.62490854, 0.60255384,
           0.59051663, 0.58327498, 0.56919707, 0.56568478, 0.55897403,
           0.56099699, 0.547806  , 0.52851161, 0.52065345, 0.51316241,
           0.50656241, 0.48832167, 0.47148438, 0.46223055, 0.45366804,
           0.4460791 , 0.43134144, 0.42360751, 0.41853541, 0.4163497 ,
           0.40951889, 0.40208505, 0.3823037 , 0.37687086, 0.37191816,
           0.36836874, 0.35005014, 0.34966   , 0.33276104, 0.32834136,
           0.31630321, 0.31598512, 0.31300187, 0.30075422, 0.30664657,
           0.29547868, 0.29034183, 0.2946106 , 0.29183007, 0.28256681,
           0.28081929, 0.27615778, 0.26854254, 0.26363671, 0.2598903 ,
           0.24986077, 0.24034282, 0.23234515, 0.22513641, 0.22900568,
           0.22178184, 0.21055134, 0.20476033, 0.20312819, 0.20685447,
           0.20362494, 0.20604962, 0.20786444, 0.20855898, 0.20375189,
           0.19929505, 0.19499355, 0.18592081, 0.18267764, 0.1832736 ,
           0.1763567 , 0.17377729, 0.16947788, 0.17044838, 0.16577198,
           0.168716  , 0.16134744, 0.15474733, 0.1569505 , 0.14699741,
           0.13762546, 0.13141374, 0.13637026, 0.12645974, 0.13250012,
           0.12508272, 0.11301017, 0.11558466, 0.11761213, 0.11287099,
           0.10737228, 0.10635569, 0.10536933, 0.10048894, 0.08626671,
           0.09851048, 0.09649739, 0.09729356, 0.08864647, 0.09435561,
           0.09716535, 0.10533503, 0.107027  , 0.10237428, 0.10812579,
           0.10614521, 0.11083037, 0.11693267, 0.10793384, 0.09807957,
           0.09431752, 0.08851986, 0.09826174, 0.09736137, 0.09682711,
           0.10396749, 0.11319867, 0.10788994, 0.11167878, 0.11029034]),
     'measures': array([[-0.171995, -0.245757, -0.400639, ..., -0.279126, -0.322652,
            -0.025887],
           [-0.538667, -0.796392, -0.645699, ...,  0.018599, -0.225508,
            -0.365224],
           [-0.959049, -0.637661, -0.873381, ..., -0.804978, -0.763453,
            -0.853771],
           ...,
           [ 0.600227,  0.003372, -0.081252, ...,  0.997311,  1.028024,
             0.711238],
           [ 0.037238, -0.092397, -0.060486, ..., -0.767103, -0.810564,
            -0.404385],
           [-1.166944, -0.979979, -1.021492, ..., -1.667007, -1.522635,
            -1.362617]]),
     'meff': (1.001957893866192, 0.0053785480735857654),
     'sigma_sqr': 0.9258521638461316}
#### bBgFPFV01UBxSG6UEUzrYA3KpAfdTjAysXjnBQ==.subsim:
    {'DeltaE': (0.20173010901460367, 0.0023336244206503425),
     'acceptance': 0.32947657100000005,
     'correlations': array([0.70637565, 0.69289511, 0.6782118 , 0.66199951, 0.64055172,
           0.61850954, 0.60681712, 0.59341318, 0.57807255, 0.57117073,
           0.55737809, 0.5391685 , 0.52752191, 0.51590343, 0.50185509,
           0.48569675, 0.47625059, 0.45592103, 0.44556041, 0.43824075,
           0.42594669, 0.41995306, 0.41393634, 0.40614479, 0.39956779,
           0.38884604, 0.3790553 , 0.37156982, 0.35900294, 0.34588886,
           0.32751273, 0.31769207, 0.31382247, 0.30224723, 0.30326155,
           0.29920265, 0.29045051, 0.28094836, 0.27215528, 0.26035367,
           0.25707482, 0.25320863, 0.25225863, 0.2463146 , 0.2416091 ,
           0.22998787, 0.22657219, 0.225921  , 0.22652798, 0.22408301,
           0.22190103, 0.22000134, 0.22290432, 0.21929325, 0.21035433,
           0.20624115, 0.19673435, 0.19511275, 0.18476454, 0.17313864,
           0.17239896, 0.17256433, 0.16615715, 0.16781464, 0.17045543,
           0.16893062, 0.16233644, 0.15643876, 0.14825441, 0.14814534,
           0.14644742, 0.13686856, 0.13450113, 0.13385311, 0.13683066,
           0.13636701, 0.13812849, 0.13716752, 0.1330665 , 0.13213718,
           0.12852925, 0.12959423, 0.12547083, 0.12106172, 0.11957273,
           0.12075482, 0.11641539, 0.11838841, 0.11532039, 0.11345482,
           0.11677596, 0.11355134, 0.11047163, 0.10820892, 0.10746186,
           0.10435429, 0.09810429, 0.09802276, 0.09652451, 0.09131417,
           0.09479487, 0.09643211, 0.09296906, 0.08938661, 0.08446325,
           0.09017434, 0.09058408, 0.08714623, 0.08896934, 0.08781098,
           0.08264016, 0.07834605, 0.0681339 , 0.06273836, 0.06053518,
           0.05456094, 0.05248521, 0.05129727, 0.05307149, 0.0578257 ,
           0.05717543, 0.05601043, 0.05276296, 0.04892464, 0.05105281,
           0.04929238, 0.055425  , 0.05197548, 0.05878065, 0.05631438,
           0.05521721, 0.0590524 , 0.05900784, 0.05534643, 0.06103753,
           0.06095925, 0.05774652, 0.0553047 , 0.05425759, 0.04834847,
           0.04800343, 0.04620704, 0.04516952, 0.04395084, 0.04160123,
           0.04227123, 0.04820309, 0.04787124, 0.0554269 , 0.04993546]),
     'measures': array([[-0.205888, -0.160629, -0.390307, ...,  0.23033 ,  0.297757,
            -0.046457],
           [ 0.491009,  0.371718,  0.12854 , ...,  0.763711,  0.34631 ,
             0.139848],
           [-0.296159, -0.210584, -0.140215, ..., -0.514196, -0.309256,
            -0.416398],
           ...,
           [-0.066328, -0.173203, -0.197487, ..., -0.251891, -0.184393,
            -0.225959],
           [-0.503838, -0.417065, -0.34182 , ..., -0.357643, -0.220464,
            -0.37938 ],
           [-0.502892, -0.454363, -0.24985 , ..., -0.478944, -0.661457,
            -0.605145]]),
     'meff': (0.9829208440075533, 0.011370479579606144),
     'sigma_sqr': 0.4607592066443831}
#### xwcc9jGZcJDEfC_uCXwa2oiaR7FHcedhLVWTLA==.subsim:
    {'acceptance': 0.306297796,
     'correlations': array([ 5.80733101e-01,  5.77564388e-01,  5.70278427e-01,  5.63924417e-01,
            5.56336287e-01,  5.44035537e-01,  5.40811805e-01,  5.34812602e-01,
            5.24059142e-01,  5.08514869e-01,  5.01283117e-01,  4.98091507e-01,
            4.91598668e-01,  4.83173132e-01,  4.72818627e-01,  4.61603178e-01,
            4.54702635e-01,  4.46480108e-01,  4.41504751e-01,  4.31544070e-01,
            4.24693733e-01,  4.23144987e-01,  4.19122634e-01,  4.19386727e-01,
            4.12544374e-01,  4.08789060e-01,  4.00814397e-01,  4.00748357e-01,
            3.93365235e-01,  3.88917455e-01,  3.84591800e-01,  3.80327160e-01,
            3.70413998e-01,  3.64850979e-01,  3.57458732e-01,  3.53237581e-01,
            3.47160874e-01,  3.37031216e-01,  3.26825656e-01,  3.22419998e-01,
            3.20452509e-01,  3.13873911e-01,  3.03486984e-01,  2.97725953e-01,
            2.92749074e-01,  2.89316739e-01,  2.87824881e-01,  2.81127915e-01,
            2.82399031e-01,  2.79143476e-01,  2.79432227e-01,  2.78979699e-01,
            2.69981828e-01,  2.65201689e-01,  2.66343743e-01,  2.61090429e-01,
            2.55203246e-01,  2.55663281e-01,  2.50446941e-01,  2.41894546e-01,
            2.41443018e-01,  2.33836037e-01,  2.30029916e-01,  2.26378793e-01,
            2.20597242e-01,  2.14912837e-01,  2.13861913e-01,  2.11933639e-01,
            2.04315174e-01,  1.97737234e-01,  1.91294126e-01,  1.93821284e-01,
            1.92368430e-01,  1.83200326e-01,  1.84890351e-01,  1.81591392e-01,
            1.80593733e-01,  1.83029626e-01,  1.82218232e-01,  1.80587835e-01,
            1.78851105e-01,  1.79212359e-01,  1.71260426e-01,  1.70575387e-01,
            1.70583539e-01,  1.61004991e-01,  1.59756402e-01,  1.44084505e-01,
            1.41751851e-01,  1.43461772e-01,  1.46632165e-01,  1.44126265e-01,
            1.41617142e-01,  1.37551344e-01,  1.38260878e-01,  1.34640751e-01,
            1.27118982e-01,  1.27961368e-01,  1.23183614e-01,  1.14487030e-01,
            1.15533711e-01,  1.11823790e-01,  1.09919577e-01,  1.12802157e-01,
            1.03837643e-01,  1.01567007e-01,  9.71503007e-02,  9.50106402e-02,
            9.46833667e-02,  9.49651938e-02,  9.64287238e-02,  9.66398533e-02,
            9.29764805e-02,  8.44196022e-02,  8.48399991e-02,  8.50756984e-02,
            8.30311196e-02,  7.88896693e-02,  8.43097739e-02,  8.32043651e-02,
            7.96267645e-02,  7.72198287e-02,  7.92937126e-02,  7.92023966e-02,
            7.81189142e-02,  8.45275283e-02,  8.39176834e-02,  8.41535344e-02,
            8.20031230e-02,  7.56447164e-02,  7.40911971e-02,  6.53377778e-02,
            5.94838865e-02,  5.73052719e-02,  5.45855789e-02,  5.84252844e-02,
            5.34261576e-02,  5.26132945e-02,  5.31597221e-02,  4.52494835e-02,
            4.28541667e-02,  4.17399584e-02,  3.97653819e-02,  3.94201339e-02,
            3.89152473e-02,  3.37034241e-02,  2.73546153e-02,  3.07528397e-02,
            2.37971055e-02,  2.60808780e-02,  2.60073017e-02,  1.77093639e-02,
            1.68311112e-02,  2.02652744e-02,  2.35835869e-02,  3.08738648e-02,
            2.83736428e-02,  2.41430461e-02,  2.63269650e-02,  2.47744038e-02,
            2.36299461e-02,  2.85051710e-02,  2.57329686e-02,  2.28980909e-02,
            2.14273265e-02,  2.63210004e-02,  2.91577514e-02,  2.38602866e-02,
            2.42859929e-02,  2.42817979e-02,  2.25122955e-02,  2.60401181e-02,
            2.18504173e-02,  1.87110723e-02,  2.03477591e-02,  1.67055635e-02,
            1.52985957e-02,  1.75586149e-02,  1.58404493e-02,  2.09377514e-02,
            1.89764269e-02,  2.02944158e-02,  2.51340898e-02,  2.32065117e-02,
            1.86531446e-02,  1.77023856e-02,  1.91840892e-02,  1.56164161e-02,
            1.13581806e-02,  1.00512039e-02,  7.22991004e-03,  3.35107593e-03,
            5.24619999e-03,  1.09004292e-02,  8.40804364e-05,  5.96119458e-05,
            2.60479087e-03, -4.20398828e-03, -5.45090525e-03, -8.03131300e-03,
           -8.30301174e-03, -1.45628432e-02, -1.64141841e-02, -1.91336762e-02,
           -1.43555131e-02, -1.53569824e-02, -1.92482648e-02, -1.86436576e-02,
           -1.68869132e-02, -1.28295892e-02, -1.09109528e-02, -9.21620173e-03,
           -1.17639053e-02, -7.21805236e-03, -4.71517298e-03, -1.89499217e-03,
           -8.60556149e-03, -1.45560833e-02, -1.30492455e-02, -1.46758957e-02,
           -1.48960393e-02, -1.53849436e-02, -1.04678725e-02, -5.34718512e-03,
           -8.30945609e-03, -5.10427060e-03, -6.55077332e-03, -7.88270400e-03,
           -5.92594900e-03, -9.44726135e-03, -1.22689880e-02, -8.87096339e-03,
           -1.28659420e-02, -1.44465682e-02, -1.25456740e-02, -1.20192367e-02,
           -1.37939498e-02, -1.34442938e-02, -1.46489211e-02, -1.02115281e-02,
           -9.89801960e-03, -4.18269427e-03, -5.57001246e-03, -1.08796631e-02,
           -8.12254447e-03, -7.73384921e-03, -1.11970769e-02, -8.70190804e-03,
           -9.13013265e-03, -1.05368571e-02, -1.03162514e-02, -6.96005532e-03,
           -1.21324455e-02, -1.51964157e-02, -9.28318952e-03, -8.52568781e-03,
           -1.73471682e-02, -1.73465444e-02, -1.98626169e-02, -1.86361309e-02,
           -1.52589822e-02, -1.31296014e-02, -1.61230830e-02, -2.06088116e-02,
           -2.31494544e-02, -1.11362921e-02, -1.32340990e-02, -1.44464369e-02,
           -1.42121274e-02, -6.30380421e-03, -6.61972023e-03, -7.52382804e-03,
           -1.05851318e-02, -8.58444385e-03, -7.86266208e-03, -6.95092273e-03,
           -7.76263386e-03, -1.24733990e-03, -3.05647584e-03,  3.86940198e-04,
           -1.15634740e-03,  2.25130873e-03,  3.55252026e-03,  4.00725170e-03,
            1.16932388e-02,  1.23485693e-02,  8.33076944e-03,  1.53034424e-02,
            1.48240765e-02,  1.20189720e-02,  1.41920151e-02,  2.15575417e-02,
            2.35638956e-02,  2.16270380e-02,  2.03549150e-02,  1.84835092e-02,
            2.24389807e-02,  1.99960149e-02,  1.83503624e-02,  1.74855722e-02,
            1.87067282e-02,  2.03136877e-02,  2.62900461e-02,  2.69649096e-02,
            1.68440323e-02,  1.94028803e-02,  2.61052749e-02,  2.32105043e-02,
            2.62950354e-02,  2.12163468e-02,  2.35764074e-02,  2.18171602e-02,
            1.94512982e-02,  1.43494609e-02,  1.09715586e-02,  1.00154386e-02,
            9.83242917e-03,  1.38521268e-02,  1.60269915e-02,  1.06228241e-02,
            1.53708906e-02,  1.66652520e-02,  1.54191104e-02,  1.43725389e-02,
            1.51228147e-02,  1.02649679e-02,  1.12042524e-02,  1.65709336e-02,
            1.67347304e-02,  1.52508321e-02,  1.81456310e-02,  2.06589752e-02,
            2.37594313e-02,  2.04122519e-02,  1.82362473e-02,  2.62297161e-02,
            2.61367687e-02,  2.49851830e-02,  2.77351728e-02,  2.59106856e-02,
            2.02792540e-02,  2.00735614e-02,  2.29623855e-02,  2.62787256e-02,
            2.04071288e-02,  1.86435981e-02,  1.61513085e-02,  1.58704997e-02,
            1.44586881e-02,  1.21735854e-02,  1.39817644e-02,  1.70845244e-02,
            1.80482332e-02,  1.88498332e-02,  1.76576678e-02,  1.24573748e-02,
            1.43268416e-02,  1.31555174e-02,  1.50066756e-02,  1.26396683e-02,
            1.12152733e-02,  1.16367639e-02,  1.66138290e-02,  2.19892563e-02,
            1.56378834e-02,  2.01019581e-02,  1.93224271e-02,  1.69268559e-02,
            2.47131996e-02,  1.69765412e-02,  2.04139580e-02,  1.85092717e-02,
            1.69040951e-02,  1.54030688e-02,  1.54407738e-02,  1.70455649e-02,
            1.49414708e-02,  1.37462116e-02,  1.57875805e-02,  1.20170654e-02,
            1.96412433e-02,  1.68156426e-02,  1.90276237e-02,  2.15547351e-02,
            2.57447769e-02,  2.24338548e-02,  3.11105967e-02,  3.25993602e-02,
            3.39241020e-02,  3.57970607e-02,  3.59199617e-02,  3.49444229e-02,
            3.64575185e-02,  3.11531742e-02,  2.99218565e-02,  2.60109667e-02,
            2.60461781e-02,  2.74211441e-02,  2.84228439e-02,  2.70615707e-02,
            2.04994435e-02,  2.60038530e-02,  2.68968044e-02,  2.42416263e-02,
            2.00383190e-02,  1.38985044e-02,  1.60691061e-02,  2.28243714e-02,
            2.07869895e-02,  1.56748161e-02,  1.24731931e-02,  1.45052844e-02,
            1.21023319e-02,  1.46529161e-02,  9.47246608e-03,  2.39549345e-03,
            7.64855292e-03,  8.35968380e-03,  3.98158086e-03,  2.51996856e-03,
           -2.34441730e-04, -3.04495899e-06, -9.53338916e-04, -4.30680478e-03,
           -4.49620353e-03, -4.57680713e-03, -2.23792873e-03,  1.62821125e-03,
           -4.13478340e-03, -4.86878787e-03, -6.36011254e-03, -4.98652129e-03,
           -8.01514326e-04, -2.52894970e-03, -7.83152990e-03, -6.85946536e-03,
           -7.15359861e-03, -1.46322042e-02, -8.91917992e-03, -4.67089046e-03,
           -1.21722554e-02, -1.23144620e-02, -7.04175718e-03, -1.19657677e-02,
           -1.01282387e-02, -1.34126783e-02, -9.63132252e-03, -6.58864482e-03,
           -7.58707251e-03, -1.01945919e-02, -5.91136101e-03, -7.86641663e-03,
           -1.01141174e-02, -1.00949892e-02, -8.29941076e-03, -9.55573570e-03,
           -1.54297811e-02, -1.09953287e-02, -1.69086863e-02, -1.88069060e-02,
           -2.65714134e-02, -2.73454071e-02, -3.31726347e-02, -3.47495964e-02,
           -3.18019580e-02, -3.61185629e-02, -3.86769265e-02, -4.04058337e-02,
           -4.13470634e-02, -4.01119979e-02, -4.70333746e-02, -4.50376578e-02,
           -4.97682186e-02, -5.38350829e-02, -5.21784784e-02, -4.70590729e-02,
           -4.62277049e-02, -4.33146814e-02, -3.88117148e-02, -4.20967525e-02,
           -3.53886966e-02, -3.84171649e-02, -4.28391788e-02, -4.48486071e-02,
           -4.73023345e-02, -4.39122852e-02, -4.30201645e-02, -4.25495527e-02,
           -4.99133466e-02, -5.35386529e-02, -4.95302404e-02, -4.62031901e-02,
           -3.85794315e-02, -3.98185346e-02, -3.95579274e-02, -4.09660013e-02,
           -4.39320005e-02, -4.46828023e-02, -4.34590467e-02, -4.47102295e-02]),
     'measures': array([[ 0.052073,  0.09864 ,  0.239302, ..., -0.234688, -0.251465,
            -0.055903],
           [ 0.777005,  0.937966,  0.768279, ...,  0.767936,  0.725802,
             0.6693  ],
           [-0.299744, -0.269528, -0.356226, ..., -0.159489, -0.116275,
            -0.157874],
           ...,
           [-1.176178, -1.126797, -1.059907, ..., -1.076196, -1.416993,
            -1.163359],
           [-0.129398, -0.153079, -0.058992, ...,  0.215328, -0.132234,
            -0.361351],
           [ 0.376181,  0.169053,  0.526248, ...,  0.107906,  0.020115,
             0.166632]]),
     'sigma_sqr': 0.5201233991533204}
#### -RayrZ15mM3J6d-D9B6KqtyUHnztMJpJjA3uPg==.subsim:
    {'acceptance': 0.27194818400000004,
     'correlations': array([ 3.38629402e-01,  3.28607638e-01,  3.23436090e-01,  3.16657057e-01,
            3.08562407e-01,  3.03990252e-01,  2.99622958e-01,  3.01021420e-01,
            3.00104146e-01,  2.92352996e-01,  2.87428859e-01,  2.83950128e-01,
            2.81035340e-01,  2.75513195e-01,  2.65837723e-01,  2.61577350e-01,
            2.56504923e-01,  2.50912361e-01,  2.48132971e-01,  2.45030797e-01,
            2.41066465e-01,  2.37683015e-01,  2.32117269e-01,  2.28421739e-01,
            2.24208475e-01,  2.19657947e-01,  2.15248097e-01,  2.15993220e-01,
            2.11940243e-01,  2.07213325e-01,  2.04230345e-01,  1.98779650e-01,
            1.93526707e-01,  1.89559252e-01,  1.83525307e-01,  1.82137045e-01,
            1.77941743e-01,  1.75381339e-01,  1.71614144e-01,  1.73605875e-01,
            1.75234823e-01,  1.69553929e-01,  1.68822761e-01,  1.63986264e-01,
            1.63392945e-01,  1.62895150e-01,  1.61589758e-01,  1.58460346e-01,
            1.60956910e-01,  1.57231412e-01,  1.52538598e-01,  1.47807672e-01,
            1.44676757e-01,  1.42703564e-01,  1.41360485e-01,  1.34177704e-01,
            1.30683642e-01,  1.29083958e-01,  1.26219638e-01,  1.26836012e-01,
            1.23009067e-01,  1.26128708e-01,  1.21106421e-01,  1.20014685e-01,
            1.18900838e-01,  1.23672720e-01,  1.25287243e-01,  1.23941340e-01,
            1.23817739e-01,  1.22297796e-01,  1.20633191e-01,  1.18359111e-01,
            1.20970558e-01,  1.27759050e-01,  1.22909493e-01,  1.24320295e-01,
            1.22047391e-01,  1.24181087e-01,  1.21556946e-01,  1.26847113e-01,
            1.25497635e-01,  1.19790030e-01,  1.20444581e-01,  1.17928901e-01,
            1.15499669e-01,  1.15279485e-01,  1.13793187e-01,  1.12404194e-01,
            1.08423333e-01,  1.09265942e-01,  1.06198931e-01,  1.03926960e-01,
            1.00685782e-01,  1.01807573e-01,  1.02400425e-01,  1.03916582e-01,
            9.83077141e-02,  9.26641888e-02,  9.39153507e-02,  9.54985076e-02,
            9.53784439e-02,  9.59065709e-02,  9.16170257e-02,  9.22577295e-02,
            9.18320517e-02,  9.21835705e-02,  9.30901967e-02,  9.39328757e-02,
            9.37140511e-02,  9.28897823e-02,  9.22618339e-02,  8.71791175e-02,
            8.25155778e-02,  8.09416849e-02,  7.73931238e-02,  7.70953628e-02,
            7.98244140e-02,  7.64398593e-02,  7.58958454e-02,  8.31189789e-02,
            8.27322799e-02,  7.98775234e-02,  8.12649021e-02,  8.23802654e-02,
            7.88107315e-02,  8.30317598e-02,  8.50141580e-02,  8.63318798e-02,
            8.46084393e-02,  7.99092343e-02,  7.68665642e-02,  7.60307895e-02,
            7.98164219e-02,  7.92817892e-02,  7.91744602e-02,  7.72683532e-02,
            7.90495554e-02,  7.65844676e-02,  7.44404086e-02,  7.23583915e-02,
            7.39021562e-02,  7.69990701e-02,  7.95038229e-02,  7.27759468e-02,
            7.23877304e-02,  6.98021860e-02,  6.60011552e-02,  6.91248489e-02,
            7.16234216e-02,  7.08753732e-02,  6.90707005e-02,  6.82157044e-02,
            6.43026874e-02,  6.65127004e-02,  6.70576966e-02,  6.69379691e-02,
            6.66864652e-02,  6.35097918e-02,  6.30173977e-02,  6.20085704e-02,
            6.59872826e-02,  6.80120444e-02,  6.56515741e-02,  6.55716512e-02,
            6.33042590e-02,  6.50432741e-02,  6.44584304e-02,  6.22351020e-02,
            6.43711433e-02,  5.87228544e-02,  5.48516484e-02,  5.63705189e-02,
            5.98038234e-02,  6.32861193e-02,  6.09037830e-02,  6.15077620e-02,
            5.90512732e-02,  5.72411185e-02,  5.76013974e-02,  5.83894106e-02,
            5.88533139e-02,  6.00727149e-02,  5.89915876e-02,  5.82220515e-02,
            5.75383421e-02,  5.66404664e-02,  5.24232885e-02,  4.89830388e-02,
            4.80176514e-02,  4.83804902e-02,  5.04634098e-02,  5.22979272e-02,
            5.17849979e-02,  5.12805793e-02,  4.91803566e-02,  4.67865479e-02,
            4.20907060e-02,  4.21760897e-02,  4.00372372e-02,  3.67797006e-02,
            3.71218898e-02,  3.84756527e-02,  3.84004818e-02,  3.84273606e-02,
            3.94113708e-02,  4.03851945e-02,  3.89333364e-02,  3.84342124e-02,
            3.62237686e-02,  3.74773888e-02,  3.31548742e-02,  3.94855206e-02,
            3.81090802e-02,  3.84077001e-02,  4.01410914e-02,  4.16648670e-02,
            4.32608166e-02,  4.26199637e-02,  4.32016832e-02,  4.57330130e-02,
            4.85651626e-02,  4.52295106e-02,  4.44536673e-02,  4.46985950e-02,
            4.25561011e-02,  3.88023500e-02,  3.80295400e-02,  3.80887081e-02,
            3.64343198e-02,  3.93626576e-02,  4.03316992e-02,  4.03641179e-02,
            3.57511180e-02,  3.53532199e-02,  3.18819202e-02,  2.82701531e-02,
            2.85851428e-02,  3.23248935e-02,  2.80759289e-02,  3.19438877e-02,
            3.32388329e-02,  3.54561242e-02,  3.30984506e-02,  3.65079779e-02,
            3.67684136e-02,  3.86326143e-02,  3.75741336e-02,  3.37240523e-02,
            3.36320975e-02,  2.69495554e-02,  2.95859895e-02,  2.85626784e-02,
            2.77267383e-02,  2.68536903e-02,  2.92303088e-02,  2.90869391e-02,
            2.77060729e-02,  2.81124310e-02,  2.76126200e-02,  2.80889928e-02,
            2.19531316e-02,  2.05064640e-02,  2.41317635e-02,  2.47602979e-02,
            2.41463090e-02,  2.37373252e-02,  2.03124533e-02,  1.87246206e-02,
            2.12157800e-02,  2.26622897e-02,  2.57629864e-02,  2.74283541e-02,
            2.46561970e-02,  2.78976716e-02,  2.94159827e-02,  2.75170958e-02,
            2.59521803e-02,  2.28019703e-02,  2.49243151e-02,  2.48005840e-02,
            2.41153299e-02,  2.43646621e-02,  2.49911531e-02,  2.27016913e-02,
            2.40682222e-02,  2.83118511e-02,  2.59384376e-02,  2.53259620e-02,
            2.28969780e-02,  2.41796688e-02,  2.64354031e-02,  2.82096441e-02,
            2.58696520e-02,  2.73982877e-02,  2.66663111e-02,  2.29052405e-02,
            2.11200973e-02,  2.25684492e-02,  2.08058470e-02,  1.90122693e-02,
            1.86932350e-02,  2.40769520e-02,  2.53660954e-02,  2.73341004e-02,
            2.80662826e-02,  2.70070819e-02,  2.50764095e-02,  2.36219011e-02,
            2.25035674e-02,  2.39883190e-02,  2.13689672e-02,  2.12434743e-02,
            2.44951382e-02,  2.54587485e-02,  2.56345730e-02,  3.15691866e-02,
            2.64138410e-02,  2.73722573e-02,  2.88904544e-02,  2.54786201e-02,
            2.61776512e-02,  2.43971766e-02,  2.57434639e-02,  2.36905926e-02,
            2.01977432e-02,  1.52778403e-02,  1.30529333e-02,  1.31231382e-02,
            1.05563189e-02,  7.14708008e-03,  9.63441454e-03,  1.28194171e-02,
            9.84248672e-03,  8.33610411e-03,  4.15745584e-03,  2.35042532e-03,
            8.59602876e-03,  3.94294608e-03,  2.56807431e-03,  4.35787081e-03,
            1.40191500e-03,  1.25027552e-03,  1.49903451e-03,  3.74392656e-04,
            1.65003510e-03, -1.46052113e-03, -4.12498635e-03, -4.91875787e-03,
           -4.70283281e-03, -3.42668795e-03, -3.80926459e-03, -1.72671162e-03,
           -5.07318755e-03, -6.89221908e-03, -3.70230073e-03,  6.35785555e-04,
            1.63425775e-03, -7.81942000e-04,  1.09872055e-03, -4.82007866e-03,
            1.34058278e-04, -3.50956255e-03, -2.06606867e-04, -3.18204992e-03,
           -1.61013934e-03, -5.75440869e-03, -5.31573508e-03, -9.27268561e-03,
           -7.10325865e-03, -2.63040395e-03, -2.49530681e-03, -7.81111460e-03,
           -1.02763094e-02, -1.23877676e-02, -6.69816877e-03, -4.91554757e-03,
           -3.69279781e-03, -5.60941883e-03, -5.68533612e-03, -3.99396912e-03,
           -6.04361958e-03, -8.30209028e-03, -5.80739311e-03, -7.53550634e-03,
           -9.26793619e-03, -6.57845854e-03, -1.05452594e-02, -5.94841684e-03,
           -6.40824493e-03, -3.34260290e-03, -3.88482976e-03,  5.36017697e-05,
           -5.15318690e-03, -2.21017356e-03, -2.22843260e-03,  1.74797541e-03,
            2.35765162e-03, -2.20719220e-04,  1.67429161e-03,  2.86924991e-03,
            2.90701400e-03, -2.56981454e-03, -1.86071286e-04, -5.09824762e-05,
           -2.57007894e-03,  2.10125708e-03,  1.03985663e-03, -2.40846154e-03,
            1.48428071e-03,  1.27796986e-03,  3.61042510e-03,  5.51154550e-03,
            8.50301018e-03,  8.15492195e-03,  1.23153504e-02,  5.65912059e-03,
            5.37140778e-03,  5.15427231e-03,  1.05278233e-03,  1.73196946e-03,
            4.15405443e-03,  1.37328259e-03,  2.56017292e-03,  2.48781230e-03,
           -1.48770292e-03, -1.96873877e-03, -2.26145529e-03, -3.95049876e-03,
           -4.05351587e-03, -1.80923783e-03, -2.96632448e-03, -4.49986636e-03,
           -5.73259037e-03, -6.95853370e-03, -4.73865002e-03, -4.99117525e-03,
           -6.77311344e-03, -1.39696172e-03, -6.18816973e-03, -3.06235575e-03,
           -4.36330641e-03, -1.05030824e-03, -1.40104860e-03, -2.71780220e-05,
            3.79111490e-03,  8.22734733e-03,  6.80665054e-03,  4.47857780e-03,
            7.18678733e-03,  9.83336118e-03,  6.66604865e-03,  8.04325157e-03,
            6.04638608e-03,  6.32366554e-03,  5.20471712e-03,  7.58160179e-03,
            8.52264383e-03,  6.66810994e-03,  7.40176236e-03,  1.05310506e-02,
            9.50813118e-03,  1.08741688e-02,  9.41412238e-03,  5.93072593e-03,
            8.64398354e-03,  9.68626714e-03,  9.18017233e-03,  7.19148555e-03,
            6.09118655e-03,  7.90943040e-03,  7.68178269e-03,  5.94925378e-03,
            1.04193835e-02,  7.08789972e-03,  3.20337367e-03,  3.38664684e-03,
            2.23874121e-04,  2.79193798e-03,  1.18317202e-03, -2.20095778e-03,
           -4.47157489e-03, -6.61087714e-03, -2.25122325e-04,  1.57200511e-03,
           -6.33804792e-04,  6.45607031e-03,  6.73230007e-03,  6.83917669e-03,
            3.79024388e-03,  8.77360023e-03,  1.05130084e-02,  1.04847071e-02,
            7.29782267e-03,  7.80051742e-03,  5.83816457e-03,  4.97828990e-03,
            5.31748659e-03,  4.36494266e-03,  3.38401940e-03,  3.86426727e-03]),
     'measures': array([[-0.200658, -0.210592, -0.279425, ..., -0.14148 , -0.186671,
            -0.30425 ],
           [-1.015889, -1.061337, -1.05938 , ..., -1.292455, -1.282702,
            -1.268771],
           [-1.379237, -1.265322, -1.20234 , ..., -1.130219, -1.074288,
            -1.178807],
           ...,
           [-0.091583, -0.047836, -0.274246, ...,  0.080244, -0.15178 ,
            -0.293459],
           [-0.689978, -0.655898, -0.360955, ..., -0.506291, -0.410355,
            -0.644327],
           [ 0.789417,  0.73178 ,  0.984611, ...,  0.860589,  0.83437 ,
             0.85403 ]]),
     'sigma_sqr': 0.3154750200244306}
#### CvAB3itWqXph3T14o6d_UbkBLep18b_xjZK2EA==.subsim:
    {'DeltaE': (0.1642543891423756, 0.0007341600399654619),
     'acceptance': 0.712351577,
     'correlations': array([10.9377031 , 10.77339587, 10.56721499, 10.39484992, 10.16995082,
            9.97831949,  9.85105586,  9.69581773,  9.54014091,  9.41363346,
            9.23274797,  9.0641524 ,  8.95952943,  8.83518594,  8.6630689 ,
            8.514677  ,  8.41009895,  8.21812222,  8.11401801,  7.99780325,
            7.93271684,  7.81957937,  7.61332471,  7.43437944,  7.28274233,
            7.1308915 ,  6.98880993,  6.86422495,  6.72402043,  6.64495341,
            6.56149532,  6.42162266,  6.35089503,  6.24745722,  6.14717583,
            6.10624863,  6.12677369,  5.97841198,  5.92626902,  5.88788027,
            5.85200028,  5.75918617,  5.64913024,  5.58438804,  5.44151676,
            5.39507246,  5.36556535,  5.23053065,  5.0686646 ,  5.03937619,
            4.95431786,  4.87558763,  4.8737357 ,  4.7202362 ,  4.60047705,
            4.55420295,  4.48716802,  4.40070682,  4.27758265,  4.20324632,
            4.17035552,  4.17297324,  4.12643857,  4.02650085,  3.99745293,
            3.89674831,  3.76009882,  3.65818016,  3.5913312 ,  3.55285829,
            3.46771986,  3.42544192,  3.41731442,  3.36919378,  3.2546157 ,
            3.19459776,  3.11170351,  3.10618961,  3.04809885,  2.94979989,
            2.94150758,  2.92435211,  2.91993534,  2.77976482,  2.77745296,
            2.7130108 ,  2.67603486,  2.61969437,  2.59401233,  2.50568484,
            2.50069797,  2.46392365,  2.42633523,  2.33863168,  2.25141121,
            2.22200718,  2.19386698,  2.10450319,  2.06746834,  2.07500154,
            2.06355842,  2.01211234,  1.98296009,  1.97267886,  1.9242266 ,
            1.88052178,  1.88509532,  1.84561821,  1.78364578,  1.77647045,
            1.71037766,  1.69747332,  1.67022307,  1.66160924,  1.65387748,
            1.64585591,  1.70080494,  1.71593066,  1.72986525,  1.68796845,
            1.73507359,  1.70001012,  1.6876383 ,  1.65489892,  1.76320488,
            1.7792889 ,  1.77492018,  1.77434183,  1.74216229,  1.77608314,
            1.72037745,  1.72202191,  1.68095424,  1.67657332,  1.68976739,
            1.7307393 ,  1.83119015,  1.8480325 ,  1.85044819,  1.76170797,
            1.80640336,  1.83976348,  1.81580296,  1.84444734,  1.75480133,
            1.7492537 ,  1.7253508 ,  1.77362552,  1.74645616,  1.74133126]),
     'measures': array([[-1.387554, -1.762092,  0.102325, ...,  0.013336, -0.864869,
            -1.012266],
           [ 2.152735,  2.153825,  2.458179, ...,  0.794028,  1.097514,
             1.385986],
           [ 3.346435,  3.071394,  3.350356, ...,  2.778983,  2.94159 ,
             3.148836],
           ...,
           [-1.808055, -1.576593, -1.710706, ..., -1.633212, -1.592631,
            -1.737503],
           [-2.313594, -2.830194, -3.169865, ..., -3.463917, -3.55547 ,
            -2.126313],
           [-2.49912 , -2.10893 , -2.102269, ..., -2.765038, -2.77622 ,
            -2.852264]]),
     'meff': (1.4826069255138725, 0.006626737741204891),
     'sigma_sqr': 5.741915148588372}
#### 9cCrmNu8TJkKUkrfo2SENmtSAPJvZg51RYrEBw==.subsim:
    {'acceptance': 0.28414545999999996,
     'measures': array([[-0.365589, -0.411653, -0.41463 , ..., -0.359916, -0.239281,
            -0.184795],
           [-0.511447, -0.516545, -0.556946, ..., -0.788212, -0.70449 ,
            -0.334912],
           [-0.033262, -0.217166, -0.306831, ..., -0.436292, -0.235875,
            -0.117769],
           ...,
           [-0.1269  , -0.130925, -0.170122, ...,  0.052688,  0.155555,
             0.078323],
           [ 0.047757, -0.017584, -0.461649, ..., -0.264504, -0.386441,
            -0.144827],
           [-0.593035, -0.47936 , -0.826773, ..., -0.406325, -0.290638,
            -0.390263]])}
#### YY_VYlCjiHRhwyGob0k5BN_D9kpvxxRuh0BuSw==.subsim:
    {'acceptance': 0.204623666,
     'measures': array([[-0.274923, -0.516901, -0.366196, ..., -0.532717, -0.480019,
            -0.327415],
           [-0.083527, -0.079607, -0.127166, ..., -0.044197, -0.247821,
            -0.060655],
           [ 0.262438,  0.298739,  0.320821, ...,  0.148197,  0.017885,
             0.119755],
           ...,
           [-0.22803 , -0.454911, -0.481704, ..., -0.105444,  0.012309,
            -0.164404],
           [-0.109068, -0.231305, -0.148235, ..., -0.363409, -0.259798,
            -0.259854],
           [-0.314745, -0.214588, -0.319766, ..., -0.244869, -0.195325,
            -0.162362]])}
#### mXg5Fb3IKR-adzZwMsP4xFrG0e4yMhR0tjcdqg==.subsim:
    {'acceptance': 0.190549572,
     'measures': array([[-0.32038 , -0.415887, -0.448534, ..., -0.436903, -0.405498,
            -0.453154],
           [-0.021147, -0.025818,  0.031172, ...,  0.110742,  0.024312,
             0.056435],
           [-0.434379, -0.425754, -0.405226, ..., -0.449992, -0.365549,
            -0.358115],
           ...,
           [-0.679448, -0.598507, -0.603189, ..., -1.000185, -0.887593,
            -0.836413],
           [-0.674567, -0.688133, -0.682037, ..., -0.59693 , -0.558214,
            -0.73985 ],
           [-0.805264, -0.92547 , -0.790064, ..., -0.969267, -0.953932,
            -0.823045]])}
#### BScl6-2utm3mu_5oobQmDdr3QMXjn_okq6mG2w==.subsim:
    {'acceptance': 0.3063221,
     'measures': array([[-0.393462, -0.418919, -0.476775, ..., -0.006592, -0.053155,
            -0.219179],
           [-0.387502, -0.143901, -0.023125, ..., -0.293068, -0.418172,
            -0.510466],
           [-0.449953, -0.423542, -0.443079, ..., -0.206637, -0.135722,
            -0.457897],
           ...,
           [ 0.120854,  0.082   , -0.219071, ...,  0.242952,  0.040411,
             0.282684],
           [-0.168463, -0.313998, -0.336353, ..., -0.347958, -0.297199,
            -0.048029],
           [ 0.593791,  0.525714,  0.576143, ...,  0.891109,  1.052534,
             0.722629]])}
#### i233F04XhpGVzRA4DpUioRxyGoxYKsILoBWJig==.simulator:
    <binary data>
#### 9YT3htNXaOxjHsa-hccf9LT6r-84e44olz92Nw==.subsim:
    {'acceptance': 0.27206154,
     'measures': array([[-0.05444 , -0.235876, -0.209126, ..., -0.215159, -0.276003,
            -0.205743],
           [ 0.737129,  0.810289,  0.572788, ...,  0.821783,  0.88272 ,
             0.780599],
           [ 0.633201,  0.427335,  0.56871 , ...,  0.545533,  0.478119,
             0.520018],
           ...,
           [-0.085749, -0.081405, -0.10492 , ..., -0.408905, -0.301825,
            -0.276609],
           [-0.120577, -0.299816, -0.200967, ..., -0.360177, -0.175625,
            -0.104301],
           [-0.356287, -0.157755, -0.221551, ..., -0.348978, -0.227494,
            -0.120875]])}
